In [1]:
# 업비트 자본 요청 부분 바꾸기 => 바이낸스 현물에서 단위 , 디미컬 라운딩 
# 수익 계산 구조 전부 바꾸기
# 업비트 입출금 정보 받는 부분 다 틀림.

# want_premium_step_2_2 에서 수익율 계산하는 부분 다시 보기.

# 모든 order 전에는 소수점을 맞춰주는 로직이 필요함.

# def up_avg_buy_price_info(): # @@ 나중에 이 부분 다 바꾸기
#     payload = {
#         'access_key': access_key,
#         'nonce': str(uuid.uuid4()), }

#     jwt_token = jwt.encode(payload, secret_key)
#     authorize_token = 'Bearer {}'.format(jwt_token)
#     headers = {"Authorization": authorize_token}

#     res = requests.get(server_url + "/v1/accounts", headers=headers)
#     return res.json()

In [2]:
import csv
import pickle  # 파이썬 딕셔너리 자료형 데이터 저장.
# ck
import datetime
# import pytictoc
import sys
import decimal  # 이것은 시간이 오래 걸리기 때문에 주문이 들어기지 않는 경우에만 사용하는 것이 좋다(전송 수량 찾을 때 굳)

sys.setrecursionlimit(1000000)  # 재귀함수 제한 늘리기
import pandas as pd
import numpy as np
import pip
import pyupbit  # 업비트#버젼1.1.7

print(pyupbit.Upbit)
import ccxt  # 바이낸스

print(ccxt)
import pprint
import time
import math
import sys
import requests
import json
# api json 파싱 python 업비트
import jwt  # PyJWT
import uuid
import requests
from urllib.parse import urlencode
from binance.client import Client # python-binance

import binance
import os
import jwt
import uuid
import hashlib
from urllib.parse import urlencode
import requests

########선물거래를 위해#######
from binance_f import RequestClient
from binance_f.constant.test import *
from binance_f.base.printobject import *
from binance_f.model.constant import *
from IPython.display import display, clear_output
from binance.spot import Spot #!pip install binance-connector

##############################소수점 정리....##############################################

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}


def upbit_get_usd_krw():
    try:
        url = 'https://quotation-api-cdn.dunamu.com/v1/forex/recent?codes=FRX.KRWUSD'
        exchange = requests.get(url, headers=headers).json()
        return exchange[0]['basePrice']
    except:
        pass


# https://docs.upbit.com/docs/market-info-trade-price-detail
################################원화 마켄 주문 최소 주문 가격 단위###########################
def UP_Minimum_Order_cash_Size(up_pre):  ##@@여기 전부 가격을 넣는 형식으로 바꿈.
    #     up_pre = pyupbit.get_current_price("KRW-" + symbol)  # 해당코인 가격조회
    # print(up_pre,"원화 최소 주문 가격 단위 조회")
    if (2000000 <= up_pre):
        return 1000
    if (1000000 <= up_pre < 2000000):
        return 500
    if (500000 <= up_pre < 1000000):
        return 100
    if (100000 <= up_pre < 500000):
        return 50
    if (10000 <= up_pre < 100000):
        return 10
    if (1000 <= up_pre < 10000):
        return 5
    if (100 <= up_pre < 1000):
        return 1
    if (10 <= up_pre < 100):
        return 0.1
    if (0 <= up_pre < 10):
        return 0.01


#########################################전송가능 심볼명#########################################
######심볼명 예외처리로 중복제거를 위해######
def removeAllOccur(l, i):
    try:
        while True: l.remove(i)
    except ValueError:
        pass


#########################################
def bin_fu_setting(send_to_bin_optimal_symbol, future_leverage):
    try:
        result = request_client.change_margin_type(symbol=send_to_bin_optimal_symbol + "USDT",
                                                   marginType=FuturesMarginType.ISOLATED)
    #         PrintBasic.print_obj(result)
    except:
        #         print("마진타입 변화 불필요")
        pass
    # 레버리지 변화 (자본에 따라 기본 10배)
    result = request_client.change_initial_leverage(symbol=send_to_bin_optimal_symbol + "USDT", leverage=future_leverage)


#     PrintBasic.print_obj(result)

def up_spot_ask_bid_info(sym_name):
    # 업비트 매수, 매도 호가창 한번에 조회
    up_sym_names = ["KRW-" + sym_name[i] for i in range(len(sym_name))]
    up_spot_ask_bid = pyupbit.get_orderbook(tickers=up_sym_names)
    return up_sym_names, up_spot_ask_bid


def up_ask_info(up_sym_names, up_spot_ask_bid):  # 업비트 매수 호가창
    up_ask_price = []
    up_ask_size = []
    up_ask_price_append = up_ask_price.append
    up_ask_size_append = up_ask_size.append
    for i in range(len(up_sym_names)):
        up_ask_price_append(float(next((item for item in up_spot_ask_bid if item['market'] == up_sym_names[i]), None)['orderbook_units'][0]['ask_price']))
        up_ask_size_append(float(next((item for item in up_spot_ask_bid if item['market'] == up_sym_names[i]), None)['orderbook_units'][0]['ask_size']))
    #         print(up_ask_price[i],up_ask_size[i],up_sym_names[i])
    return up_ask_price, up_ask_size


def up_bid_info(up_sym_names, up_spot_ask_bid):  # 업비트 매도 호가창
    up_bid_price = []
    up_bid_size = []
    up_bid_price_append = up_bid_price.append
    up_bid_size_append = up_bid_size.append
    for i in range(len(up_sym_names)):
        up_bid_price_append(float(next((item for item in up_spot_ask_bid if item['market'] == up_sym_names[i]), None)['orderbook_units'][0]['bid_price']))
        up_bid_size_append(float(next((item for item in up_spot_ask_bid if item['market'] == up_sym_names[i]), None)['orderbook_units'][0]['bid_size']))
    #         print(up_bid_price[i],up_bid_size[i],up_sym_names[i])
    return up_bid_price, up_bid_size


# up_sym_names, up_spot_ask_bid = up_spot_ask_bid_info(sym_name)
# up_ask_price, up_ask_size = up_ask_info(up_sym_names,up_spot_ask_bid)
# up_bid_price, up_bid_size= up_bid_info(up_sym_names,up_spot_ask_bid)

def bin_spot_ask_bid_info(sym_name):  # 바이낸스 현물 호가창 한번에 조회
    bin_sym_names = [sym_name[i] + "USDT" for i in range(len(sym_name))]
    bin_spot_ask_bid = client_spot.book_ticker()
    return bin_sym_names, bin_spot_ask_bid


def bin_ask_info(bin_sym_names, bin_spot_ask_bid):  # 바이낸스 현물 호가창 조회로 매수 호가창 가져옴
    bin_spot_ask_price = []
    bin_spot_ask_size = []
    bin_spot_ask_price_append = bin_spot_ask_price.append
    bin_spot_ask_size_append = bin_spot_ask_size.append
    for i in range(len(bin_sym_names)):
        bin_spot_ask_price_append(float(next((item for item in bin_spot_ask_bid if item['symbol'] == bin_sym_names[i]), None)['askPrice']))
        bin_spot_ask_size_append(float(next((item for item in bin_spot_ask_bid if item['symbol'] == bin_sym_names[i]), None)['askQty']))
    #         print(bin_spot_ask_price[i],bin_spot_ask_size[i],bin_sym_names[i])
    return bin_spot_ask_price, bin_spot_ask_size


def bin_bid_info(bin_sym_names, bin_spot_ask_bid):  # 바이낸스 현물 호가창 조회로 매도 호가창 가져옴
    bin_spot_bid_price = []
    bin_spot_bid_size = []
    bin_spot_bid_price_append = bin_spot_bid_price.append
    bin_spot_bid_size_append = bin_spot_bid_size.append

    for i in range(len(bin_sym_names)):
        bin_spot_bid_price_append(float(next((item for item in bin_spot_ask_bid if item['symbol'] == bin_sym_names[i]), None)['bidPrice']))
        bin_spot_bid_size_append(float(next((item for item in bin_spot_ask_bid if item['symbol'] == bin_sym_names[i]), None)['bidQty']))
    #         print(bin_spot_bid_price[i],bin_spot_bid_size[i],bin_sym_names[i])
    return bin_spot_bid_price, bin_spot_bid_size


# bin_sym_names, bin_spot_ask_bid = bin_spot_ask_bid_info(sym_name)
# bin_spot_ask_price,bin_spot_ask_size = bin_ask_info(bin_sym_names,bin_spot_ask_bid)
# bin_spot_bid_price,bin_spot_bid_size = bin_bid_info(bin_sym_names,bin_spot_ask_bid)

def bin_fu_ask_bid_info(sym_name):
    bin_sym_names = [sym_name[i] + "USDT" for i in range(len(sym_name))]
    bin_fu_ask_bid = client.futures_orderbook_ticker()

    return bin_sym_names, bin_fu_ask_bid


def bin_fu_ask_info(bin_sym_names, bin_fu_ask_bid):
    bin_fu_ask_price = []
    bin_fu_ask_size = []
    bin_fu_ask_price_append = bin_fu_ask_price.append
    bin_fu_ask_size_append = bin_fu_ask_size.append
    for i in range(len(bin_sym_names)):
        bin_fu_ask_price_append(float(next((item for item in bin_fu_ask_bid if item['symbol'] == bin_sym_names[i]), None)['askPrice']))
        bin_fu_ask_size_append(float(next((item for item in bin_fu_ask_bid if item['symbol'] == bin_sym_names[i]), None)['askQty']))
    #         print(bin_fu_ask_price[i],bin_fu_ask_size[i],bin_sym_names[i])
    return bin_fu_ask_price, bin_fu_ask_size


def bin_fu_bid_info(bin_sym_names, bin_fu_ask_bid):
    bin_fu_bid_price = []
    bin_fu_bid_size = []
    bin_fu_bid_price_append = bin_fu_bid_price.append
    bin_fu_bid_size_append = bin_fu_bid_size.append

    for i in range(len(bin_sym_names)):
        bin_fu_bid_price_append(float(next((item for item in bin_fu_ask_bid if item['symbol'] == bin_sym_names[i]), None)['bidPrice']))
        bin_fu_bid_size_append(float(next((item for item in bin_fu_ask_bid if item['symbol'] == bin_sym_names[i]), None)['bidQty']))
    #         print(bin_fu_bid_price[i],bin_fu_bid_size[i],bin_sym_names[i])
    return bin_fu_bid_price, bin_fu_bid_size


# bin_sym_names,bin_fu_ask_bid = bin_fu_ask_bid_info(sym_name)
# bin_fu_ask_price,bin_fu_ask_size = bin_fu_ask_info(sym_name,bin_fu_ask_bid)
# bin_fu_bid_price,bin_fu_bid_size = bin_fu_bid_info(sym_name,bin_fu_ask_bid)

def find_max_per(second_using_asset, sym_name, bin_spot_bid_price, bin_withdrawfee):
    max_per = []
    for i in range(len(sym_name)):
        max_per.append(bin_spot_bid_price[i] * bin_withdrawfee[i] / ((second_using_asset / Usdt) / 2))

    return max_per[max_per.index(max(max_per))]


def Adjusting_second_using_asset(second_using_asset, up_market_fee, bin_spot_market_fee, bin_future_market_fee, per):
    a = (1 - bin_future_market_fee)  # 바
    b = (1 - bin_spot_market_fee) * (1 - up_market_fee) * (1 - per)  # *(1-0.0004)#@ 0.0004는 추가 보정치 #업

    c = a / (a + b)  # 업
    d = b / (a + b)  # 바

    bin_spot_Usdt_Send_to_up = second_using_asset * c
    bin_fu_Usdt_asset = second_using_asset * d

    return bin_spot_Usdt_Send_to_up, bin_fu_Usdt_asset


# def Adjusting_first_using_asset(sym_name,up_bid_price,up_withdrawfee,up_asset,binance_asset,up_market_fee,bin_spot_market_fee,bin_future_market_fee):
#     max_per = []
#     for i in range(len(sym_name)):
#         max_per.append(up_bid_price[i]* up_withdrawfee[i])
#     max_val= max_per[max_per.index(max(max_per))]

#     a = (1- up_market_fee) * (1-(max_val/up_asset)) *(1- bin_spot_market_fee)
#     b = (1- bin_future_market_fee)

#     c = (a / (a + b)) * (up_asset + binance_asset)
#     d = (b / (a + b)) * (up_asset + binance_asset)
# #     print(c,d)
#     return c,d #바낸자본 업비트 자본

def up_to_bin_canwithdrow_fee(sym_name):
    server_url = 'https://api.upbit.com'
    query = {'currency': sym_name, }
    query_string = urlencode(query).encode()

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {'access_key': access_key, 'nonce': str(uuid.uuid4()), 'query_hash': query_hash,
               'query_hash_alg': 'SHA512', }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/withdraws/chance", params=query, headers=headers)
    # 해당 코인 전송수수료(해당코인 시가 * 전송 수수료)
    symbol_fee = float(res.json()['currency']['withdraw_fee'])
    #         print(symbol_fee)
    #     upbit_withdraw_fee = float(res.json()['currency']['withdraw_fee']) * up_pre
    #         print(upbit_withdraw_fee,"업비트 보낼 시 심볼 전송 가격")

    bin_sym_send_state = res.json()['withdraw_limit']['can_withdraw']  # 출금지원가능여부

    if (bin_sym_send_state == True):
        pass
    else:
        print(sym_name, "해당코인 업비트에서 바이낸스로 출금 불가상태")

    return symbol_fee, bin_sym_send_state


def up_deposit_address(sym_name):
    server_url = 'https://api.upbit.com'
    ############추가 ... 입출금 상태는 주소로 받아야댐..
    try:
        query = {'currency': sym_name, }
        query_string = urlencode(query).encode()

        m = hashlib.sha512()
        m.update(query_string)
        query_hash = m.hexdigest()

        payload = {'access_key': access_key,
                   'nonce': str(uuid.uuid4()),
                   'query_hash': query_hash,
                   'query_hash_alg': 'SHA512', }

        jwt_token = jwt.encode(payload, secret_key)
        authorize_token = 'Bearer {}'.format(jwt_token)
        headers = {"Authorization": authorize_token}

        res = requests.post(server_url + "/v1/deposits/generate_coin_address", params=query, headers=headers)
        up_add = res.json()['deposit_address']
        bin_sym_send_state_2 = True
        up_seadd = res.json()['secondary_address']

        return up_add, bin_sym_send_state_2, up_seadd
    # print(up_add, up_seadd)

    except KeyError:
        up_add = "해당코인 바이낸스에서 업비트 입금 불가상태"
        up_seadd = "해당코인 바이낸스에서 업비트 입금 불가상태"
        bin_sym_send_state_2 = False
        print(sym_name, "해당코인 바이낸스에서 업비트 입금 불가상태")
        return up_add, bin_sym_send_state_2, up_seadd



<class 'pyupbit.exchange_api.Upbit'>
<module 'ccxt' from 'c:\\users\\happy\\appdata\\local\\programs\\python\\python37\\lib\\site-packages\\ccxt\\__init__.py'>


In [3]:
def up_deposit_address_2(sym_name):
    #### 업비트 해당코인 입금 주소 생성요청
    try:
        query = {'currency': sym_name, }
        query_string = urlencode(query).encode()

        m = hashlib.sha512()
        m.update(query_string)
        query_hash = m.hexdigest()

        payload = {
            'access_key': access_key,
            'nonce': str(uuid.uuid4()),
            'query_hash': query_hash,
            'query_hash_alg': 'SHA512', }

        jwt_token = jwt.encode(payload, secret_key)
        authorize_token = 'Bearer {}'.format(jwt_token)
        headers = {"Authorization": authorize_token}

        res = requests.post(server_url + "/v1/deposits/generate_coin_address", params=query,
                            headers=headers)
        up_add = res.json()['deposit_address']
        up_sym_send_state = True
        up_seadd = res.json()['secondary_address']
        # print(up_add, up_seadd)
        return up_add, up_sym_send_state, up_seadd

    except KeyError:
        up_add = "해당코인 바이낸스에서 업비트 입금 불가상태"
        up_seadd = "해당코인 바이낸스에서 업비트 입금 불가상태"
        up_sym_send_state = False
        print(sym_name, "해당코인 바이낸스에서 업비트 입금 불가상태")
        return up_add, up_sym_send_state, up_seadd


def up_bin_withdraw_fee(sym_name, all_coin_info):
    # 심볼명,달러 등은 전부 수수료를 저장해두고 나중에 갱신.
    up_withdrawfee = []
    bin_withdrawfee = []
    up_withdrawfee_append = up_withdrawfee.append
    bin_withdrawfee_append = bin_withdrawfee.append
#     all_coin_info = client_spot.coin_info()
    for i in range(len(sym_name)):
        aa = next((item for item in all_coin_info if item['coin'] == sym_name[i]), None)['networkList']
        bin_withdrawfee_append(float(next((item for item in aa if item['network'] == sym_name[i]), None)['withdrawFee']))
        symbol_fee, bin_sym_send_state = up_to_bin_canwithdrow_fee(sym_name[i])
        up_withdrawfee_append(symbol_fee)
        time.sleep(0.4)
    return up_withdrawfee, bin_withdrawfee


def bin_withdraw_fee(all_coin_info, sym_name):
    aa = next((item for item in all_coin_info if item['coin'] == sym_name), None)['networkList']
    bin_withdrawfee = (float(next((item for item in aa if item['network'] == sym_name), None)['withdrawFee']))
    return bin_withdrawfee


def sym_intersection():
    ######바이낸스 달러 선물가능 심볼명#####
    bin_fu_sym = client.futures_symbol_ticker()
    for i in range(len(bin_fu_sym)):
        if (bin_fu_sym[i]['symbol'][-4:] == 'USDT'):
            bin_fu_sym[i] = bin_fu_sym[i]['symbol'][:-4]
        else:
            bin_fu_sym[i] = 'remove'
    removeAllOccur(bin_fu_sym, 'remove')
    ########################################
    #####바이낸스 달러 현물가능 코인 심볼명#USDT만 찾기 #####
    bin_sym = client.get_symbol_ticker()

    for i in range(len(bin_sym)):
        if (bin_sym[i]['symbol'][-4:] == 'USDT'):
            bin_sym[i] = bin_sym[i]['symbol'][:-4]
        else:
            bin_sym[i] = 'remove'
    removeAllOccur(bin_sym, 'remove')
    ########################################################
    ######업비트 코인 심볼명########
    up_sym = pyupbit.get_tickers(fiat="KRW")
    for i in range(len(up_sym)):
        # tickers[i].replace('KRW-','')
        up_sym[i] = up_sym[i][4:]
    #################################
    ######심볼명 교집합#####
    # sym_name = list(set(bin_sym) & set(up_sym))  # @
    sym_name = list(set(bin_fu_sym) & set(bin_sym) & set(up_sym))
    print(len(sym_name), len(sym_name))  # 선물없이 업비트만해선..
    return sym_name


def bin_coin_withdrow_fee(all_coin_info, sym_name):
    try:
        aa = next((item for item in all_coin_info if item['coin'] == sym_name), None)['networkList']
        withdrawfee = float(next((item for item in aa if item['network'] == sym_name), None)['withdrawFee'])
        return withdrawfee
    except:
        withdrawfee = "None_network"
        return withdrawfee
        pass


def bin_coin_withdrow_deopsit_Enable(all_coin_info, sym_name):
    try: #자기자신의 네트워크를 가지고, 현재 바이낸스의 입출금 가능상태를 가져옴
        aa = next((item for item in all_coin_info if item['coin'] == sym_name), None)['networkList']
        bb = next((item for item in aa if item['network'] == sym_name), None)
        depositEnable, withdrawEnable = bb['depositEnable'] , bb['withdrawEnable']
    except:
        depositEnable, withdrawEnable = False ,False

    return depositEnable, withdrawEnable


def del_bin_disable_withdrow_deopsit(all_coin_info, sym_name):
    delet_list = []
    delet_list_append = delet_list.append
    for i in range(len(sym_name)):  # 바이낸스에서 입금 가능 , 전송 불가능
        depositEnable, withdrawEnable = bin_coin_withdrow_deopsit_Enable(all_coin_info, sym_name[i])
        #         print(sym_name[i],depositEnable,withdrawEnable)
        if (depositEnable == False or withdrawEnable == False):
            delet_list_append(sym_name[i])
    name = list(set(sym_name) - set(delet_list))
    return name

def delet_None_network(all_coin_info, sym_name):
    delet_list = []
    delet_list_append = delet_list.append
    for i in range(len(sym_name)):  # 바이낸스에서 전송
        withdrawfee = bin_coin_withdrow_fee(all_coin_info, sym_name[i])
        #         print(sym_name[i],withdrawfee)
        if (withdrawfee == "None_network"):
            delet_list_append(sym_name[i])
    name = list(set(sym_name) - set(delet_list))
    return name



def regester_by_upbit_withdrow_addres(sym_name):
    for i in range(len(sym_name)):
#         print(sym_name[i])
        try:
            bin_address = client_spot.deposit_address(sym_name[i])['address']
            bin_Memo = client_spot.deposit_address(sym_name[i])['tag']
            time.sleep(0.1)
            print(sym_name[i])
            print('bin_address', bin_address)
            print('bin_Memo', bin_Memo)
        except:
            print(sym_name[i],"바이낸스 지갑이 닫혔을 수도 있음")


# def Adjusting_second_using_asset(second_using_asset,up_market_fee,bin_spot_market_fee,bin_future_market_fee):
#     a = (1- bin_future_market_fee) #바
#     b = (1- bin_spot_market_fee) * (1- up_market_fee) #업

#     c = a / (a+b) #업
#     d = b / (a+b) #바

#     bin_spot_Usdt_Send_to_up = second_using_asset * c
#     bin_fu_Usdt_asset = second_using_asset * d
#     return bin_spot_Usdt_Send_to_up , bin_fu_Usdt_asset

def Bin_Minimum_Spot_Order_Size(sym_name):
    Bin_Spot_Minimum_Order_Size = []
    Bin_Spot_Minimum_Order_Size_append = Bin_Spot_Minimum_Order_Size.append
    aa = client.get_exchange_info()

    for i in range(len(sym_name)):
        bb = next((item for item in aa['symbols'] if item['symbol'] == sym_name[i] + 'USDT'), None)
        Bin_Spot_Minimum_Order_Size_append(float(next((item for item in bb['filters'] if item['filterType'] == 'LOT_SIZE'), None)['minQty']))
    #     print(Bin_Spot_Minimum_Order_Size)
    return Bin_Spot_Minimum_Order_Size


def Bin_Minimum_FU_Order_Size(sym_name):
    Bin_Fu_Minimum_Order_Size = []
    Bin_Fu_Minimum_Order_Size_append = Bin_Fu_Minimum_Order_Size.append
    aa = client.futures_exchange_info()
    
    for i in range(len(sym_name)):
        bb = next((item for item in aa['symbols'] if item['symbol'] == sym_name[i] + 'USDT'), None)
        Bin_Fu_Minimum_Order_Size_append(float(next((item for item in bb['filters'] if item['filterType'] == 'MARKET_LOT_SIZE'), None)['minQty']))
    #     print(Bin_Fu_Minimum_Order_Size)
    return Bin_Fu_Minimum_Order_Size


def Bin_Minimum_Withdraw_Size(symbol,all_coin_info):  # 바이낸스 최소전송가능량
    aa = next((item for item in all_coin_info if item['coin'] == symbol), None)['networkList']
    minWithdrawAmount = float(next((item for item in aa if item['network'] == symbol), None)['withdrawIntegerMultiple'])
    return minWithdrawAmount


def del_bin_disable_deopsit(all_coin_info, sym_name):  # 바이낸스 입금 불가능 코인 삭제
    delet_list = []
    delet_list_append = delet_list.append
    for i in range(len(sym_name)):
        depositEnable, withdrawEnable = bin_coin_withdrow_deopsit_Enable(all_coin_info, sym_name[i])
        #         print(sym_name[i],depositEnable,withdrawEnable)
        if (depositEnable == False):
            delet_list_append(sym_name[i])
    name = list(set(sym_name) - set(delet_list))
    return name


def del_bin_disable_withdraw(all_coin_info, sym_name):  # 바이낸스 출금 불가능 심볼 삭제
    delet_list = []
    delet_list_append = delet_list.append
    for i in range(len(sym_name)):
        depositEnable, withdrawEnable = bin_coin_withdrow_deopsit_Enable(all_coin_info, sym_name[i])
        #         print(sym_name[i],depositEnable,withdrawEnable)
        if (withdrawEnable == False):
            delet_list_append(sym_name[i])
    name = list(set(sym_name) - set(delet_list))
    return name


def del_up_disable_deopsit(sym_name):  # 업비트 입금 불가능 코인 삭제
    server_url = 'https://api.upbit.com'
    delet_list = []
    for i in range(len(sym_name)):

        query = {'currency': sym_name[i], }
        query_string = urlencode(query).encode()

        m = hashlib.sha512()
        m.update(query_string)
        query_hash = m.hexdigest()

        payload = {'access_key': access_key, 'nonce': str(uuid.uuid4()), 'query_hash': query_hash, 'query_hash_alg': 'SHA512', }

        jwt_token = jwt.encode(payload, secret_key)
        authorize_token = 'Bearer {}'.format(jwt_token)
        headers = {"Authorization": authorize_token}

        res = requests.get(server_url + "/v1/withdraws/chance", params=query, headers=headers)
        up_sym_whthdraw_state = res.json()['currency']['wallet_support']

        if 'deposit' in up_sym_whthdraw_state:
            pass
        else:
            delet_list.append(sym_name[i])
            print(sym_name[i], "해당코인 업비트에서 바이낸스로 입금 불가상태")

        time.sleep(0.4)
    name = list(set(sym_name) - set(delet_list))
    return name

def del_up_disable_whthdraw(sym_name):  # 업비트 출금 불가능 코인 삭제
    server_url = 'https://api.upbit.com'
    delet_list = []
    for i in range(len(sym_name)):

        query = {'currency': sym_name[i], }
        query_string = urlencode(query).encode()

        m = hashlib.sha512()
        m.update(query_string)
        query_hash = m.hexdigest()

        payload = {'access_key': access_key, 'nonce': str(uuid.uuid4()), 'query_hash': query_hash, 'query_hash_alg': 'SHA512', }

        jwt_token = jwt.encode(payload, secret_key)
        authorize_token = 'Bearer {}'.format(jwt_token)
        headers = {"Authorization": authorize_token}

        res = requests.get(server_url + "/v1/withdraws/chance", params=query, headers=headers)
        up_sym_whthdraw_state = res.json()['currency']['wallet_support']

        if 'withdraw' in up_sym_whthdraw_state:
            pass
        else:
            delet_list.append(sym_name[i])
            print(sym_name[i], "해당코인 업비트에서 바이낸스로 출금 불가상태")

        time.sleep(0.4)
    name = list(set(sym_name) - set(delet_list))
    return name


#####업비트 => 바이낸스 전송가능정보와 코인 수수료 가져오는코드#########################
def Up_Minimum_Withdraw_Size(sym_name):  # 업비트 최소전송가능량
    server_url = 'https://api.upbit.com'
    query = {'currency': sym_name, }
    query_string = urlencode(query).encode()

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {'access_key': access_key, 'nonce': str(uuid.uuid4()), 'query_hash': query_hash,
               'query_hash_alg': 'SHA512', }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/withdraws/chance", params=query, headers=headers)

    return (0.1) ** float(res.json()['withdraw_limit']['fixed'])
    #         print(bin_sym_send_state,sym_name[i],"출금지원가능여부")


def up_bin_withdraw_Size(sym_name, all_coin_info):  # 바이낸스 최소전송가능량
    up_Min_Withdraw_Size = []
    bin_Min_Withdraw_Size = []
    up_Min_Withdraw_Size_append = up_Min_Withdraw_Size.append
    bin_Min_Withdraw_Size_append = bin_Min_Withdraw_Size.append
    for i in range(len(sym_name)):  # 잘되나 확인 중 나중에 지우기
        up_Min_Withdraw_Size_append(Up_Minimum_Withdraw_Size(sym_name[i]))
        bin_Min_Withdraw_Size_append(Bin_Minimum_Withdraw_Size(sym_name[i],all_coin_info))
        time.sleep(0.4)
    return up_Min_Withdraw_Size, bin_Min_Withdraw_Size



def remove_sym_name_1(all_coin_info, sym_name, Usdt, up_asset, per, ban_list, up_market_fee):  # 심볼명 리스트, 달러 환율, 자본대비 최소 퍼센테이지

    up_withdrawfee, bin_withdrawfee = up_bin_withdraw_fee(sym_name, all_coin_info)  # 업비트 전송 수수료 , 바이낸스 전송 수수료 #바이낸스 api 1번 요총
    up_Min_Withdraw_Size, bin_Min_Withdraw_Size = up_bin_withdraw_Size(sym_name,all_coin_info)  # 업비트, 바이낸스 전송최소 가능량 #업비트 심볼수만큼 api 요청

    # 자본대비 수수료 , 잡다한 코인들 제외

    up_sym_names, up_spot_ask_bid = up_spot_ask_bid_info(sym_name)
    #     up_ask_price, up_ask_size = up_ask_info(up_sym_names, up_spot_ask_bid)  # 업비트 매수 호가창 4
    up_bid_price, up_bid_size = up_bid_info(up_sym_names, up_spot_ask_bid)  # 업비트 매도 호가창 1

    for i in range(len(sym_name)):

        up_can_order_amout = up_asset * (1 - up_market_fee)  # *0.005이미함
        up_can_order_amout = up_can_order_amout + (up_withdrawfee[i] * up_bid_price[i])  ##완벽한 헷징을 위해 업비트 전송료만큼 추가..#@@이부분..계산
        Decimal_rounding = UP_Minimum_Order_cash_Size(up_bid_price[i])
        up_can_order_amout = (up_can_order_amout // Decimal_rounding) * Decimal_rounding

        can_send_Decimal = up_Min_Withdraw_Size[i]  # 전송가능 최소량@@이게 잇어야 계산 가능
        if (can_send_Decimal == 0):  # 전송 수수료가 0 인경우 나눌 수가 없음
            pass
        else:
            up_can_order_amout = (up_can_order_amout // can_send_Decimal) * can_send_Decimal

        if (((up_bid_price[i] * up_withdrawfee[i]) / up_can_order_amout) < per):  # 구매가능 정보로 해서 더 깔끔(소수점 절단)
            pass
        else:
            print((up_bid_price[i] * up_withdrawfee[i]) / up_can_order_amout, sym_name[i], "자본대비 수수료가 큰 코인")
            print(up_bid_price[i] * up_withdrawfee[i])
            sym_name[i] = 'remove'

        ###@@@ BNB코인 배제 이 부분 로직으로 대체 가능
        if ((sym_name[i] in ban_list) == True):
            sym_name[i] = 'remove'

    removeAllOccur(sym_name, 'remove')
    print(sym_name, len(sym_name))
    return sym_name

def remove_sym_name_2(all_coin_info, sym_name, Usdt, binance_asset, per, ban_list, bin_spot_market_fee):  # 심볼명 리스트, 달러 환율, 자본대비 최소 퍼센테이지

    up_withdrawfee, bin_withdrawfee = up_bin_withdraw_fee(sym_name, all_coin_info)  # 업비트 전송 수수료 , 바이낸스 전송 수수료
    up_Min_Withdraw_Size, bin_Min_Withdraw_Size = up_bin_withdraw_Size(sym_name,all_coin_info)  # 업비트, 바이낸스 전송최소 가능량
    # Bin_Fu_Minimum_Order_Size = Bin_Minimum_FU_Order_Size(sym_name)  # 바이낸스 선물 최소 주문 가능량
    Bin_Spot_Minimum_Order_Size = Bin_Minimum_Spot_Order_Size(sym_name)  # 바이낸스 현물 최소 주문가능량
    # 자본대비 수수료 , 잡다한 코인들 제외

    bin_sym_names, bin_spot_ask_bid = bin_spot_ask_bid_info(sym_name)
    bin_spot_bid_price, bin_spot_bid_size = bin_bid_info(bin_sym_names, bin_spot_ask_bid)  # 바-현 매도 호가창 3

    for i in range(len(sym_name)):

        bin_spot_Usdt_Send = binance_asset - binance_asset * bin_spot_market_fee  # 바이낸스 현물 시장가 수수료
        send_to_up_quantity = (bin_spot_Usdt_Send / bin_spot_bid_price[i])  # 바이낸스 업비트로 보낼 달러 / 티커 가격 = 수량
        send_to_up_quantity = send_to_up_quantity + bin_withdrawfee[i]
        # 전송가능 최소량과 주문가능 최소량으로 잘 자름.
        Decimal_rounding = bin_Min_Withdraw_Size[i]
        if (Decimal_rounding == 0):  # 전송 수수료가 0 인경우 나눌 수가 없음
            pass
        else:
            send_to_up_quantity = (send_to_up_quantity // Decimal_rounding) * Decimal_rounding

        Decimal_rounding = Bin_Spot_Minimum_Order_Size[i]
        bin_spot_to_up_can_order_amount = (send_to_up_quantity // Decimal_rounding) * Decimal_rounding
        bin_spot_to_up_can_order_amount = round(bin_spot_to_up_can_order_amount, 6)
        print(sym_name[i], bin_spot_to_up_can_order_amount, (bin_spot_bid_price[i] * bin_withdrawfee[i]) / (binance_asset), bin_spot_bid_price[i])

        if (((bin_spot_bid_price[i] * bin_withdrawfee[i]) / (binance_asset)) < per):  # 구매가능 정보로 해서 더 깔끔(소수점 절단)
            #             print((up_bid_price[i] * up_withdrawfee[i]) / up_can_order_amout, sym_name[i])
            pass

        else:
            print((bin_spot_bid_price[i] * bin_withdrawfee[i]) / (binance_asset), "자본대비 수수료가 큰 코인")
            print(bin_spot_bid_price[i] * bin_withdrawfee[i] * Usdt)
            sym_name[i] = 'remove'

        ###@@@ BNB코인 배제 이 부분 로직으로 대체 가능
        if ((sym_name[i] in ban_list) == True):
            sym_name[i] = 'remove'

    removeAllOccur(sym_name, 'remove')
    print(sym_name, len(sym_name))
    return sym_name

# 업비트 매수 평단가
def up_avg_buy_price(sym_name):
    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()), }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/accounts", headers=headers)
    aa = float(next((item for item in res.json() if item['currency'] == sym_name), None)['avg_buy_price'])
    bb = float(next((item for item in res.json() if item['currency'] == sym_name), None)["balance"])
    return aa, bb


def bin_fu_avg_buy_price(sym_name):
    aa = float(next((item for item in client.futures_account()['positions'] if item['symbol'] == sym_name + 'USDT'), None)['entryPrice'])
    bb = -(float(next((item for item in client.futures_account()['positions'] if item['symbol'] == sym_name + 'USDT'), None)["positionAmt"]))
    return aa, bb


def up_withdrow_request(send_to_bin_optimal_simbol, point_of_Withdrow_UP_amount_1, bin_address, bin_Memo):
    # @ 2.코인 출금 요청#########################
    # point_of_Withdrow_UP_amount_1 조심 가져오든지 해야함
    query = {
        'currency': send_to_bin_optimal_simbol,
        'amount': point_of_Withdrow_UP_amount_1,
        'address': bin_address,
        'secondary_address': bin_Memo,
        'transaction_type': 'default'  # default가 일반출금
    }
    query_string = urlencode(query).encode()

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()),
        'query_hash': query_hash,
        'query_hash_alg': 'SHA512', }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.post(server_url + "/v1/withdraws/coin", params=query, headers=headers)

    print(res.json())
    send_amount = res.json()['amount']
    send_amount = float(send_amount)
    print(send_amount)


# 업비트 입금주소 요청
def up_deposit_address_request(send_to_up_optimal_simbol):
    query = {'currency': send_to_up_optimal_simbol, }
    query_string = urlencode(query).encode()

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {'access_key': access_key,
               'nonce': str(uuid.uuid4()),
               'query_hash': query_hash,
               'query_hash_alg': 'SHA512', }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.post(server_url + "/v1/deposits/generate_coin_address", params=query,
                        headers=headers)
    up_add = res.json()['deposit_address']
    up_seadd = res.json()['secondary_address']
    return up_add, up_seadd


from binance_d import RequestClient as RequestClient_D
from binance_d.constant.test import *
from binance_d.base.printobject import *


##변화한 함수들
def Up_Minimum_Withdraw_Size_digit(sym_name):  # 자릿수
    server_url = 'https://api.upbit.com'
    query = {'currency': sym_name, }
    query_string = urlencode(query).encode()

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {'access_key': access_key, 'nonce': str(uuid.uuid4()), 'query_hash': query_hash,
               'query_hash_alg': 'SHA512', }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/withdraws/chance", params=query, headers=headers)

    return float(res.json()['withdraw_limit']['fixed'])  # 이부분 틀려있었음


###########################안쓰이는듯?
def up_avg_buy_price_USD(sym_name):
    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()), }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/accounts", headers=headers)
    aa = float(next((item for item in res.json() if item['currency'] == sym_name), None)['avg_buy_price'])
    bb = float(next((item for item in res.json() if item['currency'] == sym_name), None)["balance"])
    return aa, bb


def bin_fu_avg_buy_price_USD(sym_name):
    aa = float(next((item for item in client.futures_account()['positions'] if item['symbol'] == sym_name + 'USD'), None)['entryPrice'])
    bb = -(float(next((item for item in client.futures_account()['positions'] if item['symbol'] == sym_name + 'USD'), None)["positionAmt"]))
    return aa, bb


def bin_fu_D_ask_info(bin_sym_names, bin_fu_ask_bid):  # 선물
    bin_fu_D_ask_price = []
    bin_fu_D_ask_size = []
    for i in range(len(bin_sym_names)):
        bin_fu_D_ask_price.append(float(next((item for item in bin_fu_ask_bid if item['symbol'] == bin_sym_names[i]), None)['askPrice']))
        bin_fu_D_ask_size.append(float(next((item for item in bin_fu_ask_bid if item['symbol'] == bin_sym_names[i]), None)['askQty']))
    #         print(bin_fu_ask_price[i],bin_fu_ask_size[i],bin_sym_names[i])
    return bin_fu_D_ask_price, bin_fu_D_ask_size

def Bin_Fu_Minimum_Order_Cont_n_Size_D(sym_name_D):
    aa = request_client_D.get_exchange_information()  # Coin-M 최소 주문량 가져옮(lot_size)
    bin_fu_Minimum_Order_Cont_n_Size_D = {}
    Coin_M_Cont = []
    Coin_M_Min_Cont = []

    for i in range(len(aa.symbols)):
        bin_fu_Minimum_Order_Cont_n_Size_D[aa.symbols[i].symbol] = {"cont": aa.symbols[i].contractSize, "minQty": float(next((item for item in aa.symbols[i].filters if item['filterType'] == 'MARKET_LOT_SIZE'), None)['minQty'])}

    for i in range(len(sym_name_D)):
        Coin_M_Cont.append(bin_fu_Minimum_Order_Cont_n_Size_D[sym_name_D[i] + 'USD_PERP']["cont"])
        Coin_M_Min_Cont.append(bin_fu_Minimum_Order_Cont_n_Size_D[sym_name_D[i] + 'USD_PERP']['minQty'])

    return Coin_M_Cont, Coin_M_Min_Cont


def bin_fu_avg_buy_price_D(sym_name_D):
    aa = request_client_D.get_position()

    bin_sym_info_D = {}
    EntryPrice = []
    PositionAmt = []

    for i in range(len(aa)):
        bin_sym_info_D[aa[i].symbol] = {'entryPrice': aa[i].entryPrice, 'positionAmt': aa[i].positionAmt}

    for i in range(len(sym_name_D)):
        EntryPrice.append(bin_sym_info_D[sym_name_D[i] + 'USD_PERP']["entryPrice"])
        PositionAmt.append(bin_sym_info_D[sym_name_D[i] + 'USD_PERP']['positionAmt'])

    return EntryPrice, PositionAmt

def sym_intersection_D():
    ######바이낸스 달러 선물 가능 심볼명#####
    bin_fu_sym = client.futures_symbol_ticker()
    for i in range(len(bin_fu_sym)):
        if (bin_fu_sym[i]['symbol'][-4:] == 'USDT'):
            bin_fu_sym[i] = bin_fu_sym[i]['symbol'][:-4]
        else:
            bin_fu_sym[i] = 'remove'
    removeAllOccur(bin_fu_sym, 'remove')
    ########################################
    #####바이낸스 달러 현물 가능 코인 심볼명#USDT만 찾기 #####
    bin_sym = client.get_symbol_ticker()

    for i in range(len(bin_sym)):
        if (bin_sym[i]['symbol'][-4:] == 'USDT'):
            bin_sym[i] = bin_sym[i]['symbol'][:-4]
        else:
            bin_sym[i] = 'remove'
    removeAllOccur(bin_sym, 'remove')
    ########################################################
    ######업비트 코인 심볼명########
    up_sym = pyupbit.get_tickers(fiat="KRW")
    for i in range(len(up_sym)):
        # tickers[i].replace('KRW-','')
        up_sym[i] = up_sym[i][4:]
    #################################
    #####바이낸스 Coin-M ##############################
    bin_fu_D_sym = request_client_D.get_symbol_orderbook_ticker()
    bin_fu_D_sym_name = []
    for i in range(len(bin_fu_D_sym)):
        if (bin_fu_D_sym[i].symbol[-8:] == 'USD_PERP'):
            bin_fu_D_sym_name.append(bin_fu_D_sym[i].symbol[:-8])
        else:
            bin_fu_D_sym_name.append('remove')
    removeAllOccur(bin_fu_D_sym_name, 'remove')
    ###################################################
    ######심볼명 교집합#####
    # sym_name = list(set(bin_sym) & set(up_sym))  # @
    sym_name = list(set(bin_fu_sym) & set(bin_sym) & set(up_sym) & set(bin_fu_D_sym_name))
    print(len(sym_name), len(sym_name))  # 선물없이 업비트만해선..
    return sym_name

def Coin_M_leverage_A_margin(sym_name):
    for i in range(len(sym_name)):
        ##레버리지 변환
        try:
            result = request_client_D.change_initial_leverage(symbol=sym_name[i] + "USD_PERP", leverage=1)
        except:
            pass
        ##마진타입 변환
        try:
            result = request_client_D.change_margin_type(symbol=sym_name[i] + "USD_PERP", marginType=FuturesMarginType.ISOLATED)
        except:
            pass

def Bin_Minimum_FU_Order_Size_D(sym_name):
    #     sym_name_D = sym_name.copy()
    sym_name_D = [sym_name[i] + "USD_PERP" for i in range(len(sym_name))]
    #     print(sym_name_D)
    aa = request_client_D.get_exchange_information()  # Coin-M 최소 주문량 가져옮(lot_size)

    Bin_Fu_Minimum_Order_Size_D = []
    Bin_Fu_Minimum_Order_Size_D_append = Bin_Fu_Minimum_Order_Size_D.append
    for i in range(len(aa.symbols)):
        if (aa.symbols[i].symbol in sym_name_D):
            Bin_Fu_Minimum_Order_Size_D_append(float(next((item for item in aa.symbols[i].filters if item['filterType'] == 'LOT_SIZE'), None)['minQty']))
    return Bin_Fu_Minimum_Order_Size_D


def Bin_fu_D_sym_name():  # 오직 자기자신만의 심볼들
    bin_fu_D_sym = request_client_D.get_symbol_orderbook_ticker()
    bin_fu_D_sym_name = []
    bin_fu_D_sym_name_append = bin_fu_D_sym_name.append
    [bin_fu_D_sym_name_append(bin_fu_D_sym[i].symbol[:-8]) if (bin_fu_D_sym[i].symbol[-8:] == 'USD_PERP') else bin_fu_D_sym_name_append('remove') for i in range(len(bin_fu_D_sym))]
    removeAllOccur(bin_fu_D_sym_name, 'remove')

    return bin_fu_D_sym_name

def bin_fu_ask_bid_info_D(sym_name):
    bin_sym_names_D = sym_name.copy()

    #     bin_sym_names_D = []
    #     for i in range(len(sym_name)):
    #         bin_sym_names_D.append(sym_name[i] + "USD_PERP")
    bin_fu_ask_bid_D = request_client_D.get_symbol_orderbook_ticker()

    return bin_sym_names_D, bin_fu_ask_bid_D

def bin_fu_ask_info_D(sym_name_D, bin_fu_ask_bid_D):
    bin_sym_info_D = {}

    bin_fu_ask_price_D = []
    bin_fu_ask_price_D_append = bin_fu_ask_price_D.append
    bin_fu_ask_size_D = []
    bin_fu_ask_size_D_append = bin_fu_ask_size_D.append

    for i in range(len(bin_fu_ask_bid_D)):
        bin_sym_info_D[bin_fu_ask_bid_D[i].symbol] = {'askPrice': bin_fu_ask_bid_D[i].askPrice, 'askQty': bin_fu_ask_bid_D[i].askQty}
    for i in range(len(sym_name_D)):
        bin_fu_ask_price_D_append(bin_sym_info_D[sym_name_D[i] + 'USD_PERP']["askPrice"])
        bin_fu_ask_size_D_append(bin_sym_info_D[sym_name_D[i] + 'USD_PERP']['askQty'])
    return bin_fu_ask_price_D, bin_fu_ask_size_D

def bin_fu_bid_info_D(sym_name_D, bin_fu_ask_bid_D):
    bin_sym_info_D = {}

    bin_fu_bid_price_D = []
    bin_fu_bid_price_D_append = bin_fu_bid_price_D.append
    bin_fu_bid_size_D = []
    bin_fu_bid_size_D_append = bin_fu_bid_size_D.append
    for i in range(len(bin_fu_ask_bid_D)):
        bin_sym_info_D[bin_fu_ask_bid_D[i].symbol] = {'bidPrice': bin_fu_ask_bid_D[i].bidPrice, 'bidQty': bin_fu_ask_bid_D[i].bidQty}
    for i in range(len(sym_name_D)):
        bin_fu_bid_price_D.append(bin_sym_info_D[sym_name_D[i] + 'USD_PERP']["bidPrice"])
        bin_fu_bid_size_D.append(bin_sym_info_D[sym_name_D[i] + 'USD_PERP']['bidQty'])
        #         print(bin_fu_bid_price[i],bin_fu_bid_size[i],bin_sym_names[i])
    return bin_fu_bid_price_D, bin_fu_bid_size_D



def first_asset_division(div_unit):
    try:  # load data 파이썬 딕셔너리 자료형 데이터 로드
        with open('Profit_maximization_arbitrage_asset_division.pickle', 'rb') as fr:
            asset_division = pickle.load(fr)

        with open('Profit_maximization_arbitrage_User_data.pickle', 'rb') as fr:
            User_data = pickle.load(fr)
            qwer = User_data["qwer"]
            
        # 아직 운용 중인 자본이 있다면 로직을 사용하지 않고 그대로 배출..
        All_States = [asset_division[i]["State"] for i in range(len(asset_division))]  # 모든 States를 가져옮
        if (len(set(All_States) & {'step_1_1','send_up_to_bin', 'send_up_to_bin_Completion', 'step_1_2', 'step_2_1', 'step_2_2', 'send_bin_to_up', 'send_bin_to_up_Completion', 'step_3', 'final_fun'}) != 0):  # 교집합이 0일 경우(모든상태가 저기에 포함되지 않을경우)
            print("한 싸이클이 아직 끝나지 않음.")
            return asset_division, User_data

    except:
        asset_division = {}
        now = datetime.datetime.now()
        qwer = str(now.year) + "_" + str(now.month) + "_" + str(now.day)
        # path=["C://Users//Happy//Desktop//Binance_Futures_python-master//"]
        columns_name = ['sicle',
                        'total_before',
                        'total_after',
                        'date',
                        # 현물에 남은 자본
                        'k',  # 현재 asset_div_사이클
                        'sym_name_D',  # step_2_1에서 진입한 coin_M 심볼명
                        'point_of_buy_bin_spot_amount_2',  # step_2_1에서 진입한 coin_M 수량
                        'point_of_buy_bin_spot_ticker_2',  # step_2_1에서 진입한 coin_M 평단가

                        'rest_step_2_1_bin_spot_usdt_asset',  # Alpha_asset 바이낸스 달러 지갑에 남은 돈..()

                        'sym_name_b',  # step_2_2에서 남은 coin_M 심볼명
                        'can_send_amount - point_of_Withdrow_UP_amount_2',  # 현재 남은 coin_M => spot 수량
                        'point_of_buy_bin_spot_ticker_2_2',  # 현재 남은 coin_M => spot 평단가
                        'up_asset',
                        'future_leverage',
                        'Usdt_first',

                        'point_of_buy_UP_ticker_1',  # 업비트 매수 평단가
                        'point_of_buy_UP_amount_1',  # 업비트 매수 수량
                        'sym_name_a',

                        'step_1_1_up_krw_asset',  # 실제 사용자본 (변화량 으로 비교)
                        'rest_step_1_1_up_krw_asset',  # 남은 사용할 자본 (변화량 으로 비교)

                        'up_withdrawfee_by_asset',  # 업비트=> 바이낸스 손/수익 (김프 + 전송 수수료)
                        'point_of_Withdrow_UP_amount_1',  # 업비트에서 전송할 수량(변화량 으로 비교)
                        'Usdt_1_1',

                        'step_1_2_bin_spot_usdt_asset',  # 바이낸스에서 팔고 남은 자본(변화량 으로 비교)
                        'enter_servertime',
                        'Usdt_1_2',

                        'sym_name_D',  # 새로 들어갈 심볼명 ()
                        'bin_spot_to_coin_M_order_amount',  # 실제로 들어간 수량(Coin_M)
                        'step_2_1_bin_spot_usdt_asset',  # 바이낸스 사용한 현물 달러 (변화량으로 비교한 값)
                        'rest_step_2_1_bin_spot_usdt_asset',  # 바이낸스 현물 달러에서 쓰고 남은 달러(step2_1사용량 - 변화량)
                        'point_of_buy_bin_spot_amount_2',  # 바이낸스 현물 자본 사용량
                        'point_of_buy_bin_spot_ticker_2',  # 바이낸스 현물 평단가
                        'Usdt_2_1',  # 달러가치
                        'real_enter_amount_by_asset_ratio',  # 사용가능 자본대비 실제로 산 비율(step_2_1)
                        'datetime',  # 진입시점 날짜
                        'point_of_buy_bin_fu_ticker_1',  # 바이낸스 coin_M 선물 진입가
                        'point_of_buy_bin_fu_amount_1',  # 바이낸스 coin_M 선물 수량
                        'bin_fu_D_Cont_order',  # 바이낸스 진입 계수
                        'Coin_M_Cont_idx_num',  # 바이낸스 cont =  N * USD의 N
                        'real_enter_amount',  # 실제로 진입된 수량
                        'bin_fu_D_Order_ID',  # 바이낸스 coin_M 진입 아이디
                        'Avbl_amount',  # 쓰고 남았던 사용가능한 수량
                        'point_of_buy_bin_fu_ticker_1_D',  # 바이낸스 coin_M 평단가
                        'point_of_buy_bin_fu_amount_1_D',  # 바이낸스 coin_M 수량
                        'point_of_buy_bin_fu_commission_1_D',  # 바이낸스 coin_M 진입수수료

                        'sym_name_b',  # 선택받은 심볼명 (그냥 보내든가 아니면 새로 사서 전송하던가 )
                        'point_of_buy_bin_spot_ticker_2_2',  # 현물에서 산 평단가 (x_a의 경우엔 새로 산 평단가)
                        'point_of_buy_bin_spot_amount_2_2',  # 현물에서 산 수량 (x_a의 경우엔 새로 산 수량)
                        'point_of_Withdrow_UP_amount_2',  # 현물에서 산 수량 (x_a의 경우엔 새로 산 수량)
                        'real_send_amount',  # 업비트에 보낼 진짜 수량
                        'Bin_Spot_Minimum_Order_Size_b',  # 바이낸스 현물 최소 주문단위
                        'Usdt_2_2',  # 달러
                        'Second_Use_Bin_Spot_Usdt',  # 두번째 바이낸스 현물에서 사용할 달러
                        'bin_Min_Withdraw_Size_b',  # 바이낸스 현물 전송 최소 수량
                        'bin_withdrawfee_b',  # 바이내스=>업비트 드는 전송 수수료
                        'step_2_2_bin_spot_usdt_asset',  # 주석일 수 있음 => 해당 경우엔 _a의 경우
                        'rest_step_2_2_bin_spot_usdt_asset',  # 주석일 수 있음 => 해당 경우엔 _a의 경우

                        'will_Realize_Arbitrage',  # 실제로 볼 수익
                        'will_Realize_Arbitrage_rate',  # 실제로 볼 수익율
                        'step_3_up_krw_asset',  # step_3 업비트 자본 = 최종 자본
                        'x4',  # 예상 최종자본
                        'Usdt_step_3'  # step_3 달러가치
                        ]
        # csv파일로 적기 # newline 설정을 안하면 한줄마다 공백있는 줄이 생긴다.
        with open(qwer + 'Trading_data.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(columns_name)
    ######## 업비트 자본 ##########
    first_up_asset = upbit.get_balance("KRW")
    up_asset = first_up_asset - first_up_asset * 0.0005  # @ up_market_fee # 계좌 현금 확인
    up_asset = up_asset - up_asset * 0.0005  # @ up_market_fee # 수수료 보정치(슬리피지)
    print(up_asset, "업비드 자본(수수료 제외)")
    #####고정값들##########################
    sicle = 0
    save_state = {}  # 사전형태
    save_state[sicle] = []

    # @ 수수료들 가져오기 ex)client.get_trade_fee()
    up_market_fee = 0.0005  # 0.05%
    bin_spot_market_fee = binance.fees['trading']['taker']  # maker 0.1% taker 0.1% # 0.001
    bin_future_market_fee = 0.0004  # maker 0.02% taker 0.04%
    bin_Coin_M_future_market_fee = 0.0005  # maker 0.01% taker 0.05% # request_client_D.get_commission_rate("BTCUSD_PERP") 로 알아보면 되는데 안됨

    # Using_up_asset의 경우를 제한다.
    Sum_Using_Up_asset = 0
    for i in range(len(asset_division)):
        if (asset_division[i]['State'] == 'None_position'):  # @@ step_3_3인경우도..? 추가해야 할까? 이땐 오류가 안나긴할텐데..
            Sum_Using_Up_asset = Sum_Using_Up_asset + asset_division[i]["up_asset"]
    num = len(asset_division)
    asset_for_div_UP = up_asset - Sum_Using_Up_asset

    asset_for_div_bin = asset_for_div_UP  # 원화 단위 환산
    # asset_for_div_bin = bin_fu_ass * Usdt #원화 단위 환산

    future_leverage = 1
    ########################################
    while True:
        if ((asset_for_div_UP >= div_unit) and (asset_for_div_bin >= div_unit)):
            asset_division[num] = {'up_asset': div_unit,
                                   "binance_asset": (div_unit / future_leverage) / Usdt}

            asset_for_div_UP = asset_for_div_UP - div_unit
            asset_for_div_bin = asset_for_div_bin - div_unit

            future_leverage = 1

            asset_division[num]['future_leverage'] = future_leverage
            asset_division[num]['State'] = "None_position"
            asset_division[num]['Usdt_first'] = Usdt  # 자본 나눌 때 달러가치
            asset_division[num]['save_state'] = save_state
            asset_division[num]["binance_Alpha_asset"] = 0  # 이거 맨처음은 0
            asset_division[num]["qwer"] = qwer  # 이것은 유저 데이터에서 가져오도록 할까..?
            asset_division[num]["div_unit"] = div_unit
            asset_division[num]['want_premium_step_1_1'] = 0.015
            asset_division[num]['want_premium_step_2_2'] = 1.015
            #             asset_division[num]["up_market_fee"] = up_market_fee  # 0.05%
            #             asset_division[num]["bin_spot_market_fee"] = bin_spot_market_fee  # maker 0.1% taker 0.1%
            #             asset_division[num]["bin_future_market_fee"]= bin_future_market_fee #maker 0.02% taker 0.04%
            #             asset_division[num]["bin_Coin_M_future_market_fee"] = bin_Coin_M_future_market_fee  # maker 0.01% taker 0.05%

            num = num + 1

        # @@ 레버리지가 너무 크면 안좋을 거 같음 => 나누는 단위에 따라서 최소값 제한함
        elif ((asset_for_div_UP <= div_unit / 3) or (asset_for_div_bin <= div_unit / 3)):  # --이하는 투자하지않음(최소주문가능량 고려)
            print("option2", "업비트 남은자본:", asset_for_div_UP, "바이낸스 남은자본:", asset_for_div_bin)
            break

        elif (asset_for_div_UP > div_unit / 2.5):  # 20 만원 이상이라면
            asset_division[num] = {'State': "None_position"}
            future_leverage = 1

            asset_Up = asset_for_div_UP
            asset_bin = asset_for_div_bin

            asset_division[num]['up_asset'] = asset_Up
            asset_division[num]['binance_asset'] = asset_bin / Usdt

            asset_for_div_UP = asset_for_div_UP - asset_Up
            asset_for_div_bin = asset_for_div_bin - asset_bin

            asset_division[num]['future_leverage'] = future_leverage
            asset_division[num]['State'] = "None_position"
            asset_division[num]['Usdt_first'] = Usdt
            asset_division[num]['save_state'] = save_state
            asset_division[num]["binance_Alpha_asset"] = 0  # 이거 맨처음은 0
            asset_division[num]["qwer"] = qwer  # 이것은 유저 데이터에서 가져오도록 할까..?
            asset_division[num]["div_unit"] = div_unit
            asset_division[num]['want_premium_step_1_1'] = 0.015
            asset_division[num]['want_premium_step_2_2'] = 1.015
        #             asset_division[num]["up_market_fee"] = up_market_fee  # 0.05%
        #             asset_division[num]["bin_spot_market_fee"] = bin_spot_market_fee  # maker 0.1% taker 0.1%
        #             asset_division[num]["bin_future_market_fee"]= bin_future_market_fee #maker 0.02% taker 0.04%
        #             asset_division[num]["bin_Coin_M_future_market_fee"] = bin_Coin_M_future_market_fee  # maker 0.01% taker 0.05%

        else:
            print("option3", "업비트 남은자본:", asset_for_div_UP, "바이낸스 남은자본:", asset_for_div_bin)
            break
    # @@ ban_List 에서 트레젝션 내역 확인하는 코드
    #ban_list = ['IOTA','CHZ', 'ETH', 'ANKR', 'LINK', 'BAT', 'SXP', 'ETC', 'OMG', 'MANA', 'ZRX', 'ENJ', 'STORJ', 'KNC', 'SRM', 'SRM', 'SAND', 'CVC', 'NEO', 'BCH', 'SC', 'QTUM']
    ban_list = ['ADA','AE','ANKR','BAT','BCD','BCH','BCHA','BCN','BSV','BTC','BTG','BTT','CHZ','CVC','DASH','DCR','DGB','DOGE','ENJ','ETC','ETH','ICX','IOTA','KNC','LINK','LSK','LTC','MANA','NEO','OMG','ONT','QTUM','RVN','SAND','SC','SRM','STORJ','SXP','TRX','XMR','XTZ','ZEC','ZRX']

    User_data = {}
    #     User_data["asset_division"] = asset_division
    User_data['ban_list'] = ban_list
    User_data["up_market_fee"] = up_market_fee  # 0.05%
    User_data["bin_spot_market_fee"] = bin_spot_market_fee  # maker 0.1% taker 0.1%
    User_data["bin_future_market_fee"] = bin_future_market_fee  # maker 0.02% taker 0.04%
    User_data["bin_Coin_M_future_market_fee"] = bin_Coin_M_future_market_fee  # maker 0.01% taker 0.05%

    User_data["qwer"] = qwer

    return asset_division, User_data

def Save_ALL_DATA():
    ## 펑션을 종료시키고 싶을 경우 해당 로직 실행 
    # save data 파이썬 딕셔너리 자료형 데이터 저장.
    with open('Profit_maximization_arbitrage_asset_division.pickle','wb') as fw:
        pickle.dump(asset_division, fw)

    with open('Profit_maximization_arbitrage_User_data.pickle','wb') as fw:
        pickle.dump(User_data, fw)

    for i in range(len(asset_division)):

        print(i ,asset_division[i]['up_asset'] , asset_division[i]['State'])
    print("====================asset_division============================")
    print(asset_division)
    print("=======================User_data==============================")
    print(User_data)

def fu_D_setting():
    sym_name_D = sym_intersection_D()  # 심볾먕 교집합 #  sym_name[i]+"USD_PERP"
    Coin_M_leverage_A_margin(sym_name_D)  # Coin-M 레버리지 및 마진 타입 변화
    time.sleep(len(sym_name_D)*2)
    


In [4]:
# api 키는 바이낸스와 unbit apikey에 저장해둔다
# api 키 값 로드
with open('apikeys.pickle', 'rb') as fr:
    apikeys = pickle.load(fr)

# 암호키 가져옴.
request_client = RequestClient(api_key=g_api_key, secret_key=g_secret_key)
request_client_D = RequestClient_D(api_key=g_api_key, secret_key=g_secret_key)
server_url = 'https://api.upbit.com'
Usdt = upbit_get_usd_krw()
# Usdt = 1170
print("Usdt =", Usdt)

# 업비트 잔고 조회 및 해당 코인 가격에 맞는 가격 도출 기능
access_key = apikeys['up_access_key']
secret_key = apikeys['up_secret_key']
upbit = pyupbit.Upbit(access_key, secret_key)

# 바이낸스 잔고조회
binance = ccxt.binance({
    'apiKey': g_api_key,
    'secret': g_secret_key})

binance_fu = ccxt.binance({
    'apiKey': g_api_key,
    'secret': g_secret_key,
    'options': {'defaultType': 'future'}})

client_spot = Spot(key=g_api_key
                   , secret=g_secret_key)

client = Client(api_key=g_api_key,
                api_secret=g_secret_key)

def cheak_about_regester_by_upbit():
    with open('regester_by_upbit.pickle', 'rb') as fr:
        regester_by_upbit = pickle.load(fr)
        past_sym_name = regester_by_upbit["past_sym_name"]

    sym_name = sym_intersection()
    
    New_sym_bols = list(set(sym_name) - set(regester_by_upbit['past_sym_name']))
    
    if(len(set(sym_name) - set(regester_by_upbit['past_sym_name'])) != 0):
        print("==================================warning==================================")
        print("새로운 심볼들 !!!!(업비트 출금주소 등록할것!)",list(set(sym_name) - set(regester_by_upbit['past_sym_name'])))
        print("==================================warning==================================")
        
        print("새로운 출금 주소를 등록하시고 진행하시겠습니까 ? (Y/N)")
        print("Y입력 시 업비트에 출금 주소 입력하고 진행")
        print("N입력 시 새로 추가된 심볼은 사용하지 않고 진행함.")
        input_command  = input()
        
        if (input_command == "Y"):
            import warnings
            warnings.warn("새로운 심볼들 !!!!(업비트 출금주소 등록할것!)", DeprecationWarning)
            sys.exit() ## 강제로 파이썬 종료

        elif(input_command == "N"):
            User_data['ban_list'] = User_data['ban_list'] + New_sym_bols
#             return User_data

        else:
            print("input 입력 오류")
            sys.exit() ## 강제로 파이썬 종료

def set_premium_asset_division(): # 입력은 %로
    print("%단위로 입력할것 (김프가 그대로)")
    
    print("역프~약김프 최대 수치(프로그램 실행 시 )")
    a_1_1 = float(input())# a_1_1 = 2.5
    print("최소 차익 볼 김프")
    a_2_2 = float(input())# a_2_2 = 1
    
    print("역프~약김프 최대 수치(프로그램 실행 시 )",a_1_1,"%")
    print("최소 차익 볼 김프", ((1 + a_2_2 /100) /(1 - a_1_1/100) - 1) * 100,"%")
    
    a_1_1 = a_1_1 / 100
    a_2_2 = (1 + a_2_2/ 100 )
    for i in range(len(asset_division)):
        asset_division[i]['want_premium_step_1_1'] = a_1_1
        asset_division[i]['want_premium_step_2_2'] = a_2_2
        
    return asset_division

def up_avg_buy_price_info(): # @@ 나중에 이 부분 다 바꾸기
    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()), }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/accounts", headers=headers)
    return res.json()

Usdt = 1273.7


In [5]:
##@ 0.기본셋팅
Usdt = upbit_get_usd_krw()
print('Usdt', Usdt)

balance = binance.fetch_balance()  # free:보유중인 코인, used:거래진행중인 코인 total:전체코인
print("총 자본", upbit.get_balance("KRW") + float(next((item for item in client.futures_account()['assets'] if item['asset'] == 'USDT'), None)['walletBalance']) * Usdt + balance['USDT']['free'] * Usdt)

# 업비트 현물계좌 원화(현금) 확인
# first_up_asset = 1000000# $$@@ 가정한부분 나중에 바꾸기
# first_up_asset = upbit.get_balance("KRW")
# up_asset = first_up_asset - first_up_asset * 0.0005 # @ up_market_fee # 계좌 현금 확인
# up_asset = up_asset - up_asset * 0.0005 # @ up_market_fee # 수수료 보정치(슬리피지)
# print(up_asset, "업비드 자본(수수료 제외)")

# 바이낸스 선물계좌 달러 확인
# bin_fu_ass = 350# $$@@가정하고 계산하는 부분 나중에 바꾸기.
qwepr = float(next((item for item in client.futures_account()['assets'] if item['asset'] == 'USDT'), None)['walletBalance'])
print("바이낸스 선물계좌 달러 잔고 확인:", qwepr) #bin_fu_ass

# first_bin_fu_ass = bin_fu_ass
# bin_fu_ass = bin_fu_ass - bin_fu_ass * 0.0004 # @ bin_future_market_fee
# binance_asset = bin_fu_ass * Usdt
# print(binance_asset, "원 바이낸스 자본(조정 전)")

#@@@ 벤리스트 마지막에 처리하기. 일단 복잡하니까.=> 새로추가된 심볼명들 !!제하기
# ban_list = ['CHZ', 'ETH', 'ANKR', 'LINK', 'BAT', 'SXP', 'ETC', 'OMG', 'MANA', 'ZRX', 'ENJ', 'STORJ', 'KNC', 'SRM', 'SRM', 'SAND', 'CVC', 'NEO', 'BCH', 'SC', 'QTUM']


Usdt 1273.7
총 자본 67461.585469539
바이낸스 선물계좌 달러 잔고 확인: 0.0


In [6]:
asset_division , User_data= first_asset_division(200000)# div_unit = 2000000 #자본을 나눌 단위 (krw) #@@ input()을 추가?
cheak_about_regester_by_upbit() #별말 없으면 그냥 진행
# fu_D_setting() # coin_M레버리지 세팅 @@여기서 오래걸림

asset_division

한 싸이클이 아직 끝나지 않음.
55 55
==================================warning==================================
새로운 심볼들 !!!!(업비트 출금주소 등록할것!) ['FLOW', 'GMT', 'AVAX', 'CELO']
==================================warning==================================
새로운 출금 주소를 등록하시고 진행하시겠습니까 ? (Y/N)
Y입력 시 업비트에 출금 주소 입력하고 진행
N입력 시 새로 추가된 심볼은 사용하지 않고 진행함.
N


{0: {'up_asset': 1000000,
  'binance_asset': 834.7245409015025,
  'future_leverage': 1,
  'State': 'step_2_1',
  'Usdt_first': 1198.0,
  'save_state': {0: []},
  'binance_Alpha_asset': 0,
  'qwer': '2022_1_25',
  'div_unit': 1000000,
  'want_premium_step_1_1': 0.015,
  'want_premium_step_2_2': 1.015,
  'sym_name': ['XLM',
   'THETA',
   'XRP',
   'IOST',
   'ZIL',
   'XEM',
   'SOL',
   'EOS',
   'ATOM',
   'WAVES',
   'ALGO',
   'HBAR'],
  'sym_name[a]': 'XLM',
  'point_of_buy_UP_ticker_1': 242.99999999999025,
  'point_of_buy_UP_amount_1': 4113.176954320001,
  'point_of_Withdrow_UP_amount_1': 4113.1669543,
  'bin_address': 'GAHK7EEG2WWHVKDNT4CEQFZGKF2LGDSW2IVM4S5DP42RBW3K6BTODB4A',
  'bin_Memo': '400293429',
  'up_Min_Withdraw_Size[a]': 1.0000000000000004e-07,
  'x1': 833.958756691581,
  'Usdt_1_1': 1198.5,
  'step_1_1_up_krw_asset': 1000001.75089971,
  'rest_step_1_1_up_krw_asset': -1.7508997099939734,
  'up_withdrawfee[a]': 0.01,
  'expect_x2': 982477.8423446973,
  'Bin_Spot_Minimum

In [7]:
for i in range(len(asset_division)):
    print(asset_division[i]['State'])

step_2_1
step_2_1
step_2_1


In [9]:
asset_division

{0: {'State': 'step_2_1',
  'up_asset': 197062.46885286266,
  'binance_asset': 165.52916325313956,
  'future_leverage': 1,
  'Usdt_first': 1190.5,
  'save_state': {0: []},
  'binance_Alpha_asset': 0,
  'qwer': '2022_1_13',
  'div_unit': 200000,
  'want_premium_step_1_1': 0.015,
  'want_premium_step_2_2': 1.015,
  'sym_name': ['EOS', 'ATOM', 'XRP', 'XLM'],
  'sym_name[a]': 'XLM',
  'point_of_buy_UP_ticker_1': 242.0,
  'point_of_buy_UP_amount_1': 813.90909049,
  'point_of_Withdrow_UP_amount_1': 813.8990904,
  'bin_address': 'GAHK7EEG2WWHVKDNT4CEQFZGKF2LGDSW2IVM4S5DP42RBW3K6BTODB4A',
  'bin_Memo': '1086069876',
  'up_Min_Withdraw_Size[a]': 1.0000000000000004e-07,
  'x1': 164.47898110797493,
  'Usdt_1_1': 1197.5,
  'step_1_1_up_krw_asset': 197064.48289853,
  'rest_step_1_1_up_krw_asset': -2.014045667339815,
  'up_withdrawfee[a]': 0.01,
  'expect_x2': 193565.4998987373,
  'Bin_Spot_Minimum_Order_Size[a]': 1.0,
  'up_withdrawfee_by_asset[a]': 0.9878010483837842,
  '보내기 전 바이낸스잔고': 9.73345829,

In [6]:
print("기본 수치론 1.5, 1.5 입력 추천")
asset_division = set_premium_asset_division() #김프 변수들 입력

기본 수치론 1.5, 1.5 입력 추천
%단위로 입력할것 (김프가 그대로)
역프~약김프 최대 수치(프로그램 실행 시 )
1.5
최소 차익 볼 김프
1.5
역프~약김프 최대 수치(프로그램 실행 시 ) 1.5 %
최소 차익 볼 김프 3.045685279187804 %


In [9]:
print("역프~약김프 최대 수치(프로그램 실행 시 )")
a_1_1 = float(input())# a_1_1 = 2.5
print("최소 차익 볼 김프")
a_2_2 = float(input())# a_2_2 = 1

print("역프~약김프 최대 수치(프로그램 실행 시 )",a_1_1,"%")
print("최소 차익 볼 김프", ((1 + a_2_2 /100) /(1 - a_1_1/100) - 1) * 100,"%")

역프~약김프 최대 수치(프로그램 실행 시 )
2
최소 차익 볼 김프
1.25
역프~약김프 최대 수치(프로그램 실행 시 ) 2.0 %
최소 차익 볼 김프 3.316326530612246 %


In [10]:
ban_list = ['ADA','AE','ANKR','BAT','BCD','BCH','BCHA','BCN','BSV','BTC','BTG','BTT','CHZ','CVC','DASH','DCR','DGB','DOGE','ENJ','ETC','ETH','ICX','IOTA','KNC','LINK','LSK','LTC','MANA','NEO','OMG','ONT','QTUM','RVN','SAND','SC','SRM','STORJ','SXP','TRX','XMR','XTZ','ZEC','ZRX']
User_data["ban_list"] = ban_list
Save_ALL_DATA()

0 197062.46885286266 step_2_1
====================asset_division============================
{0: {'State': 'step_2_1', 'up_asset': 197062.46885286266, 'binance_asset': 165.52916325313956, 'future_leverage': 1, 'Usdt_first': 1190.5, 'save_state': {0: []}, 'binance_Alpha_asset': 0, 'qwer': '2022_1_13', 'div_unit': 200000, 'want_premium_step_1_1': 0.015, 'want_premium_step_2_2': 1.015, 'sym_name': ['EOS', 'ATOM', 'XRP', 'XLM'], 'sym_name[a]': 'XLM', 'point_of_buy_UP_ticker_1': 242.0, 'point_of_buy_UP_amount_1': 813.90909049, 'point_of_Withdrow_UP_amount_1': 813.8990904, 'bin_address': 'GAHK7EEG2WWHVKDNT4CEQFZGKF2LGDSW2IVM4S5DP42RBW3K6BTODB4A', 'bin_Memo': '1086069876', 'up_Min_Withdraw_Size[a]': 1.0000000000000004e-07, 'x1': 164.47898110797493, 'Usdt_1_1': 1197.5, 'step_1_1_up_krw_asset': 197064.48289853, 'rest_step_1_1_up_krw_asset': -2.014045667339815, 'up_withdrawfee[a]': 0.01, 'expect_x2': 193565.4998987373, 'Bin_Spot_Minimum_Order_Size[a]': 1.0, 'up_withdrawfee_by_asset[a]': 0.987801

In [9]:
# #수익율 보려고 만든것
# #가장 해보고 싶은거 ㅅ
# #@@ 역프 (5퍼 이상), 김프 1퍼(3퍼이상) ,김프 2.5퍼 (1퍼)
# if (len(asset_division) // 3 >= 1):
#     print("if")
#     a_1_1 = 0.025
#     a_2_2 = 1.01

#     b_1_1 = 0.02
#     b_2_2 = 1.015

#     c_1_1 = 0.015
#     c_2_2 = 1.025

#     a = math.floor(len(asset_division)/3) 
#     b = math.floor(len(asset_division)/3) 
#     c = len(asset_division) - (a + b)  
#     for i in range(0,a):
#         asset_division[i]['want_premium_step_1_1'] = a_1_1
#         asset_division[i]['want_premium_step_2_2'] = a_2_2
#         print(i)
#     for i in range(a,a+b):
#         asset_division[i]['want_premium_step_1_1'] = b_1_1
#         asset_division[i]['want_premium_step_2_2'] = b_2_2
#         print(i)
#     for i in range(a+b,a+b+c):
#         asset_division[i]['want_premium_step_1_1'] = c_1_1
#         asset_division[i]['want_premium_step_2_2'] = c_2_2
#         print(i)
# elif (len(asset_division) // 2 >= 1):
#     print("elif")
#     a_1_1 = 0.025
#     a_2_2 = 1.01

#     b_1_1 = 0.02
#     b_2_2 = 1.015
    
#     a = math.floor(len(asset_division)/2) 
#     b = math.floor(len(asset_division)/2) 
    
#     for i in range(0,a):
#         asset_division[i]['want_premium_step_1_1'] = a_1_1
#         asset_division[i]['want_premium_step_2_2'] = a_2_2
#         print(i)
#     for i in range(a,a+b):
#         asset_division[i]['want_premium_step_1_1'] = b_1_1
#         asset_division[i]['want_premium_step_2_2'] = b_2_2
#         print(i)

In [10]:
def step_1_1(sym_name, up_asset, up_market_fee, bin_spot_market_fee ,div_unit ,bin_Coin_M_future_market_fee, want_premium_step_1_1):
    
    before_step_1_1_up_krw_asset = upbit.get_balance("KRW")
    # before_step_1_1_bin_fu_usdt_asset = float(next((item for item in client.futures_account()['assets'] if item['asset'] == 'USDT'), None)['walletBalance'])

    Usdt = upbit_get_usd_krw()  # 그래서 잘 안변하는 달러는 여기서 갱신
    future_leverage = 1

    all_coin_info = client_spot.coin_info()  # 코인 정보를 전부 한번만 가져와서 저장함
    # 아래 두개는 입출금 불가 상태에서 끼는 김프를 대비
    sym_name = del_bin_disable_withdraw(all_coin_info, sym_name)  # 바이낸스 출금 불가능 심볼 삭제
    sym_name = del_up_disable_deopsit(sym_name)  # 업비트 입금 불가능 심볼 삭제
    sym_name = del_bin_disable_deopsit(all_coin_info, sym_name)  # 바이낸스 입금 불가능 코인 삭제
    sym_name = del_up_disable_whthdraw(sym_name)  # 업비트 출금 불가능 심볼 삭제

    # @@위에서 정의하면 없어도댐
    sym_name = remove_sym_name_1(all_coin_info, sym_name, Usdt, up_asset / future_leverage, 0.005,User_data['ban_list'], up_market_fee)  # 내 자본 대비 전송수수료(0.005)가 큰 코인 심볼명 삭제

    # Coin_M_leverage_A_margin(sym_name) #coim_m레버리지 변환 => @ 이부분을 전송 도중에 할까??? ㄱㅊ은데??

    # 함수 정의해도 될듯 업비트 사전 평단가, 수량 가져옴
    up_previous_ticker_list = []
    up_previous_amount_list = []
    up_previous_ticker_list_append = up_previous_ticker_list.append
    up_previous_amount_list_append = up_previous_amount_list.append
    for i in range(len(sym_name)):
        try:
            up_previous_ticker, up_previous_amount = up_avg_buy_price(sym_name[i])
            time.sleep(0.3)
        except:
            up_previous_ticker = 0
            up_previous_amount = 0
        up_previous_ticker_list_append(up_previous_ticker)
        up_previous_amount_list_append(up_previous_amount)

    up_withdrawfee, bin_withdrawfee = up_bin_withdraw_fee(sym_name, all_coin_info)  # 업비트 전송 수수료 , 바이낸스 전송 수수료
    up_Min_Withdraw_Size, bin_Min_Withdraw_Size = up_bin_withdraw_Size(sym_name,all_coin_info)  # 업비트, 바이낸스 전송최소 가능량
    # Bin_Fu_Minimum_Order_Size = Bin_Minimum_FU_Order_Size_D(sym_name)  # 바이낸스 현물 최소 주문 가능량
    Bin_Spot_Minimum_Order_Size = Bin_Minimum_Spot_Order_Size(sym_name)  # 바이낸스 선물 최소 주문가능량

    # 호가창 정보 가져옴
    up_sym_names, up_spot_ask_bid = up_spot_ask_bid_info(sym_name)
    # up_ask_price, up_ask_size = up_ask_info(up_sym_names, up_spot_ask_bid)  # 업비트 매수 호가창 4
    up_bid_price, up_bid_size = up_bid_info(up_sym_names, up_spot_ask_bid)  # 업비트 매도 호가창 1

    bin_sym_names, bin_spot_ask_bid = bin_spot_ask_bid_info(sym_name)
    bin_spot_ask_price, bin_spot_ask_size = bin_ask_info(bin_sym_names, bin_spot_ask_bid)  # 바-현 매수 호가창 2
    # bin_spot_bid_price, bin_spot_bid_size = bin_bid_info(bin_sym_names, bin_spot_ask_bid)  # 바-현 매도 호가창 3

    # bin_sym_names_D, bin_fu_ask_bid_D = bin_fu_ask_bid_info_D([sym_name_D_])
    # bin_fu_D_ask_price, bin_fu_D_ask_size = bin_fu_ask_info_D(bin_sym_names_D, bin_fu_ask_bid_D)  # 바-선D 매수 호가창
    # bin_fu_D_bid_price, bin_fu_D_bid_size = bin_fu_bid_info_D(bin_sym_names_D, bin_fu_ask_bid_D)  # 바-선D 매도 호가창

    up_to_bin_pre_rate = []
    up_to_bin_pre_rate_append = up_to_bin_pre_rate.append
    # bin_to_up_pre_rate = []
    up_withdrawfee_by_asset = []
    up_withdrawfee_by_asset_append = up_withdrawfee_by_asset.append
    # bin_withdrawfee_by_asset = []

    for i in range(len(sym_name)):
        #     up_to_bin_pre_rate.append(up_bid_price[i]/(bin_spot_ask_price[i]*Usdt)) # 역프 or 바낸 전송시 예상 손수익 %
        #     bin_to_up_pre_rate.append((bin_spot_bid_price[i]*Usdt)/up_ask_price[i]) # 김프 or 업비트 전송시 예상 손수익 %

        # 프리미엄들(역프,김프)
        up_to_bin_pre_rate_append((bin_spot_ask_price[i] * Usdt) / up_bid_price[i])  # 역프 or 바낸 전송시 예상 손수익 %(김프시 손해)
        # bin_to_up_pre_rate.append(up_ask_price[i] / (bin_spot_bid_price[i] * Usdt))  # 김프 or 업비트 전송시 예상 손수익 %(김프시 수익)

        # 자본대비 수수료 오직 비율로만 따짐.
        up_withdrawfee_by_asset_append(up_to_bin_pre_rate[i] - ((up_bid_price[i] * up_withdrawfee[i]) / up_asset))  # 내 자본 대비 전송수수료를 포함하여 가장 좋은 최적의 값 서칭
        # bin_withdrawfee_by_asset.append(bin_to_up_pre_rate[i] - ((bin_spot_bid_price[i] * bin_withdrawfee[i]) / (bin_fu_ass)))  # 바이낸스 선물에서 매수 할거니 매도호가창조회

    # 나중에 a는 변수명 최적_to_bin으로 바꾸기?b도
    # (업 => 바)전송시 최적 코인 인덱스 번호(a) , (바 => 업)전송 시 최적 코인 인덕스 번호(b)
    a = up_withdrawfee_by_asset.index(max(up_withdrawfee_by_asset))  # 리스트에서 가장 큰 값 뽑음(업 => 바 가장 큰 값 인덱스) @@맞지? 현재 김프역프 왔다리 갓다리 하니까
    # b = bin_withdrawfee_by_asset.index(max(bin_withdrawfee_by_asset))  # 리스트에서 가장 큰 값 뽑음(바 => 업 가장 큰 값 인덱스)

    # 시드의 움직임

    # 1.업비트 주문 가능량########이 부분은 빨라야 해서 함수 정의 안함########################
    up_can_order_amout = up_asset * (1 - up_market_fee)  # *0.005이미함
    up_can_order_amout = up_can_order_amout + (up_withdrawfee[a] * up_bid_price[a])  ##완벽한 헷징을 위해 업비트 전송료만큼 추가..#@@이부분..계산
    Decimal_rounding = UP_Minimum_Order_cash_Size(up_bid_price[a])
    up_can_order_amout = (up_can_order_amout // Decimal_rounding) * Decimal_rounding

    can_send_Decimal = up_Min_Withdraw_Size[a]  # 전송가능 최소량@@이게 잇어야 계산 가능
    if (can_send_Decimal == 0):  # 전송 수수료가 0 인경우 나눌 수가 없음
        pass
    else:
        up_can_order_amout = (up_can_order_amout // can_send_Decimal) * can_send_Decimal

    #################################################################################

    # 1. 업비트에서 매수
    #     x1 = up_asset * (1 - 0.0005) - (up_bid_price[a] * up_withdrawfee[a])#* 코인 현재가 #예상평단가 up_bid_price[a]
    x1 = up_can_order_amout - (up_bid_price[a] * up_withdrawfee[a])  # * 코인 현재가 #예상평단가 up_bid_price[a]
    # 1. 바이낸스 선물에서 공매도
    #     y1 = binance_asset * (1 - 0.0004) #예상 평단가 bin_fu_ask_price[a]
    # y1 = bin_can_order_asset_symbol_quantity * bin_fu_ask_price[a] * Usdt

    # 2. 바이낸스 현물에서 매도 #예상수익 역프 : up_to_bin_pre_rate

    x2 = x1 * ((bin_spot_ask_price[a] * Usdt) / up_bid_price[a]) * (1 - bin_spot_market_fee)  # 예상 평단가 bin_spot_ask_price[a]
    #     x2 = up_can_order_amout * ((bin_spot_ask_price[a]*Usdt)/up_bid_price[a]) * (1 - bin_spot_market_fee) #예상 평단가 bin_spot_ask_price[a]

    # want_premium_step_1_1(내가 원하는 김프) 이하일 경우 평균
    # option #레버리지만큼 보정치
    if (((1 - (x2 / up_asset)) < want_premium_step_1_1 - want_premium_step_1_1 * 0.08) and (((1 - (sum(up_to_bin_pre_rate) / len(up_to_bin_pre_rate)))) < want_premium_step_1_1)):
        # 자동매매 확인겸 리스크좀 지고 가는 if문
        # 업비트 호가창 확인 , 주문가능한지 확인
        # 손해율 1%미만일시 주문
        # option
        #     if (((x2 + y2) / (binance_asset + up_asset)) > 0.988):

        if ((up_bid_size[a] * 0.5 > (up_can_order_amout // up_bid_price[a]) and (up_can_order_amout < up_asset))):
        # if ((up_bid_size[a] * 0.5 > (up_can_order_amout // up_bid_price[a]) and (up_can_order_amout < up_asset) and (bin_can_order_asset_symbol_quantity * bin_fu_ask_price[a] < binance_asset / Usdt * future_leverage))):

            try:
                # @@@ 여기서 자꾸 오류가 나네..
                # 업비트 주문신청
                ret = upbit.buy_market_order("KRW-" + sym_name[a], up_can_order_amout)  # 있는현금만큼 구매#갯수로 구매가능하면..
                # 바이낸스주문신청                    # 심볼            #sell 이면 숏 buy면 롱
                #             result = request_client.post_order(symbol=sym_name[a] + "USDT", side=OrderSide.SELL,
                #                                                ordertype=OrderType.MARKET,  # 시장가 거래
                #                                                quantity=bin_can_order_asset_symbol_quantity)  # quantity수량.#Orderside sell 이면 판매

                #             PrintBasic.print_obj(result)
                pprint.pprint(ret)  # 구매주문확인


            except:  # 시장가 주문 오류 시 지정가 주문으로 들어감 이부분 위험함
                # 지정가 주문
                ret = upbit.buy_limit_order("KRW-" + sym_name[a], up_bid_price[a], up_can_order_amout / up_bid_price[a])  # 지정가 주문은 또 갯수로 받음
                # 지정가 거래
                #             result = request_client.post_order(symbol=sym_name[a] + "USDT", side=OrderSide.SELL,
                #                                                ordertype=OrderType.LIMIT,
                #                                                quantity=bin_can_order_asset_symbol_quantity,
                #                                                timeInForce="GTC",
                #                                                price=bin_fu_ask_price[a])

                #             PrintBasic.print_obj(result)
                pprint.pprint(ret)  # 구매주문확인
                # 주문이 전부 끝날 때 까지 대기.
                while True:  # @@ 이부분 애매한게 주문수량이 소수점으로 조금더  들어감
                    # @@ 사실 지정가는 안긁히면 매우 위험한 방법.. 예를들어 바이낸스는 모두 긁히고 업비트는 하나도 안긁힐 경우 큰일난다.
                    try:
                        cheak_amount_1 = float(next((item for item in upbit.get_balances() if item['currency'] == sym_name[a]), None)['balance'])
                        cheak_amount_2 = ((up_can_order_amout / up_bid_price[a]) // 0.00000001) * 0.00000001
                        #                     cheak_amount_3 = float(next((item for item in client.futures_account()['positions'] if item['symbol'] == sym_name[a] + "USDT"), None)['positionAmt'])
                        #                     cheak_amount_4 = bin_can_order_asset_symbol_quantity

                        statuses = ("업비트 : 주문완료량 / 주문량", cheak_amount_1, "/", cheak_amount_2,
                                    #                                 "바이낸스 : 주문완료량 / 주문량", cheak_amount_3, "/", cheak_amount_4,
                                    "현재시각", time.ctime(time.time())
                                    )

                        #                     if ((cheak_amount_1 >= cheak_amount_2) and (cheak_amount_3 == cheak_amount_4)):
                        if (cheak_amount_1 >= cheak_amount_2):
                            print("모든 수량 포지션 진입")
                            break

                    except TypeError:
                        statuses = ("업비트 주문량 아직 0개", "현재시각", time.ctime(time.time()))

                    if (len(statuses) != 0):
                        clear_output(wait=True)
                        display(statuses)

                    time.sleep(0.4)
                # pass
            # tt.toc()
            time.sleep(0.4)
            print("업비트 해당 심볼 주문개수", (up_can_order_amout / up_bid_price[a]))
            print("1. 최종 업비트 자본 사용량", up_can_order_amout)

            # 업비트 매수 평단가  , 수량
            point_of_buy_UP_ticker_1, point_of_buy_UP_amount_1 = up_avg_buy_price(sym_name[a])

            if (up_previous_ticker_list[a] != 0):  # 이전에 구매한 코인이 아니라면(평단가는0 이 아님)
                up_previous_ticker, up_previous_amount = up_previous_ticker_list[a], up_previous_amount_list[a]  # 이전 평단가,수량
                now_ticker, now_amount = up_avg_buy_price(sym_name[a])  # 이후 평단가,수량
                # (기존 평단가 * 기존 개수) + (point_of_buy_UP_ticker_1  * point_of_buy_UP_amount_1)/(모든 개수) = 바뀐평단가
                point_of_buy_UP_amount_1 = now_amount - up_previous_amount
                point_of_buy_UP_ticker_1 = (now_ticker * (up_previous_amount + point_of_buy_UP_amount_1) - (up_previous_amount * up_previous_ticker)) / (point_of_buy_UP_amount_1)

            ##### 잔고조회로 남는 코인자본 전송 같이 시킴
            can_send_amount = upbit.get_balance(sym_name[a])
            if (0 < ((can_send_amount - point_of_buy_UP_amount_1) * point_of_buy_UP_ticker_1) < (div_unit / 5)):
                print("심볼들 남음")
                pass  # can_send_amount 가 자기자신이 된다 (남은 자본 보냄)
            else:
                print("곂치는 심볼 없음")
                can_send_amount = point_of_buy_UP_amount_1

            #####
            point_of_Withdrow_UP_amount_1 = can_send_amount - up_withdrawfee[a]
            Up_Minimum_Withdraw_Size_dig = Up_Minimum_Withdraw_Size_digit(sym_name[a]) #소수점 단위 가져옴
            Decimal_rounding = up_Min_Withdraw_Size[a]
            if (Decimal_rounding == 0):  # 전송 수수료가 0 인경우 나눌 수가 없음
                pass
            else:
                print("곂치는 심볼 없음")
                point_of_Withdrow_UP_amount_1 = (point_of_Withdrow_UP_amount_1 // Decimal_rounding) * Decimal_rounding
                point_of_Withdrow_UP_amount_1 = round(point_of_Withdrow_UP_amount_1,int(Up_Minimum_Withdraw_Size_dig))
            #####
            time.sleep(0.4)

            # 바이낸스 주소 가져옴
            bin_address = client_spot.deposit_address(sym_name[a])['address']
            bin_Memo = client_spot.deposit_address(sym_name[a])['tag']

            print("주문신청 완료")
            time.sleep(0.4)
            after_step_1_1_up_krw_asset = upbit.get_balance("KRW")
            step_1_1_up_krw_asset = before_step_1_1_up_krw_asset - after_step_1_1_up_krw_asset
            rest_step_1_1_up_krw_asset = up_asset - step_1_1_up_krw_asset

            x1 = point_of_Withdrow_UP_amount_1 * point_of_buy_UP_ticker_1   # 업비트에서 전송수수료까지 뺸 진짜로 출금하여 사용되는 금액 #point_of_Withdrow_UP_amount_1에는 전송수수료가 제외되어 있다(따로 수수료 뺼 필요 x)
            expect_x2 = x1 * ((bin_spot_ask_price[a] * Usdt) / point_of_buy_UP_ticker_1) * (1 - bin_spot_market_fee)  # 예상 평단가 bin_spot_ask_price[a]

            return 'step_1_1', sym_name[a], point_of_buy_UP_ticker_1, point_of_buy_UP_amount_1, point_of_Withdrow_UP_amount_1, bin_address, bin_Memo, future_leverage, up_Min_Withdraw_Size[a], x1 / Usdt, Usdt, step_1_1_up_krw_asset, rest_step_1_1_up_krw_asset, up_withdrawfee[a], expect_x2 ,Bin_Spot_Minimum_Order_Size[a] ,up_withdrawfee_by_asset[a]

    else:

        statuses = ('x1', x1,
                    'x2', x2,
                    "step_2_1에서의 예상 자본", x2,
                    "첫번째 예상 손수익", x2 - up_asset,
                    "첫번째 예상 손수익율", x2 / up_asset,
                    "달러가치", Usdt,
                    "업비트 실제 쓰일 자본", up_can_order_amout - (up_bid_price[a] * up_withdrawfee[a]),
                    "평균김프(%)", (1 - (sum(up_to_bin_pre_rate) / len(up_to_bin_pre_rate))) * 100
                    )
        if (len(statuses) != 0):
            clear_output(wait=True)
            display(statuses)

    # @ 여기에 추가하고 싶은 코드 넣으면 됨 위에까지는 주문신청이라 빠르게 들어가야함.
    return 'None_position', sym_name[a], 'point_of_buy_UP_ticker_1', 'point_of_buy_UP_amount_1', 'point_of_Withdrow_UP_amount_1', 'bin_address', 'bin_Memo', future_leverage, up_Min_Withdraw_Size[a], x1 / Usdt, Usdt, 'step_1_1_up_krw_asset', 'rest_step_1_1_up_krw_asset', up_withdrawfee[a], 'expect_x2' ,Bin_Spot_Minimum_Order_Size[a] ,up_withdrawfee_by_asset[a]

In [11]:
def send_up_to_bin(send_to_bin_optimal_simbol, point_of_Withdrow_UP_amount_1, bin_address, bin_Memo):  # 업비트에서 바이낸스 전송주문 요청
    send_state_1 = [""]
    try:
        send_state_1 = up_withdrow_request(send_to_bin_optimal_simbol, point_of_Withdrow_UP_amount_1, bin_address, bin_Memo)  # 전송주문 요청

        Coin_M_leverage_A_margin([send_to_bin_optimal_simbol])  # coim_m레버리지 변환 => @ 이부분을 전송 도중에 할까??? ㄱㅊ은데??

        balance = binance.fetch_balance()  # 바이낸스 잔고 확인 (전송완료 확인때문에)#@@하지만 동시에 전송된다면..?
        return "send_up_to_bin", balance[send_to_bin_optimal_simbol]['free']

    except:  # 전송주문 실패
        statuses = ("업비트 => 바이낸스 전송주문 실패", send_state_1)
        if (len(statuses) != 0):
            clear_output(wait=True)
            display(statuses)
        return "step_1_1", "보내기 전 바이낸스잔고"
    
def send_up_to_bin_2(send_to_bin_optimal_simbol, point_of_Withdrow_UP_amount_1, bin_address, bin_Memo , Bin_Spot_Minimum_Order_Size_a):  # 업비트에서 바이낸스 전송주문 요청
    send_state_1 = [""]
    
    Decimal_rounding = Bin_Spot_Minimum_Order_Size_a
    point_of_Withdrow_UP_amount_1 = (point_of_Withdrow_UP_amount_1 // Decimal_rounding) * Decimal_rounding
#     point_of_Withdrow_UP_amount_1 = round(point_of_Withdrow_UP_amount_1, 6) ## @@ option
    
    try:
        send_state_1 = up_withdrow_request(send_to_bin_optimal_simbol, point_of_Withdrow_UP_amount_1, bin_address, bin_Memo)  # 전송주문 요청

        Coin_M_leverage_A_margin([send_to_bin_optimal_simbol])  # coim_m레버리지 변환 => @ 이부분을 전송 도중에 할까??? ㄱㅊ은데??

        balance = binance.fetch_balance()  # 바이낸스 잔고 확인 (전송완료 확인때문에)#@@하지만 동시에 전송된다면..?
        return "send_up_to_bin", balance[send_to_bin_optimal_simbol]['free'] , point_of_Withdrow_UP_amount_1

    except:  # 전송주문 실패
        statuses = ("업비트 => 바이낸스 전송주문 실패", send_state_1)
        if (len(statuses) != 0):
            clear_output(wait=True)
            display(statuses)
        return "step_1_1", "보내기 전 바이낸스잔고" , point_of_Withdrow_UP_amount_1

In [12]:
def send_up_to_bin_Completion(send_to_bin_optimal_simbol, before_send_to_bin_balace, point_of_Withdrow_UP_amount_1, Bin_Spot_Minimum_Order_Size_a):  # 업비트에서 바이낸스 전송완료?

    balance = binance.fetch_balance()
    
    quotePrecision = 8  ## @@ 옵션
    aa = before_send_to_bin_balace + point_of_Withdrow_UP_amount_1
    Decimal_rounding = Bin_Spot_Minimum_Order_Size_a
    aa = (aa // Decimal_rounding) * Decimal_rounding
    aa = round(aa,quotePrecision)
    if (balance[send_to_bin_optimal_simbol]['free'] >= aa):
        return "send_up_to_bin_Completion"

    time.sleep(3)  # @ time.sleep(10)? 바이낸스 api제한 걸림

    statuses = ("업비트 => 바이낸스",
                send_to_bin_optimal_simbol,
                "전송 중", balance[send_to_bin_optimal_simbol]['free'],
                "현재시각", time.ctime(time.time()))

    if (len(statuses) != 0):
        clear_output(wait=True)
        display(statuses)
    return "send_up_to_bin"

In [13]:
def step_1_2(up_asset, sym_name_a, point_of_buy_UP_ticker_1, Bin_Spot_Minimum_Order_Size, x1, point_of_Withdrow_UP_amount_1, point_Usdt , div_unit, ban_list, bin_spot_market_fee):  # 해야할 기능 => 바로 파는것이 나은가 아니면 그냥 파는것이 나은가
    after_Avbl_amount, point_of_buy_bin_fu_amount_1_D, point_of_buy_bin_fu_ticker_1_D, point_of_buy_bin_fu_commission_1_D = 0,0,0,0
    Usdt = upbit_get_usd_krw()

    # 헷징 종료 전 바이낸스 현물, 선물 지갑 달러잔고 확인
    before_step_1_2_bin_spot_usdt_asset = float(next((item for item in client.get_account()['balances'] if item['asset'] == 'USDT'), None)['free'])

    ##### @@ 남는자본 + div_unit / 5 보다 작은 잔고가 존재할 시 찌꺼기로 생각하고 포함해서 계산
    balance = binance.fetch_balance()  # 이거지우기
    if (0 < (balance[sym_name_a]['free'] - point_of_Withdrow_UP_amount_1) * point_Usdt < (div_unit / 5)):
        Decimal_rounding = Bin_Spot_Minimum_Order_Size  # 주문가능 최소량
        order_amount_bin_spot = (balance[sym_name_a]['free'] // Decimal_rounding) * Decimal_rounding
    else:
        Decimal_rounding = Bin_Spot_Minimum_Order_Size  # 주문가능 최소량
        order_amount_bin_spot = (point_of_Withdrow_UP_amount_1 // Decimal_rounding) * Decimal_rounding
    #####


    sym_name_D = sym_intersection_D()  # 심볾먕 교집합 #  sym_name[i]+"USD_PERP"
#     Coin_M_leverage_A_margin(sym_name_D)  # Coin-M 레버리지 및 마진 타입 변화
    sym_name_D = list(set(sym_name_D) - set(ban_list))  # @@이부분 최적 값 찾는거 다르게 바꾸기

    enter_servertime = request_client_D.get_servertime()  # 진입 시점 서버 시간 확인
    # enter_servertime < fundingTime[idx_num]

    # 전송 후
    bin_sym_names, bin_spot_ask_bid = bin_spot_ask_bid_info([sym_name_a])
    bin_spot_ask_price, bin_spot_ask_size = bin_ask_info(bin_sym_names, bin_spot_ask_bid)  # 바-현 매수 호가창 2

    # 2. 바이낸스 현물에서 매도 #예상수익 역프 : up_to_bin_pre_rate
    expect_x2 = order_amount_bin_spot * bin_spot_ask_price[0] * Usdt * (1 - bin_spot_market_fee) * ((bin_spot_ask_price[0] * Usdt) / point_of_buy_UP_ticker_1)
    expect_bin_spot_usdt = expect_x2 / Usdt

    Usdt_MIN, Usdt_MAX, Out_MIN, Out_MAX = 120, 10000, 0.975, 1 # Usdt_MIN, Usdt_MAX, Out_MIN, Out_MAX = 120, 10000, 0.985, 1 #옵션 #option
    using_asset_ratio_by_Min_Max_Sclaer = ((expect_bin_spot_usdt - Usdt_MIN) * (Out_MAX - Out_MIN)) / (Usdt_MAX - Usdt_MIN) + Out_MIN

    # tt.tic()

    ################# 몇개가 사전에 사용가능량인지 확인
    aa = request_client_D.get_account_information()
    coin_m_asset_info = {}
    for i in range(len(aa.assets)):
        if (aa.assets[i].asset in sym_name_D):
            coin_m_asset_info[aa.assets[i].asset] = {"Avbl_amount": aa.assets[i].maxWithdrawAmount}
    ##############################################

    while True:
        sym_name_a_idx_num = None
        for i in range(len(sym_name_D)):  # @@ 전송불가 코인이 되어 심볼네임에서 안나올 수 있음
            if (sym_name_D[i] == sym_name_a):
                sym_name_a_idx_num = i
        if (sym_name_a_idx_num == None):
            print("coim_M의 심볼과 곂치지 않음")
            break
        else:
            break

    if (sym_name_a_idx_num == None):
        # 2.바이낸스 & 업비트 포지션 종료
        # 잔고만큼 판매 주문 전송
        order = binance.create_market_sell_order(sym_name_a + '/USDT', order_amount_bin_spot)
        print(order)

        time.sleep(3)

        statuses = (
            "허가 된 범위에 따른 값", using_asset_ratio_by_Min_Max_Sclaer,
            "자본대비 들어갈 비율", 'real_enter_amount_by_asset_ratio_sym_name_a',
            "sym_name_a", sym_name_a
        )
        if (len(statuses) != 0):
            clear_output(wait=True)
            display(statuses)

        # 헷징 종료 후 바이낸스 현물, 선물 지갑 달러잔고 확인
        after_step_1_2_bin_spot_usdt_asset = float(next((item for item in client.get_account()['balances'] if item['asset'] == 'USDT'), None)['free'])
        step_1_2_bin_spot_usdt_asset = after_step_1_2_bin_spot_usdt_asset - before_step_1_2_bin_spot_usdt_asset
        x2 = step_1_2_bin_spot_usdt_asset * Usdt
        return "step_1_2",x2, step_1_2_bin_spot_usdt_asset, after_step_1_2_bin_spot_usdt_asset, before_step_1_2_bin_spot_usdt_asset, Usdt , 'enter_servertime', 'sym_name_D[sym_name_a_idx_num]', 'bin_spot_to_coin_M_order_amount', 'step_2_1_bin_spot_usdt_asset', 'rest_step_2_1_bin_spot_usdt_asset', 'point_of_buy_bin_spot_amount_2', 'point_of_buy_bin_spot_ticker_2', 'Usdt_2_1', 'real_enter_amount_by_asset_ratio', 'datetime', 'point_of_buy_bin_fu_ticker_1', 'point_of_buy_bin_fu_amount_1', 'bin_fu_D_Cont_order', 'Coin_M_Cont', "들어간 자본 수량", '진입 아이디', "진입시점(서버)", 'Avbl_amount', 'point_of_buy_bin_fu_amount_1_D', 'point_of_buy_bin_fu_ticker_1_D', 'point_of_buy_bin_fu_commission_1_D'

    elif(sym_name_D[sym_name_a_idx_num] == sym_name_a):
        # Bin_Spot_Minimum_Order_Size = Bin_Minimum_Spot_Order_Size(sym_name_D)  # 바이낸스 현물 최소 주문량
        Coin_M_Cont, Coin_M_Min_Cont = Bin_Fu_Minimum_Order_Cont_n_Size_D(sym_name_D)  # 바낸 coin-m 필요 정보(coin-m최소 주문량)
        bin_fu_previous_ticker_list_D, bin_fu_previous_amount_list_D = bin_fu_avg_buy_price_D(sym_name_D)  # 이전 모든 평단가, 진입가

        bin_sym_names, bin_spot_ask_bid = bin_spot_ask_bid_info(sym_name_D)
        # bin_spot_ask_price,bin_spot_ask_size = bin_ask_info(bin_sym_names,bin_spot_ask_bid)#바-현 매수 호가창 2
        bin_spot_bid_price, bin_spot_bid_size = bin_bid_info(bin_sym_names, bin_spot_ask_bid)  # 바-현 매도 호가창 3

        bin_sym_names_D, bin_fu_ask_bid_D = bin_fu_ask_bid_info_D(sym_name_D)
        bin_fu_D_ask_price, bin_fu_D_ask_size = bin_fu_ask_info_D(bin_sym_names_D, bin_fu_ask_bid_D)  # 바-선D 매수 호가창
        # bin_fu_D_bid_price, bin_fu_D_bid_size = bin_fu_bid_info_D(bin_sym_names_D, bin_fu_ask_bid_D)  # 바-선D 매도 호가창

        ##################### 콘탱코 #############################
        contango_rate_by_D = []  # backwardation
        contango_rate_by_D_append = contango_rate_by_D.append
        # 펀비는 선물가격이 현물보다 높을 시에 받는 것이므로 선물가격이 높은곳에 가서 1배숏을 한다.
        for i in range(len(sym_name_D)):
            contango_rate_by_D_append(bin_spot_bid_price[i] / bin_fu_D_ask_price[i])
        # idx_num = contango_rate_by_D.index(max(contango_rate_by_D))
        # Coin_M_Sym = sym_name_D[idx_num]
        ##################### 콘탱코 #############################
        ##################### 자본대비 실제 들어갈 경우 사용량 ########################
        real_enter_amount_by_asset_ratio = []
        real_enter_amount_by_asset_ratio_append = real_enter_amount_by_asset_ratio.append
        for i in range(len(sym_name_D)):
            coin_M_order_amount = (expect_bin_spot_usdt / bin_spot_bid_price[i])

            bin_fu_D_Cont = math.floor(round((bin_fu_D_ask_price[i] * coin_M_order_amount / Coin_M_Cont[i]), int(Coin_M_Min_Cont[i])))
            real_enter_amount = bin_fu_D_Cont * Coin_M_Cont[i] / bin_fu_D_ask_price[i]

            real_enter_amount_by_asset_ratio_append((bin_spot_bid_price[i] * real_enter_amount) / (bin_fu_D_ask_price[i] * coin_M_order_amount))
        idx_num = real_enter_amount_by_asset_ratio.index(max(real_enter_amount_by_asset_ratio))
        #######################################################################################
        ##### 기존 자본 1배숏 들어갈 만한지 확인
        bin_fu_D_Cont_sym_name_a = math.floor(round((bin_fu_D_ask_price[sym_name_a_idx_num] * point_of_Withdrow_UP_amount_1 / Coin_M_Cont[sym_name_a_idx_num]), int(Coin_M_Min_Cont[sym_name_a_idx_num])))
        real_enter_amount_sym_name_a = bin_fu_D_Cont_sym_name_a * Coin_M_Cont[sym_name_a_idx_num] / bin_fu_D_ask_price[sym_name_a_idx_num]
        real_enter_amount_by_asset_ratio_sym_name_a = (bin_spot_bid_price[sym_name_a_idx_num] * real_enter_amount_sym_name_a) / (bin_fu_D_ask_price[sym_name_a_idx_num] * point_of_Withdrow_UP_amount_1)
        #####
        if (using_asset_ratio_by_Min_Max_Sclaer < real_enter_amount_by_asset_ratio_sym_name_a) and ((bin_spot_bid_price[sym_name_a_idx_num] * real_enter_amount_sym_name_a / (up_asset/Usdt)) >= (real_enter_amount_by_asset_ratio[idx_num] * expect_bin_spot_usdt / (up_asset/Usdt))):

            point_of_buy_bin_spot_amount_2 = point_of_Withdrow_UP_amount_1 #@@ 위로 올려도 되려나 ?

            ret = client.futures_account_transfer(asset=sym_name_D[sym_name_a_idx_num], amount = point_of_buy_bin_spot_amount_2, type=3, )  # 타입3:바현=>바선D #@@디미컬 라운딩 안해도 되려나..?소수점 6자리
            time.sleep(0.4)
            point_of_buy_bin_spot_ticker_2 = point_of_buy_UP_ticker_1
            bin_spot_to_coin_M_order_amount =point_of_Withdrow_UP_amount_1
            bin_fu_D_Cont_order = math.floor(round((bin_fu_D_ask_price[sym_name_a_idx_num] * bin_spot_to_coin_M_order_amount / Coin_M_Cont[sym_name_a_idx_num]), int(Coin_M_Min_Cont[sym_name_a_idx_num])))

            time.sleep(0.4)
            # @@ 추가 할지 말지
            # while True:
            #     if():#coim - m 의 계좌의 변화량이 보낸 변화량 만큼 올라간 경우 멈춤! 대신 바이낸스 api요청을 한번 더 해 야함
            #         break

            # result = request_client_D.post_order(symbol=sym_name_D[sym_name_a_idx_num] + "USD_PERP", side=OrderSide.SELL,
            #                                      ordertype=OrderType.MARKET,
            #                                      quantity=bin_fu_D_Cont_order)
            try:
                result = request_client_D.post_order(symbol=sym_name_D[sym_name_a_idx_num] + "USD_PERP", side=OrderSide.SELL,
                                                     ordertype=OrderType.MARKET, quantity=bin_fu_D_Cont_order)
            except:
                bin_fu_D_Cont_order = bin_fu_D_Cont_order - 1
                result = request_client_D.post_order(symbol=sym_name_D[sym_name_a_idx_num] + "USD_PERP", side=OrderSide.SELL,
                                                     ordertype=OrderType.MARKET, quantity=bin_fu_D_Cont_order)
            time.sleep(3)

            after_step_2_1_bin_spot_usdt_asset = float(next((item for item in client.get_account()['balances'] if item['asset'] == 'USDT'), None)['free'])
            before_step_2_1_bin_spot_usdt_asset = after_step_2_1_bin_spot_usdt_asset
            bin_spot_usdt = after_step_2_1_bin_spot_usdt_asset
            step_2_1_bin_spot_usdt_asset = before_step_2_1_bin_spot_usdt_asset - after_step_2_1_bin_spot_usdt_asset
            rest_step_2_1_bin_spot_usdt_asset = bin_spot_usdt - step_2_1_bin_spot_usdt_asset

            point_of_buy_bin_fu_ticker_1, point_of_buy_bin_fu_amount_1 = bin_fu_avg_buy_price_D([sym_name_D[sym_name_a_idx_num]])
            point_of_buy_bin_fu_ticker_1, point_of_buy_bin_fu_amount_1 = point_of_buy_bin_fu_ticker_1[0], point_of_buy_bin_fu_amount_1[0]
            if (bin_fu_previous_ticker_list_D[sym_name_a_idx_num] != 0):  # 이전에 구매한 코인이 아니라면(평단가는0 이 아님)
                bin_fu_previous_ticker, bin_fu_previous_amount = bin_fu_previous_ticker_list_D[sym_name_a_idx_num], bin_fu_previous_amount_list_D[sym_name_a_idx_num]  # 이전 평단가,수량
                bin_fu_now_ticker, bin_fu_now_amount = bin_fu_avg_buy_price_D([sym_name_D[sym_name_a_idx_num]])  # 이후 평단가,수량
                bin_fu_now_ticker, bin_fu_now_amount = bin_fu_now_ticker[0], bin_fu_now_amount[0]
                # (기존 평단가 * 기존 개수) + (point_of_buy_bin_spot_ticker_2  * point_of_buy_bin_spot_amount_2)/(모든 개수) = 바뀐평단가
                point_of_buy_bin_fu_amount_1 = bin_fu_now_amount - bin_fu_previous_amount
                point_of_buy_bin_fu_ticker_1 = ((bin_fu_now_ticker * bin_fu_now_amount) - (bin_fu_previous_amount * bin_fu_previous_ticker)) / (point_of_buy_bin_fu_amount_1)

            # 이후에 몇개가 사용가능량인지 확인#####################################
            aa = request_client_D.get_account_information()
            for i in range(len(aa.assets)):
                if (aa.assets[i].asset == sym_name_D[sym_name_a_idx_num]):
                    after_Avbl_amount = aa.assets[i].maxWithdrawAmount

            bb = request_client_D.get_account_trades(sym_name_D[sym_name_a_idx_num] + 'USD_PERP')  # coin-m 현재 들어가 있는 포지션 정보

            for i in range(len(bb)):
                if (bb[i].orderId == result.orderId):  # 여러번 주문이 들어간 상태일 수 있으니
                    point_of_buy_bin_fu_amount_1_D = bb[i].baseQty
                    point_of_buy_bin_fu_ticker_1_D = bb[i].price
                    point_of_buy_bin_fu_commission_1_D = bb[i].commission

            before_Avbl_amount = coin_m_asset_info[sym_name_D[sym_name_a_idx_num]]['Avbl_amount']
            Avbl_amount = after_Avbl_amount - before_Avbl_amount
            ####################################################################
            statuses = ("사용할 자본 (KRW)", bin_spot_usdt * Usdt,
                        "사용할 자본 (달러)", bin_spot_usdt,
                        "진입시점 당시 콘탱코,자본 비율", real_enter_amount_by_asset_ratio[sym_name_a_idx_num],
                        "고려해준 내 자본 민멕스 스칼라", using_asset_ratio_by_Min_Max_Sclaer,
                        "진입예상시점 서버시간", enter_servertime,
                        "바이낸스 Coin_M에 전송할 현물개수", bin_spot_to_coin_M_order_amount,
                        "진입할 심볼명", sym_name_D[sym_name_a_idx_num],
                        "현재 시각", datetime.datetime.now(),
                        "달러가치", Usdt,
                        "사용한 자본 량", step_2_1_bin_spot_usdt_asset,
                        "남은 자본 량", rest_step_2_1_bin_spot_usdt_asset,
                        "주문 코인 수량", point_of_buy_bin_spot_amount_2,
                        "주문 코인 평단가", point_of_buy_bin_spot_ticker_2,
                        "바이낸스 전송 된 심볼(coin_M 즉각 사용 시)바로 사용 시 사용될 자본 ", "바이낸스 전송 된 심볼(coin_M 즉각 사용 시)바로 사용 시 사용될 자본 비율",
                        bin_spot_bid_price[sym_name_a_idx_num] * real_enter_amount_sym_name_a, bin_spot_bid_price[sym_name_a_idx_num] * real_enter_amount_sym_name_a / (up_asset / Usdt),
                        "바이낸스 현물에서 매도 후 사용 시 사용될 예상 자본 ", "바이낸스 현물에서 매도 후 사용 시 사용될 예상 자본 비율",
                        real_enter_amount_by_asset_ratio[idx_num] * expect_bin_spot_usdt, real_enter_amount_by_asset_ratio[idx_num] * expect_bin_spot_usdt / (up_asset / Usdt),
                        "허가 된 범위에 따른 값", using_asset_ratio_by_Min_Max_Sclaer,
                        "자본대비 들어갈 비율", real_enter_amount_by_asset_ratio_sym_name_a,
                        "sym_name_D[idx_num]", sym_name_D[idx_num],
                        "sym_name_a", sym_name_a,
                        "sym_name_D[sym_name_a_idx_num]", sym_name_D[sym_name_a_idx_num]
                        )
            if (len(statuses) != 0):
                clear_output(wait=True)
                display(statuses)
            coin_M_order_amount = (bin_spot_usdt / bin_spot_bid_price[sym_name_a_idx_num])  # @@
            bin_fu_D_Cont = math.floor(round((bin_fu_D_ask_price[sym_name_a_idx_num] * coin_M_order_amount / Coin_M_Cont[sym_name_a_idx_num]), int(Coin_M_Min_Cont[sym_name_a_idx_num])))

            # 헷징 종료 후 바이낸스 현물, 선물 지갑 달러잔고 확인
            after_step_1_2_bin_spot_usdt_asset = float(next((item for item in client.get_account()['balances'] if item['asset'] == 'USDT'), None)['free'])
            step_1_2_bin_spot_usdt_asset = after_step_1_2_bin_spot_usdt_asset - before_step_1_2_bin_spot_usdt_asset
            x2 = point_of_Withdrow_UP_amount_1 * bin_fu_D_ask_price[sym_name_a_idx_num] * Usdt
            return "step_2_1",x2, step_1_2_bin_spot_usdt_asset, after_step_1_2_bin_spot_usdt_asset, before_step_1_2_bin_spot_usdt_asset, Usdt , enter_servertime, sym_name_D[sym_name_a_idx_num], bin_spot_to_coin_M_order_amount, step_2_1_bin_spot_usdt_asset, rest_step_2_1_bin_spot_usdt_asset, point_of_buy_bin_spot_amount_2, point_of_buy_bin_spot_ticker_2, Usdt, real_enter_amount_by_asset_ratio[sym_name_a_idx_num], datetime.datetime.now(), point_of_buy_bin_fu_ticker_1, point_of_buy_bin_fu_amount_1, bin_fu_D_Cont_order, Coin_M_Cont[sym_name_a_idx_num], bin_fu_D_Cont * Coin_M_Cont[sym_name_a_idx_num] / bin_fu_D_ask_price[sym_name_a_idx_num], result.orderId, "cheak_fundig_fee_severtime", Avbl_amount, point_of_buy_bin_fu_amount_1_D, point_of_buy_bin_fu_ticker_1_D, point_of_buy_bin_fu_commission_1_D

    # 2.바이낸스 & 업비트 포지션 종료
    # 잔고만큼 판매 주문 전송
    order = binance.create_market_sell_order(sym_name_a + '/USDT', order_amount_bin_spot)
    print(order)

    time.sleep(3)

    # 헷징 종료 후 바이낸스 현물, 선물 지갑 달러잔고 확인
    after_step_1_2_bin_spot_usdt_asset = float(next((item for item in client.get_account()['balances'] if item['asset'] == 'USDT'), None)['free'])
    step_1_2_bin_spot_usdt_asset = after_step_1_2_bin_spot_usdt_asset - before_step_1_2_bin_spot_usdt_asset
    x2 = step_1_2_bin_spot_usdt_asset * Usdt
    # # 팔시점 아직 안나옴

    statuses = (
                "허가 된 범위에 따른 값", using_asset_ratio_by_Min_Max_Sclaer,
                "자본대비 들어갈 비율", 'real_enter_amount_by_asset_ratio_sym_name_a',
                "sym_name_a", sym_name_a
                )
    if (len(statuses) != 0):
        clear_output(wait=True)
        display(statuses)

    return "step_1_2",x2, step_1_2_bin_spot_usdt_asset, after_step_1_2_bin_spot_usdt_asset, before_step_1_2_bin_spot_usdt_asset, Usdt , 'enter_servertime', 'sym_name_D[sym_name_a_idx_num]', 'bin_spot_to_coin_M_order_amount', 'step_2_1_bin_spot_usdt_asset', 'rest_step_2_1_bin_spot_usdt_asset', 'point_of_buy_bin_spot_amount_2', 'point_of_buy_bin_spot_ticker_2', 'Usdt_2_1', 'real_enter_amount_by_asset_ratio', 'datetime', 'point_of_buy_bin_fu_ticker_1', 'point_of_buy_bin_fu_amount_1', 'bin_fu_D_Cont_order', 'Coin_M_Cont', "들어간 자본 수량", '진입 아이디', "진입시점(서버)", 'Avbl_amount', 'point_of_buy_bin_fu_amount_1_D', 'point_of_buy_bin_fu_ticker_1_D', 'point_of_buy_bin_fu_commission_1_D'


In [14]:
def step_2_1(bin_spot_usdt, bin_spot_market_fee, ban_list):
    after_Avbl_amount, point_of_buy_bin_fu_amount_1_D, point_of_buy_bin_fu_ticker_1_D, point_of_buy_bin_fu_commission_1_D = 0, 0, 0, 0

    Usdt = upbit_get_usd_krw()  # up_api_cheak_2_1

    Usdt_MIN, Usdt_MAX, Out_MIN, Out_MAX = 120, 10000, 0.975, 1  # Usdt_MIN, Usdt_MAX ,Out_MIN ,Out_MAX = 120, 10000, 0.975, 1 #옵션 option
    using_asset_ratio_by_Min_Max_Sclaer = ((bin_spot_usdt - Usdt_MIN) * (Out_MAX - Out_MIN)) / (Usdt_MAX - Usdt_MIN) + Out_MIN

    sym_name_D = sym_intersection_D()  # 심볾먕 교집합 #  sym_name[i]+"USD_PERP" #bin_api_cheak_2_1 #bin_api_cheak_2_1 #bin_api_cheak_2_1 #up_api_cheak_2_1
    #     Coin_M_leverage_A_margin(sym_name_D)  # Coin-M 레버리지 및 마진 타입 변화 #bin_api_cheak_2_1 * 심볼 수
    sym_name_D = list(set(sym_name_D) - set(ban_list))  # @@이부분 최적 값 찾는거 다르게 바꾸기

    enter_servertime = request_client_D.get_servertime()  # 진입 시점 서버 시간 확인 #bin_api_cheak_2_1
    # enter_servertime < fundingTime[idx_num]

    ################# 몇개가 사전에 사용가능량인지 확인
    aa = request_client_D.get_account_information()  # bin_api_cheak_2_1
    coin_m_asset_info = {}
    for i in range(len(aa.assets)):
        if (aa.assets[i].asset in sym_name_D):
            coin_m_asset_info[aa.assets[i].asset] = {"Avbl_amount": aa.assets[i].maxWithdrawAmount}
    ##############################################

    before_step_2_1_bin_spot_usdt_asset = float(next((item for item in client.get_account()['balances'] if item['asset'] == 'USDT'), None)['free'])  # bin_api_cheak_2_1

    Bin_Spot_Minimum_Order_Size = Bin_Minimum_Spot_Order_Size(sym_name_D)  # 바이낸스 현물 최소 주문량 #bin_api_cheak_2_1
    Coin_M_Cont, Coin_M_Min_Cont = Bin_Fu_Minimum_Order_Cont_n_Size_D(sym_name_D)  # 바낸 coin-m 필요 정보(coin-m최소 주문량) #bin_api_cheak_2_1
    bin_fu_previous_ticker_list_D, bin_fu_previous_amount_list_D = bin_fu_avg_buy_price_D(sym_name_D)  # 이전 모든 평단가, 진입가 #bin_api_cheak_2_1

    bin_sym_names, bin_spot_ask_bid = bin_spot_ask_bid_info(sym_name_D)  # bin_api_cheak_2_1
    # bin_spot_ask_price,bin_spot_ask_size = bin_ask_info(bin_sym_names,bin_spot_ask_bid)#바-현 매수 호가창 2
    bin_spot_bid_price, bin_spot_bid_size = bin_bid_info(bin_sym_names, bin_spot_ask_bid)  # 바-현 매도 호가창 3

    bin_sym_names_D, bin_fu_ask_bid_D = bin_fu_ask_bid_info_D(sym_name_D)  # bin_api_cheak_2_1
    bin_fu_D_ask_price, bin_fu_D_ask_size = bin_fu_ask_info_D(bin_sym_names_D, bin_fu_ask_bid_D)  # 바-선D 매수 호가창
    # bin_fu_D_bid_price, bin_fu_D_bid_size = bin_fu_bid_info_D(bin_sym_names_D, bin_fu_ask_bid_D)  # 바-선D 매도 호가창

    ##################### 콘탱코 #############################
    contango_rate_by_D = []  # backwardation
    # 펀비는 선물가격이 현물보다 높을 시에 받는 것이므로 선물가격이 높은곳에 가서 1배숏을 한다.
    for i in range(len(sym_name_D)):
        contango_rate_by_D.append(bin_spot_bid_price[i] / bin_fu_D_ask_price[i])
    # idx_num = contango_rate_by_D.index(max(contango_rate_by_D))
    # Coin_M_Sym = sym_name_D[idx_num]
    ##################### 콘탱코 #############################
    ##################### 자본대비 실제 들어갈 경우 사용량 ########################
    real_enter_amount_by_asset_ratio = []
    for i in range(len(sym_name_D)):
        coin_M_order_amount = (bin_spot_usdt / bin_spot_bid_price[i])

        bin_fu_D_Cont = math.floor(round((bin_fu_D_ask_price[i] * coin_M_order_amount / Coin_M_Cont[i]), int(Coin_M_Min_Cont[i])))
        real_enter_amount = bin_fu_D_Cont * Coin_M_Cont[i] / bin_fu_D_ask_price[i]

        real_enter_amount_by_asset_ratio.append((bin_spot_bid_price[i] * real_enter_amount) / (bin_fu_D_ask_price[i] * coin_M_order_amount))
    idx_num = real_enter_amount_by_asset_ratio.index(max(real_enter_amount_by_asset_ratio))
    ##################### 자본대비 실제 들어갈 경우 사용량 ########################

    # 3. 바이낸스 현물 구매주문######################################################################################

    bin_spot_Usdt_Send = (bin_spot_usdt) - (bin_spot_usdt) * bin_spot_market_fee  # 바이낸스 현물 시장가 수수료
    send_to_up_quantity = (bin_spot_Usdt_Send / bin_spot_bid_price[idx_num])  # 바이낸스 업비트로 보낼 달러 / 티커 가격 = 수량

    # 전송가능 최소량과 주문가능 최소량으로 잘 자름.
    Decimal_rounding = Bin_Spot_Minimum_Order_Size[idx_num]
    bin_spot_to_coin_M_order_amount = (send_to_up_quantity // Decimal_rounding) * Decimal_rounding
    bin_spot_to_coin_M_order_amount = round(bin_spot_to_coin_M_order_amount, 6)
    ##########################################################################################
    # 자본이 적당히 들어갈 수 있는 경우만 진입 (0.05%이상의 자본이 남으면 진입 X ) , 들어 갈 수 있는 양에 펑션 실행(사용할 자본 보다 쓸 자본이 적어야 함[슬리피지]), 콘탱코가 1이상이어야 펀비를 받을 수 있으니
    if (((bin_spot_Usdt_Send - bin_spot_to_coin_M_order_amount * bin_spot_bid_price[idx_num]) < (bin_spot_usdt * 0.002)) and ((bin_spot_usdt * (1 - bin_spot_market_fee)) > (bin_spot_to_coin_M_order_amount * bin_spot_bid_price[idx_num])) and (
            using_asset_ratio_by_Min_Max_Sclaer < real_enter_amount_by_asset_ratio[idx_num])):
        # if (((bin_spot_Usdt_Send - bin_spot_to_coin_M_order_amount * bin_spot_bid_price[idx_num]) < (bin_spot_usdt * 0.002)) and ((bin_spot_usdt * (1 - bin_spot_market_fee)) > (bin_spot_to_coin_M_order_amount * bin_spot_bid_price[idx_num])) and (using_asset_ratio_by_Min_Max_Sclaer < real_enter_amount_by_asset_ratio[idx_num]) and ((contango_rate_by_D[idx_num] > 1))):

        # 주문 이후
        order = binance.create_market_buy_order(sym_name_D[idx_num] + '/USDT', bin_spot_to_coin_M_order_amount)  # bin_api_cheak_2_1

        # 평단가 가져옮
        try:  # 시장가 주문의 경우
            point_of_buy_bin_spot_ticker_2 = order["average"]
            point_of_buy_bin_spot_amount_2 = order["amount"] - order['fee']['cost']
        except:  # 지정가 주문의 경우
            point_of_buy_bin_spot_ticker_2 = bin_spot_bid_price[idx_num]
            point_of_buy_bin_spot_amount_2 = bin_spot_to_coin_M_order_amount

        time.sleep(0.4)

        ret = client.futures_account_transfer(asset=sym_name_D[idx_num], amount=point_of_buy_bin_spot_amount_2, type=3, )  # 타입3:바현=>바선D #bin_api_cheak_2_1

        bin_fu_D_Cont_order = math.floor(round((bin_fu_D_ask_price[idx_num] * bin_spot_to_coin_M_order_amount / Coin_M_Cont[idx_num]), int(Coin_M_Min_Cont[idx_num])))

        time.sleep(0.4)
        # @@ 추가 할지 말지
        # while True:
        #     if():#coim - m 의 계좌의 변화량이 보낸 변화량 만큼 올라간 경우 멈춤! 대신 바이낸스 api요청을 한번 더 해 야함
        #         break

        # result = request_client_D.post_order(symbol=sym_name_D[idx_num] + "USD_PERP", side=OrderSide.SELL,
        #                                      ordertype=OrderType.MARKET,
        #                                      quantity=bin_fu_D_Cont_order)
        try:
            result = request_client_D.post_order(symbol=sym_name_D[idx_num] + "USD_PERP", side=OrderSide.SELL,
                                                 ordertype=OrderType.MARKET, quantity=bin_fu_D_Cont_order)
        except:
            bin_fu_D_Cont_order = bin_fu_D_Cont_order - 1
            result = request_client_D.post_order(symbol=sym_name_D[idx_num] + "USD_PERP", side=OrderSide.SELL,
                                                 ordertype=OrderType.MARKET, quantity=bin_fu_D_Cont_order)
        # bin_api_cheak_2_1

        time.sleep(0.4)

        after_step_2_1_bin_spot_usdt_asset = float(next((item for item in client.get_account()['balances'] if item['asset'] == 'USDT'), None)['free'])  # bin_api_cheak_2_1
        step_2_1_bin_spot_usdt_asset = before_step_2_1_bin_spot_usdt_asset - after_step_2_1_bin_spot_usdt_asset
        rest_step_2_1_bin_spot_usdt_asset = bin_spot_usdt - step_2_1_bin_spot_usdt_asset

        point_of_buy_bin_fu_ticker_1, point_of_buy_bin_fu_amount_1 = bin_fu_avg_buy_price_D([sym_name_D[idx_num]])
        point_of_buy_bin_fu_ticker_1, point_of_buy_bin_fu_amount_1 = point_of_buy_bin_fu_ticker_1[0], point_of_buy_bin_fu_amount_1[0]
        if (bin_fu_previous_ticker_list_D[idx_num] != 0):  # 이전에 구매한 코인이 아니라면(평단가는0 이 아님)
            bin_fu_previous_ticker, bin_fu_previous_amount = bin_fu_previous_ticker_list_D[idx_num], bin_fu_previous_amount_list_D[idx_num]  # 이전 평단가,수량
            bin_fu_now_ticker, bin_fu_now_amount = bin_fu_avg_buy_price_D([sym_name_D[idx_num]])  # 이후 평단가,수량 #bin_api_cheak_2_1
            bin_fu_now_ticker, bin_fu_now_amount = bin_fu_now_ticker[0], bin_fu_now_amount[0]
            # (기존 평단가 * 기존 개수) + (point_of_buy_bin_spot_ticker_2  * point_of_buy_bin_spot_amount_2)/(모든 개수) = 바뀐평단가
            point_of_buy_bin_fu_amount_1 = bin_fu_now_amount - bin_fu_previous_amount
            point_of_buy_bin_fu_ticker_1 = ((bin_fu_now_ticker * bin_fu_now_amount) - (bin_fu_previous_amount * bin_fu_previous_ticker)) / (point_of_buy_bin_fu_amount_1)

        # 이후에 몇개가 사용가능량인지 확인#####################################
        aa = request_client_D.get_account_information()  # bin_api_cheak_2_1
        for i in range(len(aa.assets)):
            if (aa.assets[i].asset == sym_name_D[idx_num]):
                after_Avbl_amount = aa.assets[i].maxWithdrawAmount

        bb = request_client_D.get_account_trades(sym_name_D[idx_num] + 'USD_PERP')  # coin-m 현재 들어가 있는 포지션 정보 #bin_api_cheak_2_1

        for i in range(len(bb)):
            if (bb[i].orderId == result.orderId):  # 여러번 주문이 들어간 상태일 수 있으니
                point_of_buy_bin_fu_amount_1_D = bb[i].baseQty
                point_of_buy_bin_fu_ticker_1_D = bb[i].price
                point_of_buy_bin_fu_commission_1_D = bb[i].commission

        before_Avbl_amount = coin_m_asset_info[sym_name_D[idx_num]]['Avbl_amount']
        Avbl_amount = after_Avbl_amount - before_Avbl_amount
        ####################################################################
        statuses = ("사용할 자본 (KRW)", bin_spot_usdt * Usdt,
                    "사용할 자본 (달러)", bin_spot_usdt,
                    "진입시점 당시 콘탱코,자본 비율", real_enter_amount_by_asset_ratio[idx_num],
                    "고려해준 내 자본 민멕스 스칼라", using_asset_ratio_by_Min_Max_Sclaer,
                    "진입예상시점 서버시간", enter_servertime,
                    "바이낸스 Coin_M에 전송할 현물개수", bin_spot_to_coin_M_order_amount,
                    "진입할 심볼명", sym_name_D[idx_num],
                    "현재 시각", datetime.datetime.now(),
                    "달러가치", Usdt,
                    "사용한 자본 량", step_2_1_bin_spot_usdt_asset,
                    "남은 자본 량", rest_step_2_1_bin_spot_usdt_asset,
                    "주문 코인 수량", point_of_buy_bin_spot_amount_2,
                    "주문 코인 평단가", point_of_buy_bin_spot_ticker_2
                    )
        if (len(statuses) != 0):
            clear_output(wait=True)
            display(statuses)
        coin_M_order_amount = (bin_spot_usdt / bin_spot_bid_price[idx_num])  # @@
        bin_fu_D_Cont = math.floor(round((bin_fu_D_ask_price[idx_num] * coin_M_order_amount / Coin_M_Cont[idx_num]), int(Coin_M_Min_Cont[idx_num])))

        return "step_2_1", enter_servertime, sym_name_D[idx_num], bin_spot_to_coin_M_order_amount, step_2_1_bin_spot_usdt_asset, rest_step_2_1_bin_spot_usdt_asset, point_of_buy_bin_spot_amount_2, point_of_buy_bin_spot_ticker_2, Usdt, real_enter_amount_by_asset_ratio[idx_num], datetime.datetime.now(), point_of_buy_bin_fu_ticker_1, point_of_buy_bin_fu_amount_1, bin_fu_D_Cont_order, Coin_M_Cont[idx_num], bin_fu_D_Cont * Coin_M_Cont[idx_num] / bin_fu_D_ask_price[idx_num], result.orderId, "cheak_fundig_fee_severtime", Avbl_amount, point_of_buy_bin_fu_amount_1_D, point_of_buy_bin_fu_ticker_1_D, point_of_buy_bin_fu_commission_1_D
        # print()
    else:
        after_step_2_1_bin_spot_usdt_asset = float(next((item for item in client.get_account()['balances'] if item['asset'] == 'USDT'), None)['free'])  # bin_api_cheak_2_1
        step_2_1_bin_spot_usdt_asset = before_step_2_1_bin_spot_usdt_asset - after_step_2_1_bin_spot_usdt_asset
        rest_step_2_1_bin_spot_usdt_asset = bin_spot_usdt - step_2_1_bin_spot_usdt_asset

        statuses = ("사용할 자본 (KRW)", bin_spot_usdt * Usdt,
                    "사용할 자본 (달러)", bin_spot_usdt,
                    "진입시점 당시 콘탱코,자본 비율", real_enter_amount_by_asset_ratio[idx_num],
                    "고려해준 내 자본 민멕스 스칼라", using_asset_ratio_by_Min_Max_Sclaer,
                    "진입예상시점 서버시간", enter_servertime,
                    "바이낸스 Coin_M에 전송할 현물개수", bin_spot_to_coin_M_order_amount,
                    "진입할 심볼명", sym_name_D[idx_num],
                    "현재 시각", datetime.datetime.now(),
                    "달러가치", Usdt,
                    "사용한 자본 량", step_2_1_bin_spot_usdt_asset,
                    "남은 자본 량", rest_step_2_1_bin_spot_usdt_asset,
                    "주문 코인 수량", bin_spot_to_coin_M_order_amount,
                    "주문 코인 예상 평단가", bin_spot_bid_price[idx_num]
                    )
        if (len(statuses) != 0):
            clear_output(wait=True)
            display(statuses)
        coin_M_order_amount = (bin_spot_usdt / bin_spot_bid_price[idx_num])  # @@
        bin_fu_D_Cont = math.floor(round((bin_fu_D_ask_price[idx_num] * coin_M_order_amount / Coin_M_Cont[idx_num]), int(Coin_M_Min_Cont[idx_num])))
        return 'step_1_2', enter_servertime, sym_name_D[idx_num], bin_spot_to_coin_M_order_amount, step_2_1_bin_spot_usdt_asset, rest_step_2_1_bin_spot_usdt_asset, 'point_of_buy_bin_spot_amount_2', 'point_of_buy_bin_spot_ticker_2', Usdt, real_enter_amount_by_asset_ratio[idx_num], datetime.datetime.now(), 'point_of_buy_bin_fu_ticker_1', 'point_of_buy_bin_fu_amount_1', 'bin_fu_D_Cont_order', Coin_M_Cont[idx_num], bin_fu_D_Cont * Coin_M_Cont[idx_num] / bin_fu_D_ask_price[idx_num], ' result.orderId', "cheak_fundig_fee_severtime", 'Avbl_amount', 'point_of_buy_bin_fu_amount_1_D', 'point_of_buy_bin_fu_ticker_1_D', 'point_of_buy_bin_fu_commission_1_D'
        # print()

In [15]:
def step_2_2_sym_name(bin_spot_usdt):
    sym_name = sym_intersection()
    all_coin_info = client_spot.coin_info()
    Usdt = upbit_get_usd_krw()

    sym_name = delet_None_network(all_coin_info, sym_name)  # 코인 스스로 전송 불가능 한 코인은 제외(바이낸스 체인 ,bnb만 전송가능한코인)
    sym_name = del_bin_disable_withdraw(all_coin_info, sym_name)  # 바이낸스 출금 불가능 심볼 삭제
#     sym_name = del_bin_disable_withdrow_deopsit(all_coin_info, sym_name) #@@ 바이낸스 입출금 불가 코인 삭제
    sym_name = del_up_disable_deopsit(sym_name)  # 업비트 입금 불가능 심볼 삭제
    time.sleep(1)

    # @@여기서 바이낸스 자본은 원화??
    sym_name = remove_sym_name_2(all_coin_info, sym_name, Usdt, bin_spot_usdt, 0.005, User_data['ban_list'], User_data['bin_spot_market_fee'])  # 내 자본 대비 전송수수료(0.005)가 큰 코인 심볼명 삭제
    print("자본대비 쓸 수 있는 심볼 갯수", len(sym_name))

    return sym_name

In [16]:
def step_2_2(up_asset,sym_name, sym_name_D_ ,point_of_buy_bin_fu_amount_1_D, point_of_buy_bin_fu_ticker_1_D, Avbl_amount , point_of_buy_bin_spot_ticker_2, point_of_buy_bin_spot_amount_2, bin_fu_D_Cont_order, div_unit, want_premium_step_2_2, up_market_fee, bin_Coin_M_future_market_fee, bin_spot_market_fee):
    Usdt = upbit_get_usd_krw()
    all_coin_info = client_spot.coin_info()
    ########################coin-m 해당 계좌 전송가능 심볼량 조회
    aa = request_client_D.get_account_information()
    coin_m_asset_info = {}
    for i in range(len(aa.assets)):
        if (aa.assets[i].asset in sym_name_D_):
            coin_m_asset_info[aa.assets[i].asset] = {"Avbl_amount": aa.assets[i].maxWithdrawAmount}
    before_bin_USD_Avbl_amount = coin_m_asset_info[sym_name_D_]['Avbl_amount']
    #####

    up_withdrawfee, bin_withdrawfee = up_bin_withdraw_fee(sym_name, all_coin_info)  # 업비트 전송 수수료 , 바이낸스 전송 수수료
    up_Min_Withdraw_Size, bin_Min_Withdraw_Size = up_bin_withdraw_Size(sym_name,all_coin_info)  # 업비트, 바이낸스 전송최소 가능량
    Bin_Fu_Minimum_Order_Size = Bin_Minimum_FU_Order_Size_D(sym_name)  # 바이낸스 선물 최소 주문 가능량
    Bin_Spot_Minimum_Order_Size = Bin_Minimum_Spot_Order_Size(sym_name)  # 바이낸스 현물 최소 주문가능량

    while True:
        idx_num = None
        for i in range(len(sym_name)):  # @@ 전송불가 코인이 되어 심볼네임에서 안나올 수 있음
            if (sym_name[i] == sym_name_D_):
                idx_num = i
        if (idx_num == None):
            break
        else:
            break
    # 사용전 바이낸스 현물 달러 자본 가져옴
    before_step_2_2_bin_spot_usdt_asset = float(next((item for item in client.get_account()['balances'] if item['asset'] == 'USDT'), None)['free'])
    # 호가창 정보 가져옴
    up_sym_names, up_spot_ask_bid = up_spot_ask_bid_info(sym_name)
    up_ask_price, up_ask_size = up_ask_info(up_sym_names, up_spot_ask_bid)  # 업비트 매수 호가창 4
    # up_bid_price, up_bid_size = up_bid_info(up_sym_names,up_spot_ask_bid) #업비트 매도 호가창 1

    bin_sym_names, bin_spot_ask_bid = bin_spot_ask_bid_info(sym_name)
    bin_spot_ask_price, bin_spot_ask_size = bin_ask_info(bin_sym_names, bin_spot_ask_bid)  # 바-현 매수 호가창 2
    bin_spot_bid_price, bin_spot_bid_size = bin_bid_info(bin_sym_names, bin_spot_ask_bid)  # 바-현 매도 호가창 3

    bin_sym_names_D, bin_fu_ask_bid_D = bin_fu_ask_bid_info_D([sym_name_D_])
    # bin_fu_D_ask_price, bin_fu_D_ask_size = bin_fu_ask_info_D(bin_sym_names_D, bin_fu_ask_bid_D)  # 바-선D 매수 호가창
    bin_fu_D_bid_price, bin_fu_D_bid_size = bin_fu_bid_info_D(bin_sym_names_D, bin_fu_ask_bid_D)  # 바-선D 매도 호가창
    # if(sym_name[b] != sym_name[idx_num]):
    #     print(sym_name[b],sym_name[idx_num])
    # else:
    #     clear_output(wait=True)
    #     display(statuses)

    if (idx_num != None): #심볼이 전송가능 할 경우
        # coin-m 판매 후 남을 예측 수량 (현물 전송 전)
        x3_D_amount = point_of_buy_bin_fu_amount_1_D * (point_of_buy_bin_fu_ticker_1_D / bin_fu_D_bid_price[0]) * (1 - up_market_fee) + Avbl_amount
        # 업비트에서 판매 시
        # 내 심볼 갯수 업비트 판매가 - 전송 수수료
        x4_a = x3_D_amount * up_ask_price[idx_num] * (1 - up_market_fee) - bin_withdrawfee[idx_num]

        # 전송 바로 하지 않고 한번 팔았을때의 예상 달러.
        aa = point_of_buy_bin_fu_amount_1_D * (point_of_buy_bin_fu_ticker_1_D / bin_fu_D_bid_price[0]) * (1 - bin_Coin_M_future_market_fee) * bin_spot_ask_price[idx_num] * (1 - bin_spot_market_fee)
        bb = Avbl_amount * (bin_spot_ask_price[idx_num] / (point_of_buy_bin_spot_ticker_2 * Usdt)) * (1 - bin_Coin_M_future_market_fee)  # @@point_of_buy_bin_spot_ticker_2 고쳐야함
        Second_Use_Bin_Spot_Usdt = aa + bb  # @@@ 이부분 정확한 사용량으로..!

        # bin_to_up_pre_rate_D = up_ask_price[idx_num] / (bin_spot_bid_price[idx_num] * Usdt)
        # just_go_D = bin_to_up_pre_rate_D - ((bin_spot_bid_price[idx_num] * bin_withdrawfee[idx_num]) / (Second_Use_Bin_Spot_Usdt))

        # up_to_bin_pre_rate = []
        bin_to_up_pre_rate = []
        bin_to_up_pre_rate_append = bin_to_up_pre_rate.append
        # up_withdrawfee_by_asset = []
        bin_withdrawfee_by_asset = []
        bin_withdrawfee_by_asset_append = bin_withdrawfee_by_asset.append

        for i in range(len(sym_name)):
            # 프리미엄들(역프,김프)
            #     up_to_bin_pre_rate.append((bin_spot_ask_price[i]*Usdt)/up_bid_price[i]) # 역프 or 바낸 전송시 예상 손수익 %(김프시 손해)
            bin_to_up_pre_rate_append(up_ask_price[i] / (bin_spot_bid_price[i] * Usdt))  # 김프 or 업비트 전송시 예상 손수익 %(김프시 수익)

            # 자본대비 수수료 오직 비율로만 따짐.
            #     up_withdrawfee_by_asset.append(up_to_bin_pre_rate[i]-((up_bid_price[i] * up_withdrawfee[i])/up_asset)) #내 자본 대비 전송수수료를 포함하여 가장 좋은 최적의 값 서칭
            bin_withdrawfee_by_asset_append(bin_to_up_pre_rate[i] - ((bin_spot_bid_price[i] * bin_withdrawfee[i]) / (Second_Use_Bin_Spot_Usdt)))  # 바이낸스 선물에서 매수 할거니 매도호가창조회

        # 나중에 a는 변수명 최적_to_bin으로 바꾸기?b도
        # (업 => 바)전송시 최적 코인 인덱스 번호(a) , (바 => 업)전송 시 최적 코인 인덕스 번호(b)
        # a = up_withdrawfee_by_asset.index(max(up_withdrawfee_by_asset)) # 리스트에서 가장 큰 값 뽑음(업 => 바 가장 큰 값 인덱스) @@맞지? 현재 김프역프 왔다리 갓다리 하니까
        b = bin_withdrawfee_by_asset.index(max(bin_withdrawfee_by_asset))  # 리스트에서 가장 큰 값 뽑음(바 => 업 가장 큰 값 인덱스)

        # 3. 바이낸스 현물 구매주문######################################################################################
        bin_spot_Usdt_Send = (Second_Use_Bin_Spot_Usdt) - (Second_Use_Bin_Spot_Usdt) * bin_spot_market_fee  # 바이낸스 현물 시장가 수수료
        send_to_up_quantity = (bin_spot_Usdt_Send / bin_spot_bid_price[b])  # 바이낸스 업비트로 보낼 달러 / 티커 가격 = 수량
        send_to_up_quantity = send_to_up_quantity + bin_withdrawfee[b]  # 이부분 맞나 =>maybe 이부분 때문에 자본의 비중을 현물에 더 두었음
        # 전송가능 최소량과 주문가능 최소량으로 잘 자름.
        Decimal_rounding = bin_Min_Withdraw_Size[b]
        if (Decimal_rounding == 0):  # 전송 수수료가 0 인경우 나눌 수가 없음
            pass
        else:
            send_to_up_quantity = (send_to_up_quantity // Decimal_rounding) * Decimal_rounding

        Decimal_rounding = Bin_Spot_Minimum_Order_Size[b]
        bin_spot_to_up_can_order_amount = (send_to_up_quantity // Decimal_rounding) * Decimal_rounding
        bin_spot_to_up_can_order_amount = round(bin_spot_to_up_can_order_amount, 6)
        ###############################################################################################################

        # x3_b = Second_Use_Bin_Spot_Usdt * (1 - bin_spot_market_fee) - (bin_spot_bid_price[b] * bin_withdrawfee[b])
        x3_b = bin_spot_to_up_can_order_amount * bin_spot_bid_price[b] * (1 - bin_spot_market_fee) - (bin_spot_bid_price[b] * bin_withdrawfee[b])
        x4 = x3_b * (up_ask_price[b] / (bin_spot_bid_price[b] * Usdt)) * (1 - up_market_fee) * Usdt

        if ((x4 / up_asset > want_premium_step_2_2) or (x4_a / up_asset > want_premium_step_2_2)):

            if (x4 < x4_a):  # 그냥 전송

                # Coin-M 1배숏 포지션 정리
                result = request_client_D.post_order(symbol=sym_name_D_ + "USD_PERP", side=OrderSide.BUY,
                                                     ordertype=OrderType.MARKET,
                                                     quantity=bin_fu_D_Cont_order)
                PrintBasic.print_obj(result)

                ########################coin-m 해당 계좌 전송가능 심볼량 조회
                aa = request_client_D.get_account_information()
                coin_m_asset_info = {}
                for i in range(len(aa.assets)):
                    if (aa.assets[i].asset in sym_name_D_):
                        coin_m_asset_info[aa.assets[i].asset] = {"Avbl_amount": aa.assets[i].maxWithdrawAmount}
                after_bin_USD_Avbl_amount = coin_m_asset_info[sym_name_D_]['Avbl_amount']
                #####
                # 실제 가지고 있는 해당 심볼 양 #@@result 값으로 도출 가능하면 result 값으로 하는것이 좋아보임!!
                bin_USD_Avbl_amount = after_bin_USD_Avbl_amount - before_bin_USD_Avbl_amount + Avbl_amount
                #####
                #####coin-m 남은 수량 쓰는 로직 @@ 수정 필요할 수도.
                same_sym_name_D = []  # same_sym_name_D
                for i in range(len(asset_division)):  # step_2_1인 심볼들만 가져와서
                    if (asset_division[i]['State'] == "step_2_1"):
                        same_sym_name_D.append(asset_division[i]['sym_name_D'])

                for i in range(len(asset_division)):  # step_2_1이고 중복 포지션이 없는 경우 마지막 남은 자본을 다 사용한다
                    if (asset_division[i]['State'] == "step_2_1" and asset_division[i]['State'] in same_sym_name_D):
                        pass
                    else:
                        bin_USD_Avbl_amount = after_bin_USD_Avbl_amount
                #####
                ##### 정확한 전송가능량 도출
                quotePrecision = 8  ## @@ 옵션
                Decimal_rounding = (0.1) ** float(quotePrecision)  # @@맞는지..?
                bin_USD_Avbl_amount = (bin_USD_Avbl_amount // Decimal_rounding) * Decimal_rounding
                bin_USD_Avbl_amount = round(bin_USD_Avbl_amount, quotePrecision)

                ret = client.futures_account_transfer(asset=sym_name_D_, amount=bin_USD_Avbl_amount, type=4, )  # 타입4:바선D=>바현
                time.sleep(0.4)
                #             print("바로 업비트 전송하는 코드")

                point_of_buy_bin_spot_ticker_2_2 = point_of_buy_bin_spot_ticker_2  # @@ 중복변수
                point_of_buy_bin_spot_amount_2_2 = bin_USD_Avbl_amount  # @@ 중복변수

                up_add, up_seadd = up_deposit_address_request(sym_name[idx_num])  ##업비트 주소 요청

                ##### 잔고조회로 남는 코인자본 전송 같이 시킴
                can_send_amount = float(next((item for item in binance.fetch_balance()['info']['balances'] if item['asset'] == sym_name[idx_num]), None)['free'])
                if (0 < ((can_send_amount - point_of_buy_bin_spot_amount_2_2) * point_of_buy_bin_spot_ticker_2_2 * Usdt) < (div_unit / 5)):
                    pass  # can_send_amount 가 자기자신이 된다 (남은 자본 보냄)
                else:
                    can_send_amount = point_of_buy_bin_spot_amount_2_2  # can_send_amount 가 정확히 구매한양이 된다 point_of_buy_bin_spot_amount_2
                #####

                #####전송수수료를 제외한 양과 포함한 양 도출
                real_send_amount = can_send_amount - bin_withdrawfee[idx_num]  # 구매한 수량과 전송수수료를 포함하지 않은 양으로 확인
                Decimal_rounding = bin_Min_Withdraw_Size[idx_num]
                if (Decimal_rounding == 0):  # 전송 수수료가 0 인경우 나눌 수가 없음
                    pass
                else:
                    real_send_amount = (real_send_amount // Decimal_rounding) * Decimal_rounding
                real_send_amount = round(real_send_amount, 6)
                point_of_Withdrow_UP_amount_2 = real_send_amount + bin_withdrawfee[idx_num]  # 구매한 수량과 전송수수료를 포함한 양을 전송요청
                #####

                statuses = ('x4_a', x4_a,
                            'x4', x4,
                            "바현에서 판매 시 예측 되는 가격", Second_Use_Bin_Spot_Usdt,
                            "달러가치", Usdt,
                            "바이낸스 현물 자본 사용량", point_of_buy_bin_spot_amount_2_2 * bin_spot_bid_price[b] * Usdt,
                            "평균김프(%)", (1 - (sum(bin_to_up_pre_rate) / len(bin_to_up_pre_rate))) * 100
                            )
                if (len(statuses) != 0):
                    clear_output(wait=True)
                    display(statuses)

                return 'step_2_2', sym_name_D_, point_of_buy_bin_spot_ticker_2_2, point_of_buy_bin_spot_amount_2_2, point_of_Withdrow_UP_amount_2, real_send_amount, up_add, up_seadd, Bin_Spot_Minimum_Order_Size[b], x4_a / Usdt, Usdt, Second_Use_Bin_Spot_Usdt, bin_Min_Withdraw_Size[b], bin_withdrawfee[b], 'step_2_2_bin_spot_usdt_asset', 'rest_step_2_2_bin_spot_usdt_asset', can_send_amount

            elif (x4 >= x4_a) and (bin_spot_to_up_can_order_amount * bin_spot_bid_price[b] < Second_Use_Bin_Spot_Usdt):  # and (x4 / up_asset > want_premium_step_2_2)  # 전송불가 코인의 경우 , 다시 사서 업비트에서 매도 시 수익인 경우

                # Coin-M 1배숏 포지션 정리
                result = request_client_D.post_order(symbol=sym_name_D_ + "USD_PERP", side=OrderSide.BUY,
                                                     ordertype=OrderType.MARKET,
                                                     quantity=bin_fu_D_Cont_order)
                PrintBasic.print_obj(result)

                ########################coin-m 해당 계좌 전송가능 심볼량 조회
                aa = request_client_D.get_account_information()
                coin_m_asset_info = {}
                for i in range(len(aa.assets)):
                    if (aa.assets[i].asset in sym_name_D_):
                        coin_m_asset_info[aa.assets[i].asset] = {"Avbl_amount": aa.assets[i].maxWithdrawAmount}
                after_bin_USD_Avbl_amount = coin_m_asset_info[sym_name_D_]['Avbl_amount']
                #####
                # 실제 가지고 있는 해당 심볼 양 #@@result 값으로 도출 가능하면 result 값으로 하는것이 좋아보임!!
                bin_USD_Avbl_amount = after_bin_USD_Avbl_amount - before_bin_USD_Avbl_amount + Avbl_amount
                #####
                #####coin-m 남은 수량 쓰는 로직 @@ 수정 필요할 수도.
                same_sym_name_D = []  # same_sym_name_D
                same_sym_name_D_append = same_sym_name_D.append
                for i in range(len(asset_division)):  # step_2_1인 심볼들만 가져와서
                    if (asset_division[i]['State'] == "step_2_1"):
                        same_sym_name_D_append(asset_division[i]['sym_name_D'])

                for i in range(len(asset_division)):  # step_2_1이고 중복 포지션이 없는 경우 마지막 남은 자본을 다 사용한다
                    if (asset_division[i]['State'] == "step_2_1" and asset_division[i]['State'] in same_sym_name_D):
                        pass
                    else:
                        bin_USD_Avbl_amount = after_bin_USD_Avbl_amount
                #####
                ##### 정확한 전송가능량 도출
                quotePrecision = 8  ## @@ 옵션
                Decimal_rounding = (0.1) ** float(quotePrecision)  # @@맞는지..?
                bin_USD_Avbl_amount = (bin_USD_Avbl_amount // Decimal_rounding) * Decimal_rounding
                bin_USD_Avbl_amount = round(bin_USD_Avbl_amount, quotePrecision)

                #################################################################

                ret = client.futures_account_transfer(asset=sym_name_D_, amount=bin_USD_Avbl_amount, type=4, )  # 타입4:바선D=>바현
                time.sleep(0.4)
                order = binance.create_market_sell_order(sym_name_D_ + '/USDT', bin_USD_Avbl_amount)
                print(order)

                after_step_2_2_A_bin_spot_usdt_asset = float(next((item for item in client.get_account()['balances'] if item['asset'] == 'USDT'), None)['free'])
                step_2_2_A_bin_spot_usdt_asset = after_step_2_2_A_bin_spot_usdt_asset - before_step_2_2_bin_spot_usdt_asset  # 여기서부터 step_2_2_A_bin_spot_usdt_asset 은 세컨드 유징 어셋 이랑 같음

                # 3. 바이낸스 현물 구매주문######################################################################################
                bin_spot_Usdt_Send = (step_2_2_A_bin_spot_usdt_asset) - (step_2_2_A_bin_spot_usdt_asset) * bin_spot_market_fee  # 바이낸스 현물 시장가 수수료
                send_to_up_quantity = (bin_spot_Usdt_Send / bin_spot_bid_price[b])  # 바이낸스 업비트로 보낼 달러 / 티커 가격 = 수량
                send_to_up_quantity = send_to_up_quantity + bin_withdrawfee[b]  # 이부분 맞나 =>maybe 이부분 때문에 자본의 비중을 현물에 더 두었음
                # 전송가능 최소량과 주문가능 최소량으로 잘 자름.
                Decimal_rounding = bin_Min_Withdraw_Size[b]
                if (Decimal_rounding == 0):  # 전송 수수료가 0 인경우 나눌 수가 없음
                    pass
                else:
                    send_to_up_quantity = (send_to_up_quantity // Decimal_rounding) * Decimal_rounding

                Decimal_rounding = Bin_Spot_Minimum_Order_Size[b]
                bin_spot_to_up_can_order_amount = (send_to_up_quantity // Decimal_rounding) * Decimal_rounding
                bin_spot_to_up_can_order_amount = round(bin_spot_to_up_can_order_amount, 6)
                #############################################################################################################

                #################################################################
                #         3. 바이낸스 현물 주문
                order = binance.create_market_buy_order(sym_name[b] + '/USDT', bin_spot_to_up_can_order_amount)

                point_of_buy_bin_spot_ticker_2_2 = order["average"]
                point_of_buy_bin_spot_amount_2_2 = order["amount"] - order['fee']['cost']

                up_add, up_seadd = up_deposit_address_request(sym_name[b])  ##업비트 주소 요청

                ##### 잔고조회로 남는 코인자본 전송 같이 시킴
                can_send_amount = float(next((item for item in binance.fetch_balance()['info']['balances'] if item['asset'] == sym_name[b]), None)['free'])
                if (0 < ((can_send_amount - point_of_buy_bin_spot_amount_2_2) * point_of_buy_bin_spot_ticker_2_2 * Usdt) < (div_unit / 5)):
                    pass  # can_send_amount 가 자기자신이 된다 (남은 자본 보냄)
                else:
                    can_send_amount = point_of_buy_bin_spot_amount_2_2  # can_send_amount 가 정확히 구매한양이 된다 point_of_buy_bin_spot_amount_2
                #####

                #####전송수수료를 제외한 양과 포함한 양 도출
                real_send_amount = can_send_amount - bin_withdrawfee[b]  # 구매한 수량과 전송수수료를 포함하지 않은 양으로 확인
                Decimal_rounding = bin_Min_Withdraw_Size[b]
                if (Decimal_rounding == 0):  # 전송 수수료가 0 인경우 나눌 수가 없음
                    pass
                else:
                    real_send_amount = (real_send_amount // Decimal_rounding) * Decimal_rounding
                real_send_amount = round(real_send_amount, 6)
                point_of_Withdrow_UP_amount_2 = real_send_amount + bin_withdrawfee[b]  # 구매한 수량과 전송수수료를 포함한 양을 전송요청
                #####

                print("새로 사서 전송하는 코드")
                after_step_2_2_bin_spot_usdt_asset = float(next((item for item in client.get_account()['balances'] if item['asset'] == 'USDT'), None)['free'])
                step_2_2_bin_spot_usdt_asset = before_step_2_2_bin_spot_usdt_asset - after_step_2_2_bin_spot_usdt_asset
                rest_step_2_2_bin_spot_usdt_asset = step_2_2_A_bin_spot_usdt_asset - step_2_2_bin_spot_usdt_asset

                statuses = ('x4_a', x4_a,
                            'x4', x4,
                            "바현에서 판매 시 가격", step_2_2_bin_spot_usdt_asset,
                            "달러가치", Usdt,
                            "바이낸스 현물 자본 사용량", bin_spot_to_up_can_order_amount * bin_spot_bid_price[b] * Usdt,
                            "평균김프(%)", (1 - (sum(bin_to_up_pre_rate) / len(bin_to_up_pre_rate))) * 100
                            )
                if (len(statuses) != 0):
                    clear_output(wait=True)
                    display(statuses)

                return 'step_2_2', sym_name[b], point_of_buy_bin_spot_ticker_2_2, point_of_buy_bin_spot_amount_2_2, point_of_Withdrow_UP_amount_2, real_send_amount, up_add, up_seadd, Bin_Spot_Minimum_Order_Size[b], x4 / Usdt, Usdt, Second_Use_Bin_Spot_Usdt, bin_Min_Withdraw_Size[b], bin_withdrawfee[b], step_2_2_bin_spot_usdt_asset, rest_step_2_2_bin_spot_usdt_asset, can_send_amount

        return 'step_2_1', 'sym_name[b]', 'point_of_buy_bin_spot_ticker_2_2', 'point_of_buy_bin_spot_amount_2_2', 'point_of_Withdrow_UP_amount_2', 'real_send_amount', 'up_add', 'up_seadd', 'Bin_Spot_Minimum_Order_Size[b]', 'x3 / Usdt', Usdt, 'second_using_asset', bin_Min_Withdraw_Size[b], bin_withdrawfee[b], 'step_2_2_bin_spot_usdt_asset', 'rest_step_2_2_bin_spot_usdt_asset', 'can_send_amount'


    # elif(idx_num == None):  # 전송불가 코인이 되버린 경우(idx_num == None)는 다시 사서 전송하는 방식만 고려한다 아래식처럼 하면 절대 그냥전송 안함
    else:
        x4_a = 1

        bin_sym_names_A, bin_spot_ask_bid_A = bin_spot_ask_bid_info([sym_name_D_])
        bin_spot_ask_price_A, bin_spot_ask_size_A = bin_ask_info(bin_sym_names_A, bin_spot_ask_bid_A)  # 바-현 매수 호가창 2
        bin_spot_bid_price_A, bin_spot_bid_size_A = bin_bid_info(bin_sym_names_A, bin_spot_ask_bid_A)  # 바-현 매도 호가창 3

        # 전송 바로 하지 않고 한번 팔았을때의 예상 달러.
        aa = point_of_buy_bin_fu_amount_1_D * (point_of_buy_bin_fu_ticker_1_D / bin_fu_D_bid_price[0]) * (1 - bin_Coin_M_future_market_fee) * bin_spot_ask_price_A[0] * (1 - bin_spot_market_fee)
        bb = Avbl_amount * (bin_spot_bid_price_A[0] / (point_of_buy_bin_spot_ticker_2 * Usdt)) * (1 - bin_Coin_M_future_market_fee)  # @@point_of_buy_bin_spot_ticker_2 고쳐야함
        Second_Use_Bin_Spot_Usdt = aa + bb  # @@@ 이부분 정확한 사용량으로..!

        # bin_to_up_pre_rate_D = up_ask_price[   ] / (bin_spot_bid_price[   ] * Usdt)
        # just_go_D = bin_to_up_pre_rate_D - ((bin_spot_bid_price[   ] * bin_withdrawfee[   ]) / (Second_Use_Bin_Spot_Usdt))

        # up_to_bin_pre_rate = []
        bin_to_up_pre_rate = []
        bin_to_up_pre_rate_append = bin_to_up_pre_rate.append
        # up_withdrawfee_by_asset = []
        bin_withdrawfee_by_asset = []
        bin_withdrawfee_by_asset_append = bin_withdrawfee_by_asset.append

        for i in range(len(sym_name)):
            # 프리미엄들(역프,김프)
            #     up_to_bin_pre_rate.append((bin_spot_ask_price[i]*Usdt)/up_bid_price[i]) # 역프 or 바낸 전송시 예상 손수익 %(김프시 손해)
            bin_to_up_pre_rate_append(up_ask_price[i] / (bin_spot_bid_price[i] * Usdt))  # 김프 or 업비트 전송시 예상 손수익 %(김프시 수익)

            # 자본대비 수수료 오직 비율로만 따짐.
            #     up_withdrawfee_by_asset.append(up_to_bin_pre_rate[i]-((up_bid_price[i] * up_withdrawfee[i])/up_asset)) #내 자본 대비 전송수수료를 포함하여 가장 좋은 최적의 값 서칭
            bin_withdrawfee_by_asset_append(bin_to_up_pre_rate[i] - ((bin_spot_bid_price[i] * bin_withdrawfee[i]) / (Second_Use_Bin_Spot_Usdt)))  # 바이낸스 선물에서 매수 할거니 매도호가창조회

        # 나중에 a는 변수명 최적_to_bin으로 바꾸기?b도
        # (업 => 바)전송시 최적 코인 인덱스 번호(a) , (바 => 업)전송 시 최적 코인 인덕스 번호(b)
        # a = up_withdrawfee_by_asset.index(max(up_withdrawfee_by_asset)) # 리스트에서 가장 큰 값 뽑음(업 => 바 가장 큰 값 인덱스) @@맞지? 현재 김프역프 왔다리 갓다리 하니까
        b = bin_withdrawfee_by_asset.index(max(bin_withdrawfee_by_asset))  # 리스트에서 가장 큰 값 뽑음(바 => 업 가장 큰 값 인덱스)

        # 3. 바이낸스 현물 구매주문######################################################################################
        bin_spot_Usdt_Send = (Second_Use_Bin_Spot_Usdt) - (Second_Use_Bin_Spot_Usdt) * bin_spot_market_fee  # 바이낸스 현물 시장가 수수료
        send_to_up_quantity = (bin_spot_Usdt_Send / bin_spot_bid_price[b])  # 바이낸스 업비트로 보낼 달러 / 티커 가격 = 수량
        send_to_up_quantity = send_to_up_quantity + bin_withdrawfee[b]  # 이부분 맞나 =>maybe 이부분 때문에 자본의 비중을 현물에 더 두었음
        # 전송가능 최소량과 주문가능 최소량으로 잘 자름.
        Decimal_rounding = bin_Min_Withdraw_Size[b]
        if (Decimal_rounding == 0):  # 전송 수수료가 0 인경우 나눌 수가 없음
            pass
        else:
            send_to_up_quantity = (send_to_up_quantity // Decimal_rounding) * Decimal_rounding

        Decimal_rounding = Bin_Spot_Minimum_Order_Size[b]
        bin_spot_to_up_can_order_amount = (send_to_up_quantity // Decimal_rounding) * Decimal_rounding
        bin_spot_to_up_can_order_amount = round(bin_spot_to_up_can_order_amount, 6)
        ###############################################################################################################

        # x3_b = Second_Use_Bin_Spot_Usdt * (1 - bin_spot_market_fee) - (bin_spot_bid_price[b] * bin_withdrawfee[b])
        x3_b = bin_spot_to_up_can_order_amount * bin_spot_bid_price[b] * (1 - bin_spot_market_fee) - (bin_spot_bid_price[b] * bin_withdrawfee[b])
        x4 = x3_b * (up_ask_price[b] / (bin_spot_bid_price[b] * Usdt)) * (1 - up_market_fee) * Usdt

        if (x4 >= x4_a) and (bin_spot_to_up_can_order_amount * bin_spot_bid_price[b] < Second_Use_Bin_Spot_Usdt) and (x4 / up_asset > want_premium_step_2_2):  # and (x4 / up_asset > want_premium_step_2_2)  # 전송불가 코인의 경우 , 다시 사서 업비트에서 매도 시 수익인 경우

            # Coin-M 1배숏 포지션 정리
            result = request_client_D.post_order(symbol=sym_name_D_ + "USD_PERP", side=OrderSide.BUY,
                                                 ordertype=OrderType.MARKET,
                                                 quantity=bin_fu_D_Cont_order)
            PrintBasic.print_obj(result)

            ########################coin-m 해당 계좌 전송가능 심볼량 조회
            aa = request_client_D.get_account_information()
            coin_m_asset_info = {}
            for i in range(len(aa.assets)):
                if (aa.assets[i].asset in sym_name_D_):
                    coin_m_asset_info[aa.assets[i].asset] = {"Avbl_amount": aa.assets[i].maxWithdrawAmount}
            after_bin_USD_Avbl_amount = coin_m_asset_info[sym_name_D_]['Avbl_amount']
            #####
            # 실제 가지고 있는 해당 심볼 양 #@@result 값으로 도출 가능하면 result 값으로 하는것이 좋아보임!!
            bin_USD_Avbl_amount = after_bin_USD_Avbl_amount - before_bin_USD_Avbl_amount + Avbl_amount
            #####
            #####coin-m 남은 수량 쓰는 로직 @@ 수정 필요할 수도.
            same_sym_name_D = []  # same_sym_name_D
            same_sym_name_D_append = same_sym_name_D.append
            for i in range(len(asset_division)):  # step_2_1인 심볼들만 가져와서
                if (asset_division[i]['State'] == "step_2_1"):
                    same_sym_name_D_append(asset_division[i]['sym_name_D'])

            for i in range(len(asset_division)):  # step_2_1이고 중복 포지션이 없는 경우 마지막 남은 자본을 다 사용한다
                if (asset_division[i]['State'] == "step_2_1" and asset_division[i]['State'] in same_sym_name_D):
                    pass
                else:
                    bin_USD_Avbl_amount = after_bin_USD_Avbl_amount
            #####
            ##### 정확한 전송가능량 도출
            quotePrecision = 8  ## @@ 옵션
            Decimal_rounding = (0.1) ** float(quotePrecision)  # @@맞는지..?
            bin_USD_Avbl_amount = (bin_USD_Avbl_amount // Decimal_rounding) * Decimal_rounding
            bin_USD_Avbl_amount = round(bin_USD_Avbl_amount, quotePrecision)

            #################################################################

            ret = client.futures_account_transfer(asset=sym_name_D_, amount=bin_USD_Avbl_amount, type=4, )  # 타입4:바선D=>바현
            time.sleep(0.4)
            order = binance.create_market_sell_order(sym_name_D_ + '/USDT', bin_USD_Avbl_amount)
            print(order)

            after_step_2_2_A_bin_spot_usdt_asset = float(next((item for item in client.get_account()['balances'] if item['asset'] == 'USDT'), None)['free'])
            step_2_2_A_bin_spot_usdt_asset = after_step_2_2_A_bin_spot_usdt_asset - before_step_2_2_bin_spot_usdt_asset# 여기서부터 step_2_2_A_bin_spot_usdt_asset 은 세컨드 유징 어셋 이랑 같음

            # 3. 바이낸스 현물 구매주문######################################################################################
            bin_spot_Usdt_Send = (step_2_2_A_bin_spot_usdt_asset) - (step_2_2_A_bin_spot_usdt_asset) * bin_spot_market_fee  # 바이낸스 현물 시장가 수수료
            send_to_up_quantity = (bin_spot_Usdt_Send / bin_spot_bid_price[b])  # 바이낸스 업비트로 보낼 달러 / 티커 가격 = 수량
            send_to_up_quantity = send_to_up_quantity + bin_withdrawfee[b]  # 이부분 맞나 =>maybe 이부분 때문에 자본의 비중을 현물에 더 두었음
            # 전송가능 최소량과 주문가능 최소량으로 잘 자름.
            Decimal_rounding = bin_Min_Withdraw_Size[b]
            if (Decimal_rounding == 0):  # 전송 수수료가 0 인경우 나눌 수가 없음
                pass
            else:
                send_to_up_quantity = (send_to_up_quantity // Decimal_rounding) * Decimal_rounding

            Decimal_rounding = Bin_Spot_Minimum_Order_Size[b]
            bin_spot_to_up_can_order_amount = (send_to_up_quantity // Decimal_rounding) * Decimal_rounding
            bin_spot_to_up_can_order_amount = round(bin_spot_to_up_can_order_amount, 6)
            #############################################################################################################

            #################################################################
            #         3. 바이낸스 현물 주문
            order = binance.create_market_buy_order(sym_name[b] + '/USDT', bin_spot_to_up_can_order_amount)

            point_of_buy_bin_spot_ticker_2_2 = order["average"]
            point_of_buy_bin_spot_amount_2_2 = order["amount"] - order['fee']['cost']

            up_add, up_seadd = up_deposit_address_request(sym_name[b])  ##업비트 주소 요청

            ##### 잔고조회로 남는 코인자본 전송 같이 시킴
            can_send_amount = float(next((item for item in binance.fetch_balance()['info']['balances'] if item['asset'] == sym_name[b]), None)['free'])
            if (0 < ((can_send_amount - point_of_buy_bin_spot_amount_2_2) * point_of_buy_bin_spot_ticker_2_2 * Usdt) < (div_unit / 5)):
                pass  # can_send_amount 가 자기자신이 된다 (남은 자본 보냄)
            else:
                can_send_amount = point_of_buy_bin_spot_amount_2_2  # can_send_amount 가 정확히 구매한양이 된다 point_of_buy_bin_spot_amount_2
            #####


            #####전송수수료를 제외한 양과 포함한 양 도출
            real_send_amount = can_send_amount - bin_withdrawfee[b]  # 구매한 수량과 전송수수료를 포함하지 않은 양으로 확인
            Decimal_rounding = bin_Min_Withdraw_Size[b]
            if (Decimal_rounding == 0):  # 전송 수수료가 0 인경우 나눌 수가 없음
                pass
            else:
                real_send_amount = (real_send_amount // Decimal_rounding) * Decimal_rounding
            real_send_amount = round(real_send_amount, 6)
            point_of_Withdrow_UP_amount_2 = real_send_amount + bin_withdrawfee[b]  # 구매한 수량과 전송수수료를 포함한 양을 전송요청
            #####

            print("새로 사서 전송하는 코드")
            after_step_2_2_bin_spot_usdt_asset = float(next((item for item in client.get_account()['balances'] if item['asset'] == 'USDT'), None)['free'])
            step_2_2_bin_spot_usdt_asset = before_step_2_2_bin_spot_usdt_asset - after_step_2_2_bin_spot_usdt_asset
            rest_step_2_2_bin_spot_usdt_asset = step_2_2_A_bin_spot_usdt_asset - step_2_2_bin_spot_usdt_asset

            statuses = ('x4_a', x4_a,
                        'x4', x4,
                        "바현에서 판매 시 가격", step_2_2_bin_spot_usdt_asset,
                        "달러가치", Usdt,
                        "바이낸스 현물 자본 사용량", bin_spot_to_up_can_order_amount * bin_spot_bid_price[b] * Usdt,
                        "평균김프(%)", (1 - (sum(bin_to_up_pre_rate) / len(bin_to_up_pre_rate))) * 100
                        )
            if (len(statuses) != 0):
                clear_output(wait=True)
                display(statuses)

            return 'step_2_2', sym_name[b], point_of_buy_bin_spot_ticker_2_2, point_of_buy_bin_spot_amount_2_2, point_of_Withdrow_UP_amount_2, real_send_amount, up_add, up_seadd, Bin_Spot_Minimum_Order_Size[b], x4 / Usdt, Usdt, 'Second_Use_Bin_Spot_Usdt', bin_Min_Withdraw_Size[b], bin_withdrawfee[b], step_2_2_bin_spot_usdt_asset, rest_step_2_2_bin_spot_usdt_asset, can_send_amount
        else:
            return 'step_2_1', 'sym_name[b]', 'point_of_buy_bin_spot_ticker_2_2', 'point_of_buy_bin_spot_amount_2_2', 'point_of_Withdrow_UP_amount_2', 'real_send_amount', 'up_add', 'up_seadd', 'Bin_Spot_Minimum_Order_Size[b]', 'x3 / Usdt', Usdt, 'second_using_asset', bin_Min_Withdraw_Size[b], bin_withdrawfee[b], 'step_2_2_bin_spot_usdt_asset', 'rest_step_2_2_bin_spot_usdt_asset', 'can_send_amount'

In [17]:
def send_bin_to_up(send_to_up_optimal_simbol, up_add, up_seadd, real_send_amount, point_of_Withdrow_UP_amount_2):  # 바이낸스에서 업비트 전송주문 요청

    send_state_2 = [up_add, up_seadd, real_send_amount, "전송실패"]
    # 바 => 업 전송, 오류날 경우를 대비

    try:
        if (send_to_up_optimal_simbol == 'BHC'):  # BCH코인은 주소가 좀 이상하게 나옴, 그리고 태그도 없어야 전송이 됨.
            up_add = up_add[12:]
            res = client_spot.withdraw(coin=send_to_up_optimal_simbol, amount=point_of_Withdrow_UP_amount_2, address=up_add, addressTag=up_seadd)
            # addressTag=up_seadd)

            send_state_2 = res
            print(res)
            return 'send_bin_to_up', upbit.get_balance(send_to_up_optimal_simbol)
            # 성공시 send_bin_to_up

        if (up_seadd == None):

            res = client_spot.withdraw(coin=send_to_up_optimal_simbol, amount=point_of_Withdrow_UP_amount_2, address=up_add, )
            # 두번째 주소가 없는경우 그 양식에 맞게 전송주문을 제출 하여야 함.
            # addressTag=up_seadd)

            send_state_2 = res
            print(res)
            return 'send_bin_to_up', upbit.get_balance(send_to_up_optimal_simbol)
            # 성공시 send_bin_to_up

        else:
            # 바이낸스 코인 전송

            res = client_spot.withdraw(coin=send_to_up_optimal_simbol, amount=point_of_Withdrow_UP_amount_2, address=up_add, addressTag=up_seadd)
            send_state_2 = res
            print(res)
            return 'send_bin_to_up', upbit.get_balance(send_to_up_optimal_simbol)
            # 성공시 send_bin_to_up

    except:

        statuses = (send_state_2)

        if (len(statuses) != 0):
            clear_output(wait=True)
            display(statuses)
        return 'step_2_2', "보내기 전 업비트 잔고"

In [18]:
def send_bin_to_up_Completion(send_to_up_optimal_simbol, real_send_amount, before_send_to_up_balace, Bin_Spot_Minimum_Order_Size_b):  # 바이낸스에서 업비트 전송완료

    up_simbol_balance = upbit.get_balance("KRW-" + send_to_up_optimal_simbol)
    
    quotePrecision = 8  ## @@ 옵션
    aa = real_send_amount + before_send_to_up_balace
    Decimal_rounding = Bin_Spot_Minimum_Order_Size_b
    aa = (aa // Decimal_rounding) * Decimal_rounding
    aa = round(aa, quotePrecision)
    
    if (up_simbol_balance >= aa):
        print("업비트", send_to_up_optimal_simbol, "전송 완료", up_simbol_balance)
        return 'send_bin_to_up_Completion'

    # 계좌 잔고 확인 option

    statuses = ("바이낸스 => 업비트", send_to_up_optimal_simbol, "전송 중", up_simbol_balance,
                "현재시각", time.ctime(time.time()))

    if (len(statuses) != 0):
        clear_output(wait=True)
        display(statuses)

    time.sleep(1)
    return 'send_bin_to_up'

In [19]:
def step_3(up_asset, sym_name, real_send_amount, div_unit , up_market_fee):  # 전송완료 후 예상 손해보다 덜 보는 상태에서 매도
    Usdt = upbit_get_usd_krw()

    ##전송 후
    up_sym_names, up_spot_ask_bid = up_spot_ask_bid_info([sym_name])
    up_ask_price, up_ask_size = up_ask_info(up_sym_names, up_spot_ask_bid)  # 업비트 매수 호가창 4

    ##### @@ 남는자본 + div_unit / 5 보다 작은 잔고가 존재할 시 찌꺼기로 생각하고 포함해서 계산
    rest_sym = upbit.get_balance("KRW-" + sym_name)  # (upbit.get_balance("KRW-" + sym_name) 이부분 업비트 요청 1번으로 바꾸기 위해 변수명 생성
    before_step_3_up_krw_asset = upbit.get_balance("KRW")

    if (0 < (rest_sym - real_send_amount) * up_ask_price[0] < (div_unit / 5)):  # 0< 인 이유는 다른 자본 사용 시 혹시모를 경우 대비(마지막에 찌꺼기 처럼 남은 자본)
        # @@주문가능 최소량
        Decimal_rounding = 0.00000001
        real_send_amount = (rest_sym // Decimal_rounding) * Decimal_rounding
    else:
        pass
    #####

    # 2. 바이낸스 현물에서 매도 #예상수익 역프 : up_to_bin_pre_rate
    x4 = real_send_amount * up_ask_price[0] * (1 - up_market_fee)

    will_Realize_Arbitrage = x4 - up_asset
    will_Realize_Arbitrage_rate = x4 / up_asset
    # 기대수익 보다 높거나 차익 실현율이 0.5% 이상이면 종료 #호가창 안전한지 화인 (x3은 수수료 제외한 양.)

    # option
    # if (((expect_x4 + expect_y4) <= (x4 + y4)) and (will_Realize_Arbitrage_rate > 1.007) and (x3 < up_ask_price[b] * up_ask_size[b] * 0.5)):
    #     if ((will_Realize_Arbitrage_rate > 1.005) and (x3 < up_ask_price[0] * up_ask_size[0] * 0.5)):
    if ((will_Realize_Arbitrage_rate > 1.01) and (x4 < up_ask_price[0] * up_ask_size[0] * 0.5)):
        #         print("매도주문하는 코드 넣기")
        # 여기서 오류가 난다면 업비트 최소 매수 가능금액 소수점 자르기(Decimal_rounding)
        #         @@ 이 부분 남은 잔고가 div_unit보다 작으면 잔고확인으로 주문들어가기?
        #         up_simbol_balance = upbit.get_balance("KRW-" + sym_name)
        #         res = upbit.sell_market_order("KRW-" + sym_name, up_simbol_balance)
        res = upbit.sell_market_order("KRW-" + sym_name, real_send_amount)

        pprint.pprint(res)

        after_step_3_up_krw_asset = upbit.get_balance("KRW")
        step_3_up_krw_asset = after_step_3_up_krw_asset - before_step_3_up_krw_asset

        return 'step_3', will_Realize_Arbitrage, will_Realize_Arbitrage_rate, step_3_up_krw_asset, x4 / Usdt, Usdt


    else:
        # 팔시점 아직 안나옴
        statuses = ("실제 수익", will_Realize_Arbitrage,
                    "수익율", will_Realize_Arbitrage_rate,
                    "(거래후)업비트 예상자본", x4
                    )
        if (len(statuses) != 0):
            clear_output(wait=True)
            display(statuses)
    time.sleep(1)

    # 바이낸스 현물가 , 바이낸스 선물가 , 실제 전송 후 손해 ,최적이였던 전송코인

    return 'send_bin_to_up_Completion', will_Realize_Arbitrage, will_Realize_Arbitrage_rate, 'step_3_up_krw_asset', x4, Usdt

In [20]:
def final_fun(qwer, save_state, sicle, k, up_asset, step_3_up_krw_asset, rest_step_1_1_up_krw_asset, sym_name_D, point_of_buy_bin_spot_amount_2, point_of_buy_bin_spot_ticker_2, rest_step_2_1_bin_spot_usdt_asset, sym_name_b, can_send_amount, point_of_Withdrow_UP_amount_2,point_of_buy_bin_spot_ticker_2_2, Usdt_first, point_of_buy_UP_ticker_1, point_of_buy_UP_amount_1, sym_name_a, step_1_1_up_krw_asset, up_withdrawfee_by_asset, point_of_Withdrow_UP_amount_1, Usdt_1_1, step_1_2_bin_spot_usdt_asset, enter_servertime, Usdt_1_2,bin_spot_to_coin_M_order_amount, step_2_1_bin_spot_usdt_asset, Usdt_2_1, real_enter_amount_by_asset_ratio, point_of_bin_enter_time, point_of_buy_bin_fu_ticker_1, point_of_buy_bin_fu_amount_1, bin_fu_D_Cont_order, Coin_M_Cont_idx_num, real_enter_amount, bin_fu_D_Order_ID, Avbl_amount, point_of_buy_bin_fu_ticker_1_D, point_of_buy_bin_fu_amount_1_D, point_of_buy_bin_fu_commission_1_D, point_of_buy_bin_spot_amount_2_2, real_send_amount, Bin_Spot_Minimum_Order_Size_b, Usdt_2_2, Second_Use_Bin_Spot_Usdt, bin_Min_Withdraw_Size_b, bin_withdrawfee_b,step_2_2_bin_spot_usdt_asset, rest_step_2_2_bin_spot_usdt_asset, x4, Usdt_step_3, future_leverage):
    now = datetime.datetime.now()
    date = str(now.year) + "Y_" + str(now.month) + "M_" + str(now.day) + "D_" + str(now.hour) + "h_" + str(now.minute) + "m_" + str(now.second) + "s"
    total_before = up_asset

    # @@ total_after=> step_1_1에서 남는돈 , step_2_1에서 남는돈 합해야 정확함
    total_after = step_3_up_krw_asset + rest_step_1_1_up_krw_asset
    Realize_Arbitrage = total_after - up_asset
    Realize_Arbitrage_rate = total_after / up_asset
    # x2=step_1_2_bin_spot_usdt_asset,y2=step_1_2_bin_fu_usdt_asset,x4=step_2_2_up_krw_asset,y4=step_2_2_bin_fu_usdt_asset

    save_state[sicle] = [sicle,
                         total_before,
                         total_after,
                         date,
                         # 현물에 남은 자본
                         k,  # 현재 asset_div_사이클
                         sym_name_D,  # step_2_1에서 진입한 coin_M 심볼명
                         point_of_buy_bin_spot_amount_2,  # step_2_1에서 진입한 coin_M 수량
                         point_of_buy_bin_spot_ticker_2,  # step_2_1에서 진입한 coin_M 평단가

                         rest_step_2_1_bin_spot_usdt_asset,  # Alpha_asset 바이낸스 달러 지갑에 남은 돈..()

                         sym_name_b,  # step_2_2에서 남은 coin_M 심볼명
                         can_send_amount - point_of_Withdrow_UP_amount_2,  # 현재 남은 coin_M => spot 수량
                         point_of_buy_bin_spot_ticker_2_2,  # 현재 남은 coin_M => spot 평단가
                         up_asset,
                         future_leverage,
                         Usdt_first,

                         point_of_buy_UP_ticker_1,  # 업비트 매수 평단가
                         point_of_buy_UP_amount_1,  # 업비트 매수 수량
                         sym_name_a,

                         step_1_1_up_krw_asset,  # 실제 사용자본 (변화량 으로 비교)
                         rest_step_1_1_up_krw_asset,  # 남은 사용할 자본 (변화량 으로 비교)

                         up_withdrawfee_by_asset,  # 업비트=> 바이낸스 손/수익 (김프 + 전송 수수료)
                         point_of_Withdrow_UP_amount_1,  # 업비트에서 전송할 수량(변화량 으로 비교)
                         Usdt_1_1,

                         step_1_2_bin_spot_usdt_asset,  # 바이낸스에서 팔고 남은 자본(변화량 으로 비교)
                         enter_servertime,
                         Usdt_1_2,

                         sym_name_D,  # 새로 들어갈 심볼명 ()
                         bin_spot_to_coin_M_order_amount,  # 실제로 들어간 수량(Coin_M)
                         step_2_1_bin_spot_usdt_asset,  # 바이낸스 사용한 현물 달러 (변화량으로 비교한 값)
                         rest_step_2_1_bin_spot_usdt_asset,  # 바이낸스 현물 달러에서 쓰고 남은 달러(step2_1사용량 - 변화량)
                         point_of_buy_bin_spot_amount_2,  # 바이낸스 현물 자본 사용량
                         point_of_buy_bin_spot_ticker_2,  # 바이낸스 현물 평단가
                         Usdt_2_1,  # 달러가치
                         real_enter_amount_by_asset_ratio,  # 사용가능 자본대비 실제로 산 비율(step_2_1)
                         point_of_bin_enter_time,  # 진입시점 날짜
                         point_of_buy_bin_fu_ticker_1,  # 바이낸스 coin_M 선물 진입가
                         point_of_buy_bin_fu_amount_1,  # 바이낸스 coin_M 선물 수량
                         bin_fu_D_Cont_order,  # 바이낸스 진입 계수
                         Coin_M_Cont_idx_num,  # 바이낸스 cont =  N * USD의 N
                         real_enter_amount,  # 실제로 진입된 수량
                         bin_fu_D_Order_ID,  # 바이낸스 coin_M 진입 아이디
                         Avbl_amount,  # 쓰고 남았던 사용가능한 수량
                         point_of_buy_bin_fu_ticker_1_D,  # 바이낸스 coin_M 평단가
                         point_of_buy_bin_fu_amount_1_D,  # 바이낸스 coin_M 수량
                         point_of_buy_bin_fu_commission_1_D,  # 바이낸스 coin_M 진입수수료

                         sym_name_b,  # 선택받은 심볼명 (그냥 보내든가 아니면 새로 사서 전송하던가 )
                         point_of_buy_bin_spot_ticker_2_2,  # 현물에서 산 평단가 (x_a의 경우엔 새로 산 평단가)
                         point_of_buy_bin_spot_amount_2_2,  # 현물에서 산 수량 (x_a의 경우엔 새로 산 수량)
                         point_of_Withdrow_UP_amount_2,  # 현물에서 산 수량 (x_a의 경우엔 새로 산 수량)
                         real_send_amount,  # 업비트에 보낼 진짜 수량
                         Bin_Spot_Minimum_Order_Size_b,  # 바이낸스 현물 최소 주문단위
                         Usdt_2_2,  # 달러
                         Second_Use_Bin_Spot_Usdt,  # 두번째 바이낸스 현물에서 사용할 달러
                         bin_Min_Withdraw_Size_b,  # 바이낸스 현물 전송 최소 수량
                         bin_withdrawfee_b,  # 바이내스=>업비트 드는 전송 수수료
                         step_2_2_bin_spot_usdt_asset,  # 주석일 수 있음 => 해당 경우엔 _a의 경우
                         rest_step_2_2_bin_spot_usdt_asset,  # 주석일 수 있음 => 해당 경우엔 _a의 경우

                         Realize_Arbitrage,  # 실제로 볼 수익
                         Realize_Arbitrage_rate,  # 실제로 볼 수익율
                         step_3_up_krw_asset,  # step_3 업비트 자본 = 최종 자본
                         x4,  # 예상 최종자본
                         Usdt_step_3  # step_3 달러가치
                         ]

    # csv파일로 적기 # newline 설정을 안하면 한줄마다 공백있는 줄이 생긴다.
    with open(qwer + 'Trading_data.csv', 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(save_state[sicle])

    sicle = sicle + 1
    save_state[sicle] = []

    # @@ total_after=> step_1_1에서 남는돈 , step_2_1에서 남는돈 합해야 정확함
    up_asset = step_3_up_krw_asset + rest_step_1_1_up_krw_asset
    if (rest_step_2_1_bin_spot_usdt_asset == "rest_step_2_1_bin_spot_usdt_asset"):
        rest_step_2_1_bin_spot_usdt_asset = 0
    if (rest_step_2_2_bin_spot_usdt_asset == "rest_step_2_1_bin_spot_usdt_asset"):
        rest_step_2_2_bin_spot_usdt_asset = 0
    binance_Alpha_asset = rest_step_2_1_bin_spot_usdt_asset + rest_step_2_2_bin_spot_usdt_asset##@@이거 rest_step_2_2_bin_spot_usdt_asset가 숫자가 안나올수 있음

    return 'None_position', save_state, up_asset, binance_Alpha_asset

In [118]:
while True:  # 업비트 => 바이낸스 Coin_M
    # coim_M한배숏들어가기 전까지 하나씩의 자본만 운용함
    All_States = [asset_division[i]["State"] for i in range(len(asset_division))]  # 모든 States를 가져옮
    print(4)
    for i in range(len(asset_division)):
        if (len(set(All_States) & {'step_1_1','send_up_to_bin', 'send_up_to_bin_Completion', 'step_1_2'}) != 0):#교집합이 0일 경우(모든상태가 저기에 포함되지 않을경우)
            print(7)
            pass
        elif (asset_division[i]["State"] == "None_position"):  # 모든 asset_division의 State가 coim_M포지션 진입 전이라면.
            print(10)
            all_coin_info = client_spot.coin_info()  # 코인 정보를 전부 한번만 가져와서 저장함
            #sym_name = sym_intersection()  # 업비트 현물 , 바이낸스 현물 , 바이낸스 선물의 교집합을 가진 심볼명만 가져옴
            sym_name = sym_intersection_D()  # 업비트 현물 , 바이낸스 현물 , 바이낸스 선물의 교집합을 가진 심볼명만 가져옴

            Usdt = upbit_get_usd_krw()  # 업비트에서 1달러당 환율 가져옴
            sym_name = delet_None_network(all_coin_info, sym_name)
            # 시간 줄이기 위함, 각 자본마다 심볼명 저장해두고 그것중에서 선택 => 전송수수료가 싸진 코인은 나중에서야 알수있는 단점이 있음
            asset_division[i]['sym_name'] = remove_sym_name_1(all_coin_info, sym_name, Usdt, asset_division[i]['up_asset'] / asset_division[i]["future_leverage"], 0.005, User_data['ban_list'], User_data['up_market_fee'])
            asset_division[i]["State"], asset_division[i]['sym_name[a]'], asset_division[i]['point_of_buy_UP_ticker_1'], asset_division[i]['point_of_buy_UP_amount_1'], asset_division[i]['point_of_Withdrow_UP_amount_1'], asset_division[i]['bin_address'], asset_division[i]['bin_Memo'], asset_division[i]['future_leverage'], asset_division[i]['up_Min_Withdraw_Size[a]'], asset_division[i]['x1'], asset_division[i]['Usdt_1_1'], asset_division[i]['step_1_1_up_krw_asset'], asset_division[i]['rest_step_1_1_up_krw_asset'], asset_division[i]['up_withdrawfee[a]'], asset_division[i]['expect_x2'], asset_division[i]['Bin_Spot_Minimum_Order_Size[a]'], asset_division[i]['up_withdrawfee_by_asset[a]']  = step_1_1(asset_division[i]['sym_name'], asset_division[i]['up_asset'], User_data['up_market_fee'], User_data['bin_spot_market_fee'], asset_division[i]['div_unit'] ,User_data['bin_Coin_M_future_market_fee'], asset_division[i]['want_premium_step_1_1'])
            print(i,"step_1_1")
            time.sleep(7)
            if (asset_division[i]["State"] == 'step_1_1'):  # 애초에 포지션 진입을 안하니까 이런식으로 작성함
                print(22)
                asset_division[i]["State"], asset_division[i]["보내기 전 바이낸스잔고"] = send_up_to_bin(asset_division[i]["sym_name[a]"], asset_division[i]['point_of_Withdrow_UP_amount_1'], asset_division[i]['bin_address'], asset_division[i]['bin_Memo'])  # 업비트에서 바이낸스 전송주문 요청
                print(i,"send_up_to_bin")
                time.sleep(7)
            
    for i in range(len(asset_division)):
        if (asset_division[i]["State"] == 'step_1_1'):  # 애초에 포지션 진입을 안하니까 이런식으로 작성함
            print(29)
            asset_division[i]["State"], asset_division[i]["보내기 전 바이낸스잔고"], asset_division[i]["point_of_Withdrow_UP_amount_1"] = send_up_to_bin_2(asset_division[i]["sym_name[a]"], asset_division[i]['point_of_Withdrow_UP_amount_1'], asset_division[i]['bin_address'], asset_division[i]['bin_Memo'],asset_division[i]["Bin_Spot_Minimum_Order_Size[a]"])  # 업비트에서 바이낸스 전송주문 요청
            print(i,"send_up_to_bin_2")
            
        if (asset_division[i]["State"] == 'send_up_to_bin'):
            print(34)
            asset_division[i]["State"] = send_up_to_bin_Completion(asset_division[i]["sym_name[a]"], asset_division[i]["보내기 전 바이낸스잔고"], asset_division[i]["point_of_Withdrow_UP_amount_1"],asset_division[i]['Bin_Spot_Minimum_Order_Size[a]'])
            print(i,"send_up_to_bin_Completion")
        #         time.sleep(5)

        if (asset_division[i]["State"] == 'send_up_to_bin_Completion'):
            print(40)
            asset_division[i]["State"], asset_division[i]["x2"], asset_division[i]["step_1_2_bin_spot_usdt_asset"], asset_division[i]["after_step_1_2_bin_spot_usdt_asset"], asset_division[i]["before_step_1_2_bin_spot_usdt_asset"], asset_division[i]["Usdt_1_2"], asset_division[i]["enter_servertime"], asset_division[i]["sym_name_D"], asset_division[i]["bin_spot_to_coin_M_order_amount"], asset_division[i]["step_2_1_bin_spot_usdt_asset"], asset_division[i]["rest_step_2_1_bin_spot_usdt_asset"], asset_division[i]["point_of_buy_bin_spot_amount_2"], asset_division[i]["point_of_buy_bin_spot_ticker_2"], asset_division[i]['Usdt_2_1'], asset_division[i]['real_enter_amount_by_asset_ratio'], asset_division[i]['datetime'], asset_division[i]['point_of_buy_bin_fu_ticker_1'], asset_division[i]['point_of_buy_bin_fu_amount_1'], asset_division[i]['bin_fu_D_Cont_order'], asset_division[i]['Coin_M_Cont[idx_num]'], asset_division[i]['real_enter_amount'], asset_division[i]['bin_fu_D_Order_ID'], asset_division[i]['cheak_fundig_fee_severtime'], asset_division[i]['Avbl_amount'], asset_division[i]['point_of_buy_bin_fu_amount_1_D'], asset_division[i]['point_of_buy_bin_fu_ticker_1_D'], asset_division[i]['point_of_buy_bin_fu_commission_1_D'] = step_1_2(asset_division[i]["up_asset"], asset_division[i]["sym_name[a]"], asset_division[i]["point_of_buy_UP_ticker_1"], asset_division[i]["Bin_Spot_Minimum_Order_Size[a]"],asset_division[i]["x1"], asset_division[i]["point_of_Withdrow_UP_amount_1"], asset_division[i]["Usdt_1_1"], asset_division[i]["div_unit"], User_data['ban_list'],User_data['bin_spot_market_fee'])
            print(i,"step_1_2")
            time.sleep(14)

        if (asset_division[i]["State"] == "step_1_2"):
            print(47)
            asset_division[i]["State"], asset_division[i]["enter_servertime"], asset_division[i]["sym_name_D"], asset_division[i]["bin_spot_to_coin_M_order_amount"], asset_division[i]["step_2_1_bin_spot_usdt_asset"], asset_division[i]["rest_step_2_1_bin_spot_usdt_asset"], asset_division[i]["point_of_buy_bin_spot_amount_2"], asset_division[i]["point_of_buy_bin_spot_ticker_2"], asset_division[i]["Usdt_2_1"], asset_division[i]["real_enter_amount_by_asset_ratio"], asset_division[i]["datetime"], asset_division[i]["point_of_buy_bin_fu_ticker_1"], asset_division[i]["point_of_buy_bin_fu_amount_1"], asset_division[i]['bin_fu_D_Cont_order'], asset_division[i]['Coin_M_Cont[idx_num]'], asset_division[i]['real_enter_amount'], asset_division[i]['bin_fu_D_Order_ID'], asset_division[i]['cheak_fundig_fee_severtime'], asset_division[i]['Avbl_amount'], asset_division[i]['point_of_buy_bin_fu_amount_1_D'], asset_division[i]['point_of_buy_bin_fu_ticker_1_D'], asset_division[i]['point_of_buy_bin_fu_commission_1_D'] = step_2_1(asset_division[i]["step_1_2_bin_spot_usdt_asset"] + asset_division[i]['binance_Alpha_asset'], User_data['bin_spot_market_fee'],User_data['ban_list'])
            print(i,"step_2_1")
            time.sleep(14)#심볼명 두배..coim m 레버리지 다른곳에서 조정하기
    print("멈추기 위해서 타임슬립")
    time.sleep(5)
    statuses = ("한싸이클 돌고 디스플레이 초기화",asset_division)
    Save_ALL_DATA()
    
    if (len(statuses) != 0):
        clear_output(wait=True)
        display(statuses)
        
    All_States = [asset_division[i]["State"] for i in range(len(asset_division))]  # 전부 coim_M에 있는 상태여야 종료
    if (len(set(All_States) & {'None_position', 'step_1_1', 'send_up_to_bin', 'send_up_to_bin_Completion', "step_1_2"}) == 0):
        break
Save_ALL_DATA()

('한싸이클 돌고 디스플레이 초기화',
 {0: {'up_asset': 200000,
   'binance_asset': 168.7051876845213,
   'future_leverage': 1,
   'State': 'step_2_1',
   'Usdt_first': 1185.5,
   'save_state': {0: [0,
     200000,
     175352.32000000004,
     '2022Y_1M_18D_19h_18m_42s',
     0,
     'DOGE',
     851.148,
     0.1896,
     0.22567319999998858,
     'ALGO',
     9.86999999999999,
     1.3356,
     200000,
     1,
     1185.5,
     1280.0000467633577,
     156.1796875,
     'ICX',
     200009.955,
     -9.954999999987194,
     'ㅇㅇ',
     156.3760397,
     1188.0,
     161.76487319999998,
     1642179991211,
     1188.0,
     'DOGE',
     852.0,
     161.5392,
     0.22567319999998858,
     851.148,
     0.1896,
     1188.0,
     0.9901764305428871,
     1642179991211,
     0.189395,
     -16.0,
     16,
     10.0,
     844.7908086760016,
     4391185347,
     4.31537541,
     0.189395,
     791.99556482,
     0.39599778,
     'ALGO',
     1.3356,
     119.88,
     110.01,
     110.0,
     1.0,
     119

0 200000 step_2_1
1 197062.46885286266 step_2_1
====================asset_division============================
{0: {'up_asset': 200000, 'binance_asset': 168.7051876845213, 'future_leverage': 1, 'State': 'step_2_1', 'Usdt_first': 1185.5, 'save_state': {0: [0, 200000, 175352.32000000004, '2022Y_1M_18D_19h_18m_42s', 0, 'DOGE', 851.148, 0.1896, 0.22567319999998858, 'ALGO', 9.86999999999999, 1.3356, 200000, 1, 1185.5, 1280.0000467633577, 156.1796875, 'ICX', 200009.955, -9.954999999987194, 'ㅇㅇ', 156.3760397, 1188.0, 161.76487319999998, 1642179991211, 1188.0, 'DOGE', 852.0, 161.5392, 0.22567319999998858, 851.148, 0.1896, 1188.0, 0.9901764305428871, 1642179991211, 0.189395, -16.0, 16, 10.0, 844.7908086760016, 4391185347, 4.31537541, 0.189395, 791.99556482, 0.39599778, 'ALGO', 1.3356, 119.88, 110.01, 110.0, 1.0, 1191.5, 'Second_Use_Bin_Spot_Usdt', 10.0, 0.01, -0.5761908, 161.4243816, -24647.679999999964, 0.8767616000000001, 175362.27500000002, 147.63911036508603, 1191.5], 1: []}, 'binance_Alpha

In [21]:
asset_division

{0: {'up_asset': 1000000,
  'binance_asset': 834.7245409015025,
  'future_leverage': 1,
  'State': 'step_2_1',
  'Usdt_first': 1198.0,
  'save_state': {0: []},
  'binance_Alpha_asset': 0,
  'qwer': '2022_1_25',
  'div_unit': 1000000,
  'want_premium_step_1_1': 0.015,
  'want_premium_step_2_2': 1.015,
  'sym_name': ['XLM',
   'THETA',
   'XRP',
   'IOST',
   'ZIL',
   'XEM',
   'SOL',
   'EOS',
   'ATOM',
   'WAVES',
   'ALGO',
   'HBAR'],
  'sym_name[a]': 'XLM',
  'point_of_buy_UP_ticker_1': 242.99999999999025,
  'point_of_buy_UP_amount_1': 4113.176954320001,
  'point_of_Withdrow_UP_amount_1': 4113.1669543,
  'bin_address': 'GAHK7EEG2WWHVKDNT4CEQFZGKF2LGDSW2IVM4S5DP42RBW3K6BTODB4A',
  'bin_Memo': '400293429',
  'up_Min_Withdraw_Size[a]': 1.0000000000000004e-07,
  'x1': 833.958756691581,
  'Usdt_1_1': 1198.5,
  'step_1_1_up_krw_asset': 1000001.75089971,
  'rest_step_1_1_up_krw_asset': -1.7508997099939734,
  'up_withdrawfee[a]': 0.01,
  'expect_x2': 982477.8423446973,
  'Bin_Spot_Minimum

In [22]:
Save_ALL_DATA()

0 1000000 step_2_1
1 1000000 step_2_1
2 1000000 step_2_1
====================asset_division============================
{0: {'up_asset': 1000000, 'binance_asset': 834.7245409015025, 'future_leverage': 1, 'State': 'step_2_1', 'Usdt_first': 1198.0, 'save_state': {0: []}, 'binance_Alpha_asset': 0, 'qwer': '2022_1_25', 'div_unit': 1000000, 'want_premium_step_1_1': 0.015, 'want_premium_step_2_2': 1.015, 'sym_name': ['XLM', 'THETA', 'XRP', 'IOST', 'ZIL', 'XEM', 'SOL', 'EOS', 'ATOM', 'WAVES', 'ALGO', 'HBAR'], 'sym_name[a]': 'XLM', 'point_of_buy_UP_ticker_1': 242.99999999999025, 'point_of_buy_UP_amount_1': 4113.176954320001, 'point_of_Withdrow_UP_amount_1': 4113.1669543, 'bin_address': 'GAHK7EEG2WWHVKDNT4CEQFZGKF2LGDSW2IVM4S5DP42RBW3K6BTODB4A', 'bin_Memo': '400293429', 'up_Min_Withdraw_Size[a]': 1.0000000000000004e-07, 'x1': 833.958756691581, 'Usdt_1_1': 1198.5, 'step_1_1_up_krw_asset': 1000001.75089971, 'rest_step_1_1_up_krw_asset': -1.7508997099939734, 'up_withdrawfee[a]': 0.01, 'expect_x2':

In [129]:
while True:  # coim_M => 업비트
    All_States_2 = [asset_division[i]["State"] for i in range(len(asset_division))]  # 모든 States를 가져옮
    for i in range(len(asset_division)):
        if(len(set(All_States_2) & {'step_2_2', 'send_bin_to_up', 'send_bin_to_up_Completion','step_3','final_fun'}) != 0):#교집합이 0일 경우(모든상태가 저기에 포함되지 않을경우)
            pass
        elif (asset_division[i]["State"] == "step_2_1"):
            asset_division[i]['sym_name'] = step_2_2_sym_name(asset_division[i]['step_2_1_bin_spot_usdt_asset'])
            asset_division[i]['State'], asset_division[i]['sym_name[b]'], asset_division[i]['point_of_buy_bin_spot_ticker_2_2'], asset_division[i]['point_of_buy_bin_spot_amount_2_2'], asset_division[i]['point_of_Withdrow_UP_amount_2'], asset_division[i]['real_send_amount'], asset_division[i]['up_add'], asset_division[i]['up_seadd'], asset_division[i]['Bin_Spot_Minimum_Order_Size[b]'], asset_division[i]['x4'], asset_division[i]['Usdt_2_2'], asset_division[i]['Second_Use_Bin_Spot_Usdt'], asset_division[i]['bin_Min_Withdraw_Size[b]'], asset_division[i]['bin_withdrawfee[b]'], asset_division[i]['step_2_2_bin_spot_usdt_asset'], asset_division[i]['rest_step_2_2_bin_spot_usdt_asset'], asset_division[i]['can_send_amount'] = step_2_2(asset_division[i]['up_asset'], asset_division[i]['sym_name'], asset_division[i]['sym_name_D'],asset_division[i]['point_of_buy_bin_fu_amount_1_D'], asset_division[i]['point_of_buy_bin_fu_ticker_1_D'],asset_division[i]['Avbl_amount'], asset_division[i]['point_of_buy_bin_spot_ticker_2'],asset_division[i]['point_of_buy_bin_spot_amount_2'], asset_division[i]['bin_fu_D_Cont_order'],asset_division[i]['div_unit'], asset_division[i]['want_premium_step_2_2'], User_data['up_market_fee'],User_data['bin_Coin_M_future_market_fee'], User_data['bin_spot_market_fee'])
            print(i,"step_2_2")
            if (asset_division[i]["State"] == 'step_2_2'):
                asset_division[i]['State'], asset_division[i]['보내기 전 업비트 잔고'] = send_bin_to_up(asset_division[i]['sym_name[b]'], asset_division[i]['up_add'], asset_division[i]['up_seadd'], asset_division[i]['real_send_amount'], asset_division[i]['point_of_Withdrow_UP_amount_2'])
                print(i,"send_bin_to_up")
#                 time.sleep()
            # @@ 가치변동에 따른 위험이 있음!!!! => 전송 후 매도 까지 한번에..?? => 혼자만 보내는 중에만 실행하도록 한다!(step_2_2가 step 2_2끝난 상태이거나, 전송 중이 아닐 시에만 실행하면 된다)
    for i in range(len(asset_division)):
        if (asset_division[i]["State"] == 'step_2_2'):
            asset_division[i]['State'], asset_division[i]['보내기 전 업비트 잔고'] = send_bin_to_up(asset_division[i]['sym_name[b]'], asset_division[i]['up_add'], asset_division[i]['up_seadd'], asset_division[i]['real_send_amount'], asset_division[i]['point_of_Withdrow_UP_amount_2'])
            print(i,"send_bin_to_up")
        if (asset_division[i]["State"] == 'send_bin_to_up'):
            asset_division[i]["State"] = send_bin_to_up_Completion(asset_division[i]["sym_name[b]"], asset_division[i]["real_send_amount"], asset_division[i]["보내기 전 업비트 잔고"], asset_division[i]['Bin_Spot_Minimum_Order_Size[b]'])
            print(i,"send_bin_to_up_Completion")
        if (asset_division[i]["State"] == 'send_bin_to_up_Completion'):
            asset_division[i]["State"], asset_division[i]["will_Realize_Arbitrage"], asset_division[i]["will_Realize_Arbitrage_rate"], asset_division[i]["step_3_up_krw_asset"], asset_division[i]["x4"], asset_division[i]["Usdt_step_3"] = step_3(asset_division[i]["up_asset"],asset_division[i]["sym_name[b]"],asset_division[i]["real_send_amount"],asset_division[i]['div_unit'],User_data['up_market_fee'])
            print(i,"step_3")
        if (asset_division[i]["State"] == 'step_3'):
            asset_division[i]['State'], asset_division[i]['save_state'], asset_division[i]['up_asset'], asset_division[i]['binance_Alpha_asset'] = final_fun(asset_division[i]['qwer'], asset_division[i]['save_state'], len(asset_division[i]['save_state']) - 1, i, asset_division[i]['up_asset'],asset_division[i]['step_3_up_krw_asset'], asset_division[i]['rest_step_1_1_up_krw_asset'], asset_division[i]['sym_name_D'],asset_division[i]['point_of_buy_bin_spot_amount_2'], asset_division[i]['point_of_buy_bin_spot_ticker_2'],asset_division[i]['rest_step_2_1_bin_spot_usdt_asset'], asset_division[i]['sym_name[b]'], asset_division[i]['can_send_amount'],asset_division[i]['point_of_Withdrow_UP_amount_2'], asset_division[i]['point_of_buy_bin_spot_ticker_2_2'], asset_division[i]['Usdt_first'],asset_division[i]['point_of_buy_UP_ticker_1'], asset_division[i]['point_of_buy_UP_amount_1'], asset_division[i]['sym_name[a]'],asset_division[i]['step_1_1_up_krw_asset'], asset_division[i]['up_withdrawfee_by_asset[a]'],asset_division[i]['point_of_Withdrow_UP_amount_1'], asset_division[i]['Usdt_1_1'], asset_division[i]['step_1_2_bin_spot_usdt_asset'],asset_division[i]['enter_servertime'], asset_division[i]['Usdt_1_2'], asset_division[i]['bin_spot_to_coin_M_order_amount'],asset_division[i]['step_2_1_bin_spot_usdt_asset'], asset_division[i]['Usdt_2_1'], asset_division[i]['real_enter_amount_by_asset_ratio'],asset_division[i]['enter_servertime'], asset_division[i]['point_of_buy_bin_fu_ticker_1'], asset_division[i]['point_of_buy_bin_fu_amount_1'],asset_division[i]['bin_fu_D_Cont_order'], asset_division[i]['Coin_M_Cont[idx_num]'], asset_division[i]['real_enter_amount'],asset_division[i]['bin_fu_D_Order_ID'], asset_division[i]['Avbl_amount'], asset_division[i]['point_of_buy_bin_fu_ticker_1_D'],asset_division[i]['point_of_buy_bin_fu_amount_1_D'], asset_division[i]['point_of_buy_bin_fu_commission_1_D'],asset_division[i]['point_of_buy_bin_spot_amount_2_2'], asset_division[i]['real_send_amount'],asset_division[i]['Bin_Spot_Minimum_Order_Size[b]'], asset_division[i]['Usdt_2_2'], asset_division[i]['Second_Use_Bin_Spot_Usdt'],asset_division[i]['bin_Min_Withdraw_Size[b]'], asset_division[i]['bin_withdrawfee[b]'], asset_division[i]['step_2_2_bin_spot_usdt_asset'],asset_division[i]['rest_step_2_2_bin_spot_usdt_asset'], asset_division[i]['x4'], asset_division[i]['Usdt_step_3'],asset_division[i]['future_leverage'])
            # 리턴 값 빼고 전부 초기화
            asset_division_keys = list(asset_division[i].keys())
            asset_division_keys = list(set(asset_division_keys) - {'State', 'save_state', 'up_asset', 'Usdt_first', 'save_state', 'binance_Alpha_asset', 'qwer', 'div_unit', 'up_market_fee', 'bin_spot_market_fee', 'bin_Coin_M_future_market_fee', 'want_premium_step_1_1','want_premium_step_2_2','future_leverage'})  # @@ 옵션 (option) 여기에 없애면 안될 변수들 저장.
            print(i,"final_fun")
            for j in range(len(asset_division_keys)):
                asset_division[i][asset_division_keys[j]] = asset_division_keys[j]
                
    print("멈추기 위해서 타임슬립")
    time.sleep(5)
    statuses = ("한싸이클 돌고 디스플레이 초기화",asset_division)
    Save_ALL_DATA()
    
    if (len(statuses) != 0):
        clear_output(wait=True)
        display(statuses)

    All_States_2 = [asset_division[i]["State"] for i in range(len(asset_division))]  # 전부 coim_M에 있는 상태여야 종료
    if (len(set(All_States_2) & {'step_2_1', 'step_2_2', 'send_bin_to_up', 'send_bin_to_up_Completion', 'step_3', 'final_fun'}) == 0):
        break
Save_ALL_DATA()

('바이낸스 => 업비트', 'XLM', '전송 중', 6e-08, '현재시각', 'Tue May 10 18:07:49 2022')

0 send_bin_to_up_Completion
멈추기 위해서 타임슬립


KeyboardInterrupt: 

In [128]:
Save_ALL_DATA()

0 1000000 send_bin_to_up
1 1000000 step_2_1
2 1000000 step_2_1
====================asset_division============================
{0: {'up_asset': 1000000, 'binance_asset': 834.7245409015025, 'future_leverage': 1, 'State': 'send_bin_to_up', 'Usdt_first': 1198.0, 'save_state': {0: []}, 'binance_Alpha_asset': 0, 'qwer': '2022_1_25', 'div_unit': 1000000, 'want_premium_step_1_1': 0.015, 'want_premium_step_2_2': 1.015, 'sym_name': ['XEM', 'IOST', 'THETA', 'ATOM', 'NEAR', 'ZIL', 'XRP', 'HBAR', 'SOL', 'XLM', 'VET', 'DOT', 'WAVES', 'MATIC', 'EOS', 'KAVA', 'ALGO'], 'sym_name[a]': 'XLM', 'point_of_buy_UP_ticker_1': 242.99999999999025, 'point_of_buy_UP_amount_1': 4113.176954320001, 'point_of_Withdrow_UP_amount_1': 4113.1669543, 'bin_address': 'GAHK7EEG2WWHVKDNT4CEQFZGKF2LGDSW2IVM4S5DP42RBW3K6BTODB4A', 'bin_Memo': '400293429', 'up_Min_Withdraw_Size[a]': 1.0000000000000004e-07, 'x1': 833.958756691581, 'Usdt_1_1': 1198.5, 'step_1_1_up_krw_asset': 1000001.75089971, 'rest_step_1_1_up_krw_asset': -1.750899

In [124]:
i = 0 

asset_division[i]['State'], asset_division[i]['보내기 전 업비트 잔고'] = 'send_bin_to_up',upbit.get_balance(asset_division[i]['sym_name[b]'])

In [89]:
def send_bin_to_up(send_to_up_optimal_simbol, up_add, up_seadd, real_send_amount, point_of_Withdrow_UP_amount_2):  # 바이낸스에서 업비트 전송주문 요청

    send_state_2 = [up_add, up_seadd, real_send_amount, "전송실패"]
    # 바 => 업 전송, 오류날 경우를 대비
    quotePrecision = 8
    point_of_Withdrow_UP_amount_2 = float("{:0.0{}f}".format(point_of_Withdrow_UP_amount_2, quotePrecision))
    
    try:
        if (send_to_up_optimal_simbol == 'BHC'):  # BCH코인은 주소가 좀 이상하게 나옴, 그리고 태그도 없어야 전송이 됨.
            up_add = up_add[12:]
            res = client_spot.withdraw(coin=send_to_up_optimal_simbol, amount=point_of_Withdrow_UP_amount_2, address=up_add, addressTag=up_seadd)
            # addressTag=up_seadd)

            send_state_2 = res
            print(res)
            return 'send_bin_to_up', upbit.get_balance(send_to_up_optimal_simbol)
            # 성공시 send_bin_to_up

        if (up_seadd == None):

            res = client_spot.withdraw(coin=send_to_up_optimal_simbol, amount=point_of_Withdrow_UP_amount_2, address=up_add, )
            # 두번째 주소가 없는경우 그 양식에 맞게 전송주문을 제출 하여야 함.
            # addressTag=up_seadd)

            send_state_2 = res
            print(res)
            return 'send_bin_to_up', upbit.get_balance(send_to_up_optimal_simbol)
            # 성공시 send_bin_to_up

        else:
            # 바이낸스 코인 전송

            res = client_spot.withdraw(coin=send_to_up_optimal_simbol, amount=point_of_Withdrow_UP_amount_2, address=up_add, addressTag=up_seadd)
            send_state_2 = res
            print(res)
            return 'send_bin_to_up', upbit.get_balance(send_to_up_optimal_simbol)
            # 성공시 send_bin_to_up

    except:

        statuses = (send_state_2)

        if (len(statuses) != 0):
            clear_output(wait=True)
            display(statuses)
        return 'step_2_2', "보내기 전 업비트 잔고"

In [127]:
i = 0
if (asset_division[i]["State"] == 'send_bin_to_up'):
    asset_division[i]["State"] = send_bin_to_up_Completion(asset_division[i]["sym_name[b]"], asset_division[i]["real_send_amount"], asset_division[i]["보내기 전 업비트 잔고"], asset_division[i]['Bin_Spot_Minimum_Order_Size[b]'])

('바이낸스 => 업비트', 'XLM', '전송 중', 6e-08, '현재시각', 'Tue May 10 16:53:03 2022')

In [ ]:
i =  0
asset_division[i]['sym_name'] = step_2_2_sym_name(asset_division[i]['step_2_1_bin_spot_usdt_asset'])
asset_division[i]['State'], asset_division[i]['sym_name[b]'], asset_division[i]['point_of_buy_bin_spot_ticker_2_2'], asset_division[i]['point_of_buy_bin_spot_amount_2_2'], asset_division[i]['point_of_Withdrow_UP_amount_2'], asset_division[i]['real_send_amount'], asset_division[i]['up_add'], asset_division[i]['up_seadd'], asset_division[i]['Bin_Spot_Minimum_Order_Size[b]'], asset_division[i]['x4'], asset_division[i]['Usdt_2_2'], asset_division[i]['Second_Use_Bin_Spot_Usdt'], asset_division[i]['bin_Min_Withdraw_Size[b]'], asset_division[i]['bin_withdrawfee[b]'], asset_division[i]['step_2_2_bin_spot_usdt_asset'], asset_division[i]['rest_step_2_2_bin_spot_usdt_asset'], asset_division[i]['can_send_amount'] = step_2_2(asset_division[i]['up_asset'], asset_division[i]['sym_name'], asset_division[i]['sym_name_D'],asset_division[i]['point_of_buy_bin_fu_amount_1_D'], asset_division[i]['point_of_buy_bin_fu_ticker_1_D'],asset_division[i]['Avbl_amount'], asset_division[i]['point_of_buy_bin_spot_ticker_2'],asset_division[i]['point_of_buy_bin_spot_amount_2'], asset_division[i]['bin_fu_D_Cont_order'],asset_division[i]['div_unit'], asset_division[i]['want_premium_step_2_2'], User_data['up_market_fee'],User_data['bin_Coin_M_future_market_fee'], User_data['bin_spot_market_fee'])
print(i,"step_2_2")
if (asset_division[i]["State"] == 'step_2_2'):
    asset_division[i]['State'], asset_division[i]['보내기 전 업비트 잔고'] = send_bin_to_up(asset_division[i]['sym_name[b]'], asset_division[i]['up_add'], asset_division[i]['up_seadd'], asset_division[i]['real_send_amount'], asset_division[i]['point_of_Withdrow_UP_amount_2'])
    print(i,"send_bin_to_up")

In [39]:
# 다시 넣어줘야 할 값들
i = 0
asset_division[i]['step_2_1_bin_spot_usdt_asset'] = asset_division[i]['binance_asset']

In [41]:
# 함수 하나씩 잘 돌아가는지 
i = 0
asset_division[i]['sym_name'] = step_2_2_sym_name(asset_division[i]['step_2_1_bin_spot_usdt_asset'])

55 55
BCH 3.463 0.0002884784 240.8
XEM 13156.0 0.0003038128 0.0634
IOST 39766.0 2.5122060000000002e-05 0.02097
THETA 447.0 0.00026825616000000004 1.866
SC 127311.0 7.846900000000001e-07 0.00655
ATOM 57.95 8.61961e-05 14.39
NEAR 78.9 0.00012654474 10.563
ZIL 11042.1 1.8094592000000003e-05 0.07552
ONT 2136.0 0.00046769920000000004 0.3904
XRP 1575.0 0.0001585553 0.5294
QTUM 169.1 5.9049420000000006e-05 4.929
DOGE 7294.0 0.0006852560000000001 0.1144
HBAR 7091.0 0.0001408848 0.1176
ETH 0.3472 0.004624145824 2412.43
SOL 11.71 0.0008538146 71.27
AVAX 17.42 0.0005737222 47.89
XTZ 373.5 0.00026751340000000007 2.233
BTC 0.02604 0.007729956032000001 32261.92
0.007729956032000001 자본대비 수수료가 큰 코인
8224.8538848
ETC 33.19 0.0003010574 25.13
LTC 10.096 9.89548e-05 82.6
XLM 5418.0 3.687444000000001e-06 0.1539
NEO 55.99 0.0 14.74
CELO 393.9 2.5361660000000003e-06 2.117
IOTA 1922.0 0.0002598462 0.4338
VET 19645.5 0.0010180604 0.04249
ADA 1240.9 0.0008056550000000001 0.6725
DOT 69.99 0.0014292140000000003 1

In [43]:
i = 0
up_asset,sym_name, sym_name_D_ ,point_of_buy_bin_fu_amount_1_D, point_of_buy_bin_fu_ticker_1_D, Avbl_amount , point_of_buy_bin_spot_ticker_2, point_of_buy_bin_spot_amount_2, bin_fu_D_Cont_order, div_unit, want_premium_step_2_2, up_market_fee, bin_Coin_M_future_market_fee, bin_spot_market_fee = asset_division[i]['up_asset'], asset_division[i]['sym_name'], asset_division[i]['sym_name_D'],asset_division[i]['point_of_buy_bin_fu_amount_1_D'], asset_division[i]['point_of_buy_bin_fu_ticker_1_D'],asset_division[i]['Avbl_amount'], asset_division[i]['point_of_buy_bin_spot_ticker_2'],asset_division[i]['point_of_buy_bin_spot_amount_2'], asset_division[i]['bin_fu_D_Cont_order'],asset_division[i]['div_unit'], asset_division[i]['want_premium_step_2_2'], User_data['up_market_fee'],User_data['bin_Coin_M_future_market_fee'], User_data['bin_spot_market_fee']

In [ ]:

asset_division[i]['State'], asset_division[i]['sym_name[b]'], asset_division[i]['point_of_buy_bin_spot_ticker_2_2'], asset_division[i]['point_of_buy_bin_spot_amount_2_2'], asset_division[i]['point_of_Withdrow_UP_amount_2'], asset_division[i]['real_send_amount'], asset_division[i]['up_add'], asset_division[i]['up_seadd'], asset_division[i]['Bin_Spot_Minimum_Order_Size[b]'], asset_division[i]['x4'], asset_division[i]['Usdt_2_2'], asset_division[i]['Second_Use_Bin_Spot_Usdt'], asset_division[i]['bin_Min_Withdraw_Size[b]'], asset_division[i]['bin_withdrawfee[b]'], asset_division[i]['step_2_2_bin_spot_usdt_asset'], asset_division[i]['rest_step_2_2_bin_spot_usdt_asset'], asset_division[i]['can_send_amount'] = step_2_2(asset_division[i]['up_asset'], asset_division[i]['sym_name'], asset_division[i]['sym_name_D'],asset_division[i]['point_of_buy_bin_fu_amount_1_D'], asset_division[i]['point_of_buy_bin_fu_ticker_1_D'],asset_division[i]['Avbl_amount'], asset_division[i]['point_of_buy_bin_spot_ticker_2'],asset_division[i]['point_of_buy_bin_spot_amount_2'], asset_division[i]['bin_fu_D_Cont_order'],asset_division[i]['div_unit'], asset_division[i]['want_premium_step_2_2'], User_data['up_market_fee'],User_data['bin_Coin_M_future_market_fee'], User_data['bin_spot_market_fee'])